# STAR - Statistical Toolkit for Analysis of Radiotherapy Treatments

This is an iPython notebook. It consists of multiple cells, which can either be markdown/text or executable code. You will have to run these code cells in the correct order, for example, you will need to run the cells containing the functions, before you can use them. If you run a cell, the output, if there is one, will be displayed below it.

This notebook is split up into three main sections, which have to be run in order:
1. **Support Files and SQL:** Define the locations of the additional files needed to run this analysis and, if needed, set up the SQL connection.
2. **Functions:** Defines all functions, has to be run before the analysis can be performed.
3. **Function Calls:** The function calls needed to run the complete analysis

## Packages, Support Files and SQL

### Packages

Before you can run this script for the first time, you have to set up a Python environment with all needed packages. Best practice is creating a fresh virtual environment for this. Then run the following command, which will install all the packages from the "requirements.txt" file in the exact version which has been tested by us.

*pip install -r requirements.txt*

Alternatively, you can try to install the latest versions using 

*pip install plotly kaleido==0.1.0post1 pydicom numpy pandas scipy nbformat openpyxl scikit-learn python-gdcm pyodbc statsmodels trimesh scikit-image*

Afterwards, restart the kernel and import the packages below. The import is always the first thing to run before the analysis.

In [ ]:
import pydicom
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linprog
from scipy.ndimage import binary_dilation, binary_erosion, binary_fill_holes, zoom, distance_transform_edt
from scipy.interpolate import interpn
import pandas as pd
import os
import warnings
import glob
from sklearn.decomposition import PCA
import logging
import shutil
import gzip
import pyodbc
import json
import statsmodels.api as sm

### Variables/Supporting Files

The following three sheets need to be supplied by you for the analysis to run:
- **"DICOMLocations"** contains the locations of the DICOM files for which the analysis should be performed. For our SQL database, we need the identifiers 'PMRN', 'PlanSetupSer','StructureSet_UID','RTPlan_UID' to find the correct locations. This will likely be different for your center. In the end, you will need to have a "Filename_Updated.pickle" with the columns 'PMRN', 'PlanSetupSer', 'StructureSet_UID', 'RTPlan_UID', 'Dose_Path', 'StructureSet_Path', 'Plan_Path', which is generated from our SQL database using the "updateLocs" function.
- **"PTVInfoSheet"** is an Excel sheet which links the Plan ID and PTV Structure name to other information like site. In our case, it contains the following columns, but in general it could be any information. Important is, that the identifiers, PlanSetupSer, match in all files. The PTV names will also be used to only analyze these PTVs and not any additional ones in the plan, if wanted. It can be way longer than the DICOMLocations list.
PlanSetupSer	PMRN	Patient	Course	PrescriptionSer	Prescription	Rx site	Rx technique	Plan	Num Fractions	Dose/Frc (cGy)	DVH site	DVH modality	Content Date	PTV name	PTV volume (cc)	Min dose (cGy)	Max dose (cGy)	Mean dose (cGy)
- **"CGTVInfoSheet"** is similar to above for CTV and GTV. Contains Columns:
#PlanSetupSer	PMRN	Patient	Course	PrescriptionSer	Prescription	Rx site	Rx technique	Plan	Num Fractions	Dose/Frc (cGy)	DVH site	DVH modality	Content Date	Target name	Target volume (cc)	Min dose (cGy)	Max dose (cGy)	Mean dose (cGy)	

The Timmerman files are provided by us, but might need to be adapted to fit your OAR naming conventions:
- **"TimmermanSheet"** is a CSV sheet containing simplified constraints from the Timmerman tables
- **"TimmermanDict"** is a dictionary supplying search terms to match OAR structure names to constraints

The program generates a coding dictionary when it anonymizes plan identifiers on the first run. If you already have one generated, you need to load it by specifying the location in the  **"CodingDict"** variable.

Last but not least, **"SaveFolder"** defines the directory, where all results, namely pickles and plots, will be stored.

In [ ]:
DICOMLocations="//Cifs2/imrt_qa$/EIT/Marvin/STAR/DicomStructures_Joe_10kto100.xlsx"
PTVInfoSheet="//Cifs2/imrt_qa$/EIT/Marvin/STAR/PTV_stats_Verifier_2.xlsx"
CGTVInfoSheet="//Cifs2/imrt_qa$/EIT/Marvin/STAR/CTVGTV_stats_Verifier.xlsx"
TimmermanSheet="//Cifs2/imrt_qa$/EIT/Marvin/STAR/Timmerman Tables/Timmerman.csv"
TimmermanDict="//Cifs2/imrt_qa$/EIT/Marvin/STAR/Timmerman Tables/TimmermanSearchTerms.json"
with open(TimmermanDict, 'r') as fp:
    TimmermanDict = json.load(fp)
TimmermanDict = {k: v for k, v in TimmermanDict.items()}
# CodingDict='//Cifs2/imrt_qa$/EIT/Marvin/STAR/AnonymizationDictPlan,Struc.json'
SaveFolder='//Cifshd/homedir$/Evaluation/FinalTest'

### SQL Connection

Our center has a DICOM storage and a SQL database, which keeps track of the locations of the DICOM files in that storage.
Thus, this connection is only useful, if your center has a SQL database with DICOM locations. You will likely need to adapt the "*updateLocs*" function below to perform correct queries for your database. If there is no SQL database that you can use, you will need generate an Excel sheet with locations of the DICOM files that you want to analyze yourself. It should contain the following columns:

'PMRN', 'PlanSetupSer', 'StructureSet_UID', 'RTPlan_UID', 'Dose_Path', 'StructureSet_Path', 'Plan_Path'

You will have to install Microsoft SQL driver first for this connection to work reliably! 
https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server

SQLConnectString.txt should look like this, with ..... filled in with your data:

Driver={SQL Server};  
Server=.....;  
Database=.....;  
uid=.....;pwd=.....

You need to establish this connection once everytime you start the kernel, and afterwards the functions will make use of it.

In [ ]:
with open('//Cifs2/imrt_qa$/EIT/Marvin/XLSX Files/SQLConnectString.txt','r') as fp:
    cnxn = pyodbc.connect(fp.read())

## Functions (Run this section)

This section defines all the functions used to run the analysis. You can just run it completely and move on to the function calls, or, if you need to change something, dive in.

### Helper Functions

In [19]:
def updateLocs(locsheet=DICOMLocations,debug=False):
    """
    Connect to SQL database and query dose, structure and plan DICOM file locations using UIDs. Connection cnxn has to be established beforehand, otherwise it tries to load old locations.
    This obviously needs to be adapted to your center. 

    Output location pickle contains colums:
    'PMRN', 'PlanSetupSer', 'StructureSet_UID', 'RTPlan_UID', 'Dose_Path', 'StructureSet_Path', 'Plan_Path', 'ExportedDateTime'
    """
    if debug or not 'cnxn' in globals():
        print('Read stored locations.')
        locs=pd.read_pickle(f"{locsheet}_Updated.pickle")
    else:
        locs=pd.read_excel(locsheet)[['PMRN', 'PlanSetupSer','StructureSet_UID','RTPlan_UID']]
        dfSQL = pd.read_sql_query("SELECT CONCAT(ShareLocation, FilePath) AS Dose_Path, rtdosefile.RefRTPlanUID AS RTPlan_UID FROM dbo.rtdosefile JOIN dbo.pathroot ON rtdosefile.PathRootIDX = pathroot.PathRootIDX WHERE DoseSumType = 'PLAN'", cnxn)
        dfSQL2 = pd.read_sql_query("SELECT CONCAT(ShareLocation, FilePath) AS StructureSet_Path, structuresetfile.InstanceUID AS StructureSet_UID FROM dbo.structuresetfile JOIN dbo.pathroot ON structuresetfile.PathRootIDX = pathroot.PathRootIDX", cnxn)
        dfSQL3 = pd.read_sql_query("SELECT CONCAT(ShareLocation, FilePath) AS Plan_Path, rtplanfile.InstanceUID AS RTPlan_UID, rtplanfile.ExportedDateTime as ExportedDateTime FROM dbo.rtplanfile JOIN dbo.pathroot ON rtplanfile.PathRootIDX = pathroot.PathRootIDX", cnxn)
        locs=pd.merge(locs,dfSQL,on='RTPlan_UID',how='left')
        locs=pd.merge(locs,dfSQL2,on='StructureSet_UID',how='left')
        locs=pd.merge(locs,dfSQL3,on='RTPlan_UID',how='left')
        pd.to_pickle(locs,f"{locsheet}_Updated.pickle")
    return locs

def get_points(structs,calccntrs = False, CT = False):
    """
    Helper function to convert DICOM structures to numpy arrays. 
    Can also return the matching CT ids as a dictionary and the centers of the contours, if needed.
    """
    #Initialize return arrays, if needed.
    pts = np.array([])
    if calccntrs: 
            cntrs = np.array([])
            cntr = np.array([])
    if CT:
        CTids = dict()
    #Go through all contour layers
    for struc in structs:
        if calccntrs:
            cntrsstruct = np.array([])
        #and concatenate them into one array
        for cc in struc.ContourSequence:
            pts= np.concatenate((pts,cc.ContourData))
            if calccntrs:
                #print(cc.ContourImageSequence[0].ReferencedSOPInstanceUID)
                cntrsstruct = np.concatenate((cntrsstruct,np.mean(np.array(cc.ContourData).reshape(-1,3)[:,:2],axis=0),[pts[-1]]))
            if CT:
                CTids[pts[-1]] = cc.ContourImageSequence[0].ReferencedSOPInstanceUID #RS21.ROIContourSequence[0].ContourSequence[0].ContourImageSequence[0].ReferencedSOPInstanceUID
        if calccntrs: 
            cntr = np.concatenate((cntr,np.mean(cntrsstruct.reshape(-1,3),axis=0)))
            cntrs = np.concatenate((cntrs,cntrsstruct))
    if calccntrs: 
        if len(cntr)>3:
            return pts.reshape(-1,3), cntrs.reshape(-1,3), cntr.reshape(-1,3)
        else:
            return pts.reshape(-1,3), cntrs.reshape(-1,3), cntr[np.newaxis,:]
    else: 
        if CT:
            return pts.reshape(-1,3), CTids
        else:    
            return pts.reshape(-1,3), _

def subsample(PTVslice):
    """
    Helper function to linearly insert new contour points between existing ones. Useful, when the voxel grid is tighter than the point spacing of the contours.
    """
    half=(PTVslice+np.roll(PTVslice,1,0))/2
    quarter1=(PTVslice+half)/2
    quarter2 = (half+np.roll(PTVslice,1,0))/2
    # Adding an additional level of points
    eighth1 = (PTVslice + quarter1) / 2
    eighth2 = (quarter1 + half) / 2
    eighth3 = (half + quarter2) / 2
    eighth4 = (quarter2 + np.roll(PTVslice, 1, 0)) / 2
    st1 = (PTVslice + eighth1) / 2
    st2 = (eighth1 + quarter1) / 2
    st3 = (quarter1 + eighth2) / 2
    st4 = (eighth2 + half) / 2
    st5 = (half + eighth3) / 2
    st6 = (eighth3 + quarter2) / 2
    st7 = (quarter2 + eighth4) / 2
    st8 = (eighth4 + np.roll(PTVslice, 1, 0)) / 2
    PTVslice=np.concatenate((half,quarter1,quarter2, eighth1, eighth2, eighth3, eighth4, st1, st2, st3, st4, st5, st6, st7, st8))
    return PTVslice
       

def interpolate_contour(TVpts,z,debug=False):
    """
    Sample contour tighter with points. Useful, when the voxel grid is tighter than the point spacing of the contours.
    """
    #Chose points belonging to slice
    mask = np.isclose(TVpts[:,2],z)
    if np.any(mask):
        PTVslice=TVpts[mask][:,:2]#+spacing/2
        if debug:
            fig0=px.scatter(x=PTVslice[:,1],y=PTVslice[:,0])
            fig0.show()
        #Fill gaps, get better sampling of contour
        half=(PTVslice+np.roll(PTVslice,1,0))/2
        #Filter for disconnected contours
        valid=(np.linalg.norm(half-PTVslice,axis=1))<2*np.median(np.linalg.norm(PTVslice-np.roll(PTVslice,1,0),axis=1))
        if np.all(valid):
            PTVslice=np.concatenate((PTVslice,subsample(PTVslice)))
        else:
            PTVsliceall=PTVslice.copy()
            splits=np.arange(0,len(valid))[~valid]
            for i in range(len(splits)):
                if i==len(splits)-1:
                    upper=len(valid)+1
                else:
                    upper=splits[i+1]
                PTVslice=np.concatenate((PTVslice,subsample(PTVsliceall[splits[i]:upper])))
        if debug:
            print(np.all(valid))
            fig1=px.scatter(x=PTVslice[:,1],y=PTVslice[:,0])
            fig1.show()
            print(np.max(PTVslice),np.min(PTVslice))
        return PTVslice
    else:
        return False

def to_grid(TVpts,z,origin,spacing,size,debug=False):
    """
    Turn contour into a 2d boolean grid using specified dimensions and coordinates.
    """
    PTVgrid=np.full(size, False)
    PTVslice= interpolate_contour(TVpts,z,debug)
    if np.any(PTVslice):
        PTVslice = coordinate_space(PTVslice,origin,spacing)
        PTVgrid[PTVslice[:,1],PTVslice[:,0]] = True #y is row, x is column
        if debug:
            print(np.max(PTVslice),np.min(PTVslice))
            print(PTVslice.shape,PTVgrid.shape)
            fig2=px.scatter(x=PTVslice[:,0],y=PTVslice[:,1])
            fig2.show()
            fig=px.imshow(PTVgrid)
            fig.show()
            input()
    return PTVgrid

def coordinate_space(pt,origin,spacing,start=False):
    """
    Convert contour point coordinates in DICOM space into array indices 
    """
    if start:
        r= np.floor((pt-origin[:len(spacing)])/(spacing*np.array([2,2,1])[:len(spacing)])).astype(int)*np.array([2,2,1])[:len(spacing)]
    else:
        if len(pt.shape)>2:
            r= np.floor((pt-origin[np.newaxis,:len(spacing),np.newaxis,np.newaxis])/spacing[np.newaxis,:,np.newaxis,np.newaxis]).astype(int)
        else:
            r= np.floor((pt-origin[:len(spacing)])/(spacing*np.array([1,1,1])[:len(spacing)])).astype(int)*np.array([1,1,1])[:len(spacing)]
    return r 

def real_space(coord,origin,spacing):
    """
    Convert array indices to DICOM space coordinates
    """
    pts=coord[:,[1,0,2]]*spacing+origin[:len(spacing)]
    if len(spacing)==2:
        pts+=spacing/2
    elif len(spacing)==3:
        pts+=np.array([spacing[0],spacing[1],0])/2
    return pts

def getoss(XYZ):
    """Get approximate spacing without CT.
    The idea is to find points that share the same x or y coordinate and then take the most common spacing between them as spacing.
    """
    Spts=np.round(XYZ,2)
    mask0=Spts[:,0]==np.roll(Spts[:,0],1)
    mask2=Spts[:,2]==np.roll(Spts[:,2],1) #CAN NOT BE CALCULATED IF STRUC HAS ONLY ONE SLICE, WOULD NEED TO STORE SLICE SPACING
    mask=mask0*mask2
    mask1=Spts[:,1]==np.roll(Spts[:,1],1)
    mask=mask1*mask2
    difs0=np.abs(Spts[mask][:,0]-np.roll(Spts[:,0],1)[mask])    
    difs1=np.abs(Spts[mask][:,1]-np.roll(Spts[:,1],1)[mask])
    difs=np.concatenate([difs0,difs1])
    unique, counts = np.unique(difs[difs>0.5], return_counts=True)
    spacin=np.round(unique[np.argmax(counts)],2)
    spacing=np.array([spacin,spacin])   
    size=np.roll(np.ceil(np.abs((np.max(XYZ[:,:2],0)-np.min(XYZ[:,:2],0)))/spacin).astype(int),1)+np.array([2,2])
    minxyz=np.min(XYZ,axis=0)
    return minxyz, spacing, size

def correct_matches(PTVmatch,spacing):
    PTVmatch=np.array(PTVmatch)
    if len(PTVmatch.shape)>2:
        difs=np.linalg.norm(PTVmatch-np.roll(PTVmatch,1,2),axis=1)
        jumps1=difs>2*spacing[0]
        jumps2=np.isfinite(PTVmatch[:,0,:,:]) 
        jumps=np.all([jumps1,jumps2],0)
        # Assuming 'a' is your multi-dimensional boolean array and 'axis' is your specified axis
        a = jumps
        axis = 1

        # Find the indices of the last True values along the specified axis
        # indices = a.shape[axis] - np.argmax(a[:,::-1,:], axis=axis) -1
        indices = np.argmax(a, axis=axis)

        # Create a new boolean array of the same shape as 'a'
        b = np.zeros_like(a)

        # Set only the elements at the found indices to True
        np.put_along_axis(b, np.expand_dims(indices, axis=axis), True, axis=axis)
        x,y,z = np.meshgrid(np.arange(PTVmatch.shape[0]),np.arange(PTVmatch.shape[1]),np.arange(PTVmatch.shape[3]))
        return PTVmatch[x,y,indices,z].transpose((1,0,2)),b

def flatten(xss):
    """Get rid of higher dimension in lists."""
    return np.array([x for xs in xss for x in xs])

def getfolder(base, kind):
    """Helper function which searches sibling folders for CT files"""
    CTfiles=glob.glob(f"{base}/*/{kind}*")
    if len(CTfiles)>0:
        CTfolder=np.unique([f.split('\\')[-2] for f in CTfiles])
    else: CTfolder = []
    return CTfolder

def mergeIntervalResults(folder=SaveFolder):
    """Combine all the different interval files into one."""
    df1=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/PTV[*")],ignore_index=True)
    df1.to_pickle(f"{folder}/PTV.pickle")
    df2=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/Folders[*")],ignore_index=True)
    df2.to_pickle(f"{folder}/Folders.pickle")
    df4=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/MU[*")],ignore_index=True)
    df4.to_pickle(f"{folder}/MU.pickle")
    df5=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/PCAMesh[*")],ignore_index=True)
    df5.to_pickle(f"{folder}/PCAMesh.pickle")
    df6=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/Treatable[*")],ignore_index=True)
    df6.to_pickle(f"{folder}/Treatable.pickle")
    df7=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/Dose[*")],ignore_index=True)
    df7.to_pickle(f"{folder}/Dose.pickle")
    df8=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/DistsPTVOAR[*")],ignore_index=True)
    df8.to_pickle(f"{folder}/DistsPTVOAR.pickle")
    df9=pd.concat([pd.read_pickle(i) for i in glob.glob(f"{folder}/HU[*")],ignore_index=True)
    df9.to_pickle(f"{folder}/HU.pickle")
    return df1,df2,df4,df5,df6,df7,df8,df9 

def collectStructureNames(folder='//Cifs2/imrt_qa$/EIT/Marvin/DicomStructures_Joe_2.xlsx',save=SaveFolder):
    """Check structure names to find what to search for, in this case whether Lungs is enough"""
    ds = pd.read_excel(folder)
    ds2= pd.read_excel(PTVInfoSheet)
    countermoved=0
    result={'PlanSetupSer' : [], 'Structures' : []}
    counterlung=0
    for i,RSf in enumerate(ds['StructureSet_DicomFIle_Path']):
        site = ds2['Rx site'][ds2['PlanSetupSer']==ds['PlanSetupSer'][i]].iloc[0]
        if np.any([s in site for s in ["Lung", "LUNG", "lung", "LUL", "RUL", "LLL", "RLL", "Thoracic-NSCLC", "Thoracic-SCLC"]]):
            try:
                RS =  pydicom.dcmread(RSf)
            except:
                countermoved+=1
                continue
            names=[]
            for s in RS.StructureSetROISequence:
                names.append(s.ROIName)
            result['PlanSetupSer'].append(ds['PlanSetupSer'][i])
            result['Structures'].append(names)
            counterlung+=1
            if counterlung == 50:
                df = pd.DataFrame.from_dict(result)
                df.to_pickle(f"{save}/LungStructuresNames.pickle")
                print(f'Unavailable files: {countermoved}')
                return df

### Depth & HU

#### Generate Data

In [5]:
def analyzeStructuresCT(RS=None,RP=None,save=None,folder=None,LoadCT=False,PTVnames=None,debug=0,mode=0,TimmermanDict=TimmermanDict):
    """
    Main function to evaluate a structure file in connection to CT images, if wanted.

    Input:
    - RS : DICOM StructureSet
    - RP : DICOM Plan (optional, then tries to load structure set specified in plan)
    - save : str ; Folder where the result should be stored.
    - folder : list ; folder locations of different DICOM files
    - LoadCT : bool ; Whether to use CT images or not
    - PTVnames : list ; List of PTV names that should be used for the analysis instead of searching for 'ptv' in structure names
    - debug : bool ; changes behaviour of code to supply some more information while running
    - mode : str ; Set to 'HU' to only calculate HU of structures.

    Output:
    - Dist.pickle : Pandas dataframe ; contains the depth analysis results. Columns: "Name", "z", "q0", "d2", "d4", "d4_att", "q0_att", "q0_attLung"
    - results2: Pandas dataframe ; contains the TV structures as point clouds. Colmumns: "Name",'TV Type',"CTOriginSpacingSize", 'XYZ'
    - HU.pickle : Pandas dataframe; contains HU results. Columns: "PlanSetupSer","Name", 'TV Type', 'HU Counts', 'HU Values', 'HU Mean', 'HU First Quartile', 'HU Median', 'HU Third Quartile', 'In Lung', 'In Left Lung', 'In Right Lung'
    """
    #Initialize result variables
    results = {"Name" : [], "z" : [], "q0":[], "d2" : [], "d4" : [], "d4_att" : [], "q0_att" : [], "q0_attLung" : []}
    results2 = {"Name" : [],'TV Type':[],"CTOriginSpacingSize" : [], 'XYZ' :[]}
    results3 = {"Name" : [], 'Type' : [], 'HU Counts' : [], 'HU Values' : [], 'HU Mean' : [], 'HU First Quartile' : [], 'HU Median' : [], 'HU Third Quartile' : [], 'In Lung' : [], 'In Left Lung' : [], 'In Right Lung' : []}
    ROITV = []
    TVNameD = {}
    GCPTVD = {}
    ROIOAR= []
    ROIbody = None
    ROISkin = None
    ROILung = None
    ROILungL = None
    ROILungR = None
    if RP:
        RS=pydicom.dcmread(f"{folder[0]}/RS.{RP.ReferencedStructureSetSequence[0].ReferencedSOPInstanceUID}.dcm")
    #Go through all Structures and try to match them to given categories
    for s in RS.StructureSetROISequence:
        if debug: print(s.ROIName)
        cfname=s.ROIName.casefold()
        if cfname[0] in 'zx^':
            continue
        elif cfname == 'body' or cfname == 'outer contour':
            if debug: print(s.ROIName,'is body')
            ROIbody = s.ROINumber
            continue
        elif cfname == 'skin':
            if debug: print(s.ROIName,'is Skin')
            ROISkin = s.ROINumber
            continue
        elif "ctv" == cfname[:3]:
            if debug: print(s.ROIName,'is CTV')
            TVNameD[s.ROINumber]=(s.ROIName)
            ROITV.append(s.ROINumber)
            GCPTVD[s.ROINumber]='CTV'
            continue
        elif "gtv" in cfname[:4]:
            if debug: print(s.ROIName,'is GTV')
            TVNameD[s.ROINumber]=(s.ROIName)
            ROITV.append(s.ROINumber)
            GCPTVD[s.ROINumber]='GTV'
            continue
        if PTVnames:
            if s.ROIName in PTVnames:
                if debug: print(s.ROIName,'is PTV')
                TVNameD[s.ROINumber]=(s.ROIName)
                ROITV.append(s.ROINumber)
                GCPTVD[s.ROINumber]='PTV'
                continue
        else:
            if "ptv" in cfname: #in general want PTV and then blank,_,^,number
                if not "avoid" in cfname and not "overlap" in cfname:
                    if debug: print(s.ROIName,'is PTV')
                    TVNameD[s.ROINumber]=(s.ROIName)
                    ROITV.append(s.ROINumber)
                    GCPTVD[s.ROINumber]='PTV'
                    continue
        if "lungs" in cfname: 
            if debug: print(s.ROIName, 'is Lung')
            ROILung = s.ROINumber
        elif cfname=="lung_l":
            ROILungL=s.ROINumber
        elif cfname=="lung_r":
            ROILungR=s.ROINumber
        #Not updated to match Timmerman
        elif np.any([oar in cfname for oar in TimmermanDict.keys()]):
            ROIOAR.append(s.ROINumber)
            if debug: print(s.ROIName, 'is OAR')
            
    if not ROIbody:
        ROIbody = ROISkin
    if len(ROITV)==0 or not ROIbody:
        logging.warning(f'No PTV or no body in RS UID {RS.SOPInstanceUID}')
        if PTVnames: print("Searching for:",PTVnames)
        for s in RS.StructureSetROISequence:
            print(s.ROIName)
        return [],[]
    ctrs = RS.ROIContourSequence
    TVs = []
    OARs = []
    TVName=[]
    GCPTV=[]
    #Match contours to structures
    for c in ctrs:
        try:
            _=c.ContourSequence
        except:
                continue
        if c.ReferencedROINumber in ROITV:
            TVs.append(c)
            TVName.append(TVNameD[c.ReferencedROINumber])
            GCPTV.append(GCPTVD[c.ReferencedROINumber])
        elif c.ReferencedROINumber == ROIbody: 
            body = [c]
        elif c.ReferencedROINumber == ROILung: 
            Lung = [c] 
        elif c.ReferencedROINumber == ROILungL: 
            LungL = [c] 
        elif c.ReferencedROINumber == ROILungR: 
            LungR = [c] 
        elif c.ReferencedROINumber in ROIOAR:
            OARs.append(c)
    #Load contour points
    Spts,CTids  = get_points(body,CT=LoadCT)
    if ROILung:
        try:
            Lpts,_ = get_points(Lung)
        except:
            logging.warning(f"Lung Points could not be loaded in RS UID {RS.SOPInstanceUID}")
            ROILung=None
    if ROILungL:
        try:
            LptsL,_ = get_points(LungL)
        except:
            logging.warning(f"Lung Points could not be loaded in RS UID {RS.SOPInstanceUID}")
            ROILungL=None
    if ROILungR:
        try:
            LptsR,_ = get_points(LungR)
        except:
            logging.warning(f"Lung Points could not be loaded in RS UID {RS.SOPInstanceUID}")
            ROILungR=None
    if mode != 'HU':
        OARpts=[get_points([oar])[0] for oar in OARs]
    TVptss=[get_points([PTV],True) for PTV in TVs]
    #Try to load CT files from folders. Assumes that all of them are in one folder.
    LoadCTmain = LoadCT
    if LoadCT:
        found=False
        for f in folder:
            try:
                CT = pydicom.dcmread(f"{f}/CT.{CTids[Spts[0,2]]}")
                found=True
                folder=[f]
                break
            except:
                continue
        if found == False:
            try:
                CT = pydicom.dcmread(f"{folder[0]}/CT.{CTids[Spts[0,2]]}.dcm")
            except:
                logging.warning(f'CT file not encountered for {RS.SOPInstanceUID}, {Spts[0,2]}')
                LoadCT=False
        if LoadCT:
            origin=np.array(CT.ImagePositionPatient)
            spacing=np.array(CT.PixelSpacing)
            origin-=np.array([spacing[0],spacing[1],0])/2
            if spacing[0]!=spacing[1]:
                warnings.warn(f"Spacing not symmetric, change code")
            size=np.array([CT.Columns,CT.Rows])
            for i,o in enumerate(origin[:2]): #origin should be negative, so switch corner 
                if o>0:
                    origin[i]=o-size[i]*spacing[i]
    if not LoadCT:
        origin,spacing,size=getoss(Spts)
        try:
            _,spacing1,_=getoss(TVptss[0][0]) #might fail if ptv too small
            size=np.ceil(size*(spacing/spacing1)).astype(int)
            spacing=spacing1
        except:
            logging.warning(f'PTV spacing failed for {RS.SOPInstanceUID}, {TVName[0]}')   
    #Determine Slice Spacing
    unz = np.unique(Spts[:,2])
    spac, counts= np.unique(np.abs(unz-np.roll(unz,1)), return_counts=True)
    slicespacing=spac[np.argmax(counts)]
    #Set up dictionaries to save boolean masks of contours for different structures and slices
    PD_dict={}
    if mode != 'HU':
        MaskOAR_dict={}
        Sgrid_dict={}
    if ROILung: 
        MaskLung_dict={}
        if LoadCT:
            LungHU_dict={}
    if ROILungL: 
        MaskLungL_dict={}
        if LoadCT:
            LungLHU_dict={}
    if ROILungR: 
        MaskLungR_dict={}
        if LoadCT:
            LungRHU_dict={}
    sampling=0.8 #Sampling parameter for the ray tracing algortihm. Gets multiplied with the spacing to determine the distance of ray sample points.
    #Loop through all TVs 
    for p,_ in enumerate(TVs):
        TVpts, TVcntrs, TVcntr = TVptss[p]
        if mode != 'HU':
            results2["Name"].append(TVName[p])
            results2["TV Type"].append(GCPTV[p])
            results2['XYZ'].append(TVpts)
            results2["CTOriginSpacingSize"].append([origin,np.array([spacing[0],spacing[1],slicespacing]),size])
        if LoadCT:
            results3["Name"].append(TVName[p])
            results3["Type"].append(GCPTV[p])
            TVHU_dict={}
        if RP: 
            IsoDev = np.array(TVcntr[0]-RP.BeamSequence[0].ControlPointSequence[0].IsocenterPosition)
            print(f"StrucIso-PlanIso={IsoDev}")
        #Determine slices that need to be loaded for TV and loop through them
        zs=np.unique(TVcntrs[:,2])
        IsoinPTV=0
        for z in zs:
            #Try to find CT for slice and then convert it to HU
            if LoadCT: 
                PD = PD_dict.get(z,False)
                if not np.any(PD):
                    try:
                        CTids[z]
                    except:
                        continue
                    try:
                        CT = pydicom.dcmread(f"{folder[0]}/CT.{CTids[z]}")
                    except:
                        try:
                            CT = pydicom.dcmread(f"{folder[0]}/CT.{CTids[z]}.dcm")
                        except:
                            logging.warning(f'CT file not encountered for {RS.SOPInstanceUID}')
                            LoadCT=False
                    if LoadCT: 
                        PD=(CT.RescaleSlope*CT.pixel_array+CT.RescaleIntercept)/1000#(-CT.RescaleIntercept)#1000 #use intercept instead of 1000, because min pixel is 0 and we do not want values <-1, as that would lead to a negative distance
                        # PD[PD<0]=0
                        PD_dict[z]=PD
            #Turn Structures into binary masks
            iso= TVcntrs[TVcntrs[:,2]==z][0][:2]
            isoc = coordinate_space(iso,origin,spacing)
            PTVgrid = binary_dilation(to_grid(TVpts,z,origin,spacing,size))
            if mode != 'HU':
                Sgrid = Sgrid_dict.get(z,False)
                if not np.any(Sgrid):
                    Sgrid = binary_dilation(to_grid(Spts,z,origin,spacing,size))#,iterations=2)
                    Sgrid_dict[z]=Sgrid
                if debug==2:
                    if 'PTV' in TVName[p]:
                        print(isoc,TVName[p])
                        fig = px.imshow(Sgrid+PTVgrid, color_continuous_scale='gray',title=z)
                        fig.show()
                        # pxy=TVpts[TVpts[:,2]==z]
                        # fig2=px.scatter(x=pxy[:,0],y=pxy[:,1])
                        # fig2.show()
                        input()
                MaskOAR = MaskOAR_dict.get(z,False)
                if not np.any(MaskOAR):
                    MaskOAR=np.full(size, False)
                    for oar in OARpts:
                        MaskOAR+=binary_erosion(binary_fill_holes(binary_dilation(to_grid(oar,z,origin,spacing,size))))
                    MaskOAR_dict[z]=MaskOAR
            if ROILung: 
                MaskLung = MaskLung_dict.get(z,False)
                if not np.any(MaskLung):
                    Lgrid = binary_dilation(to_grid(Lpts,z,origin,spacing,size))#,iterations=2)
                    MaskLung = binary_erosion(binary_fill_holes(Lgrid))#,iterations=2)
                    if mode != 'HU':
                        MaskOAR+=MaskLung
                    MaskLung_dict[z]=MaskLung
                    if LoadCT:
                        LungHU_dict[z]=(PD*1000+1e4)*MaskLung
            if ROILungL: 
                MaskLungL = MaskLungL_dict.get(z,False)
                if not np.any(MaskLungL):
                    LgridL = binary_dilation(to_grid(LptsL,z,origin,spacing,size))#,iterations=2)
                    MaskLungL = binary_erosion(binary_fill_holes(LgridL))#,iterations=2)
                    MaskLungL_dict[z]=MaskLungL
                    if LoadCT:
                        LungLHU_dict[z]=(PD*1000+1e4)*MaskLungL
            if ROILungR: 
                MaskLungR = MaskLungR_dict.get(z,False)
                if not np.any(MaskLungR):
                    LgridR = binary_dilation(to_grid(LptsR,z,origin,spacing,size))#,iterations=2)
                    MaskLungR = binary_erosion(binary_fill_holes(LgridR))#,iterations=2)
                    MaskLungR_dict[z]=MaskLungR
                    if LoadCT:
                        LungRHU_dict[z]=(PD*1000+1e4)*MaskLungR
            MaskPTV = binary_erosion(binary_fill_holes(PTVgrid))
            #Perform Raytracing to find out distance from all points of TV contour for all angles to body contour
            if mode != 'HU':
                Smask = Spts[:,2]==z
                mask = TVpts[:,2]==z
                PTVptsxy=TVpts[mask][:,:2]
                md=np.mean(np.linalg.norm(PTVptsxy-np.roll(PTVptsxy,1,0),axis=1))
                skip=np.max([int(10/md),1])
                PTVptsxy=PTVptsxy[::skip]
                lengthray=(np.max(np.linalg.norm(Spts[Smask][:,np.newaxis,:2]-PTVptsxy[np.newaxis,:,:],axis=2))/np.min(spacing*sampling)+1).astype(int)
                if MaskPTV[isoc[1],isoc[0]]:
                    IsoinPTV+=1
                    targets=np.concatenate((iso[np.newaxis,:],PTVptsxy))
                else:
                    targets=PTVptsxy
                rays = targets[:,:,np.newaxis,np.newaxis] - np.arange(lengthray)[np.newaxis,np.newaxis,:,np.newaxis] * spacing[np.newaxis,:,np.newaxis,np.newaxis]*sampling * np.array([np.sin(-np.arange(0,360,2)* np.pi / 180.),np.cos(-np.arange(0,360,2)* np.pi / 180.)])[np.newaxis,:,np.newaxis,:]
                mask=np.all([np.all([rays[:,0,:,:]<=np.max(Spts[Smask][:,0]), rays[:,0,:,:]>=np.min(Spts[Smask][:,0])],0),np.all([rays[:,1,:,:]<=np.max(Spts[Smask][:,1]), rays[:,1,:,:]>=np.min(Spts[Smask][:,1])],0)],0)
                raycoordinates = coordinate_space(rays,origin,spacing)
                inOAR=np.full(mask.shape,False)
                inOAR[mask]=MaskOAR[raycoordinates[:,1,:,:][mask],raycoordinates[:,0,:,:][mask]]
                inSkin=np.full(mask.shape,False)
                inSkin[mask]=Sgrid[raycoordinates[:,1,:,:][mask],raycoordinates[:,0,:,:][mask]]
                Smatchb=rays.copy()
                Smatchb[~np.stack([inSkin,inSkin],1)]=np.inf
                Smatch,Sbool = correct_matches(Smatchb,spacing)
                PTVtoSkin=np.cumsum(np.roll(Sbool,1,1)[:,::-1,:],1)[:,::-1,:]
                q0=np.sum(spacing[0]*sampling*PTVtoSkin,1)+spacing[0]
                PTVtoSkininOAR=np.all([PTVtoSkin,inOAR],0)
                d4=np.sum(spacing[0]*sampling*PTVtoSkininOAR,1)
                if ROILung: 
                    inLung=np.full(mask.shape,False)
                    inLung[mask]=MaskLung[raycoordinates[:,1,:,:][mask],raycoordinates[:,0,:,:][mask]]
                    PTVtoSkininLung=np.all([PTVtoSkin,inLung],0)
                    d2=np.sum(spacing[0]*sampling*PTVtoSkininLung,1)
                    q0_attLung=q0+d2*((-650)/1000)
            #Calculate Attenuation Corrected Water Equivalent Distances using HU from CT
            if LoadCT:  
                if mode != 'HU':
                    inc=np.zeros(mask.shape)
                    inc[mask]=spacing[0]*sampling+spacing[0]*sampling*PD[raycoordinates[:,1,:,:][mask],raycoordinates[:,0,:,:][mask]]
                    q0_att=np.sum(inc*PTVtoSkin,1)+spacing[0]
                    # q0_att_max=np.max(np.min(q0_att,1))
                    d4_att=np.sum(inc*PTVtoSkininOAR,1)
                TVHU_dict[z]=(PD*1000+1e4)*MaskPTV
            else:
                if mode != 'HU':
                    d4_att=np.nan
                    q0_att=np.nan
            if mode != 'HU':
                results["Name"].append(TVName[p])
                results["z"].append(z)
                results["d4"].append(d4.T/10) #Distance between PTV and skin inside of OARs
                results["q0"].append(q0.T/10) #Distance between PTV and skin
                try:
                    q0_att=q0_att.T
                except:
                    pass
                try:
                    d4_att=d4_att.T
                except:
                    pass
                results["q0_att"].append(q0_att/10) #Distance between PTV and skin attenuation corrected
                results["d4_att"].append(d4_att/10) #Distance between PTV and skin inside of OARs attenuation corrected
                if ROILung:  
                    results["d2"].append(d2.T/10) #Distance between PTV and skin inside of lung
                    results["q0_attLung"].append(q0_attLung.T/10) #Distance between PTV and skin attenuation corrected only in lung
                else:                
                    results["d2"].append(np.zeros(d4.T.shape))
                    results["q0_attLung"].append(q0.T/10)
            if LoadCTmain: LoadCT=True
        #Store HU of structures and check if in lungs
        if LoadCT:
            TVHU_acc=[]
            if ROILung or ROILungL or ROILungR:      
                inLung=True           
                inLungL=True              
                inLungR=True       
            else:       
                inLung=False           
                inLungL=False              
                inLungR=False 
            for key, value in TVHU_dict.items():
                TVHU_acc.append((value[value>0]-1e4).astype(int))
                if ROILung or ROILungL or ROILungR:
                    inner=binary_dilation(value.astype(bool),iterations=1)
                    circ=binary_dilation(inner,iterations=1)^inner
                    if ROILung:
                        L_value=LungHU_dict[key]
                        intsec=circ & L_value.astype(bool)
                        # fig = px.imshow(inner+(circ*10)+(intsec*30))
                        # fig.show()
                        # input()
                        if np.sum(intsec)/np.sum(circ)<0.8:
                            inLung=False
                        if GCPTV[p]=='CTV' or GCPTV[p]=='GTV':
                            L_value[value.astype(bool)]=0
                            LungHU_dict[key]=L_value
                    else:
                        inLung=False
                    if ROILungL:
                        L_value=LungLHU_dict[key]
                        intsec=circ & L_value.astype(bool)
                        if np.sum(intsec)/np.sum(circ)<0.8:
                            inLungL=False
                        if GCPTV[p]=='CTV' or GCPTV[p]=='GTV':
                            L_value[value.astype(bool)]=0
                            LungLHU_dict[key]=L_value
                    else:
                        inLungL=False
                    if ROILungR:
                        L_value=LungRHU_dict[key]
                        intsec=circ & L_value.astype(bool)
                        if np.sum(intsec)/np.sum(circ)<0.8:
                            inLungR=False
                        if GCPTV[p]=='CTV' or GCPTV[p]=='GTV':
                            L_value[value.astype(bool)]=0
                            LungRHU_dict[key]=L_value
                    else:
                        inLungR=False
            TVHU_acc=flatten(TVHU_acc)
            value,count=np.unique(TVHU_acc,return_counts=True)
            results3["HU Counts"].append(count)
            results3["HU Values"].append(value)
            results3["HU Mean"].append(np.mean(TVHU_acc))
            one,two,three=np.quantile(TVHU_acc,[0.25,0.50,0.75])
            results3["HU First Quartile"].append(one)
            results3["HU Median"].append(two)
            results3["HU Third Quartile"].append(three)
            results3['In Lung'].append(inLung)
            results3['In Left Lung'].append(inLungL)
            results3['In Right Lung'].append(inLungR)
    if LoadCT:
        if ROILung:
            results3["Name"].append('Lung')
            results3["Type"].append('Lung')
            LungHU_acc=[]  
            for key, value in LungHU_dict.items():
                LungHU_acc.append((value[value!=0]-1e4).astype(int))
            LungHU_acc=flatten(LungHU_acc)
            value,count=np.unique(LungHU_acc,return_counts=True)
            results3["HU Counts"].append(count)
            results3["HU Values"].append(value)
            results3["HU Mean"].append(np.mean(LungHU_acc))
            one,two,three=np.quantile(LungHU_acc,[0.25,0.50,0.75])
            results3["HU First Quartile"].append(one)
            results3["HU Median"].append(two)
            results3["HU Third Quartile"].append(three)
            results3['In Lung'].append(True)
            results3['In Left Lung'].append(False)
            results3['In Right Lung'].append(False)
        if ROILungL:
            results3["Name"].append('Left Lung')
            results3["Type"].append('Left Lung')
            LungHU_acc=[]  
            for key, value in LungLHU_dict.items():
                LungHU_acc.append((value[value!=0]-1e4).astype(int))
            LungHU_acc=flatten(LungHU_acc)
            value,count=np.unique(LungHU_acc,return_counts=True)
            results3["HU Counts"].append(count)
            results3["HU Values"].append(value)
            results3["HU Mean"].append(np.mean(LungHU_acc))
            one,two,three=np.quantile(LungHU_acc,[0.25,0.50,0.75])
            results3["HU First Quartile"].append(one)
            results3["HU Median"].append(two)
            results3["HU Third Quartile"].append(three)
            results3['In Lung'].append(True)
            results3['In Left Lung'].append(True)
            results3['In Right Lung'].append(False)
        if ROILungR:
            results3["Name"].append('Right Lung')
            results3["Type"].append('Right Lung')
            LungHU_acc=[]  
            for key, value in LungRHU_dict.items():
                LungHU_acc.append((value[value!=0]-1e4).astype(int))
            LungHU_acc=flatten(LungHU_acc)
            value,count=np.unique(LungHU_acc,return_counts=True)
            results3["HU Counts"].append(count)
            results3["HU Values"].append(value)
            results3["HU Mean"].append(np.mean(LungHU_acc))
            one,two,three=np.quantile(LungHU_acc,[0.25,0.50,0.75])
            results3["HU First Quartile"].append(one)
            results3["HU Median"].append(two)
            results3["HU Third Quartile"].append(three)
            results3['In Lung'].append(True)
            results3['In Left Lung'].append(False)
            results3['In Right Lung'].append(True)
    df=pd.DataFrame.from_dict(results)
    if save: 
        if not os.path.exists(save):
            os.makedirs(save)
        #Contains the calculated distances from raytracing
        if mode != 'HU':
            if debug: df.to_csv(f"{save}/Dist.csv", index=False)
            df.to_pickle(f"{save}/Dist.pickle")
        #contains HU values
        if LoadCT:
            df3=pd.DataFrame.from_dict(results3)
            df3.to_pickle(f"{save}/HU.pickle")
    return df,results2

def batchAnalyzeStructuresCT_sheet(loc_sheet='//Cifs2/imrt_qa$/EIT/Marvin/DicomStructures_Joe_3.xlsx',PTV_sheet=PTVInfoSheet,interval=[],debug=False,mode=0, save=SaveFolder):
    """
    Given an excel sheet of RT DICOM file locations and another with PTV names and Plan IDs, this function runs the main analysis function in batch mode.
    
    Input:
    loc_sheet : excel sheet of RT DICOM file locations
    PTV_sheet : excel sheet of PTV names and Plan IDs
    interval : list of int ; Only load locations in this interval from sheet
    debug : bool ; changes behaviour of code to supply some more information while running
    mode : str ; Set to 'HU' to only calculate HU of structures.

    Output:
    PTV.pickle: Pandas dataframe ; contains the TV structures as point clouds, columns: 'PlanSetupSer',"Name",'TV Type',"CTOriginSpacingSize",'XYZ'
    Files.pickle : Pandas dataframe; contains the CT folder location. Assumes that CT images are stored in a different folder from the structures, but in the same base folder. Like base/structures and base/CT. Could be solved more elegant by providing CT folder location in the location sheet. Columns: 'PlanSetupSer', 'Root Folder', 'CT Folder'
    """
    results2 = {'PlanSetupSer':[],"Name" : [],'TV Type':[],"CTOriginSpacingSize" : [],'XYZ' :[]}
    # results3 = {'PlanSetupSer':[], 'Root Folder' : [],'RP Folder' : [], 'CT Folder' : [], 'RD Folder' : [], 'RI Folder' : []}
    results3 = {'PlanSetupSer':[], 'Root Folder' : [], 'CT Folder' : []}
    ds=updateLocs(loc_sheet,debug)
    if interval: ds=ds.iloc[interval[0]:interval[1]]
    ds2= pd.read_excel(PTV_sheet)
    countermoved=0
    if not os.path.exists(save):
        os.makedirs(save)
    logging.basicConfig(filename=f'{save}/LogsPTV.log', encoding='utf-8', level=logging.WARNING)
    for i,RSf in enumerate(ds['StructureSet_Path']):
        save2 = f"{save}/{ds['PlanSetupSer'].iloc[i]}"
        if not os.path.exists(save2) or mode == 'HU':
            try:
                RS =  pydicom.dcmread(RSf)
                base='/'.join(RSf.split('\\')[:-2])
                # RPfolder = getfolder(base, "RP")
                CTfolder = getfolder(base, "CT")    #assumes that CT images are stored in a different folder from the structures, but in the same base folder. Like base/structures and base/CT. Could be solved more elegant by providing CT folder location in the location sheet.
                # RDfolder = getfolder(base, "RD")
                # RIfolder = getfolder(base, "RI")
                results3['PlanSetupSer'].append(ds['PlanSetupSer'].iloc[i])
                results3['Root Folder'].append(base)
                # results3['RP Folder'].append(RPfolder)
                results3['CT Folder'].append(CTfolder)
                # results3["RD Folder"].append(RDfolder)
                # results3['RI Folder'].append(RIfolder)
            except:
                countermoved+=1
                logging.exception(f"Could not find PlanSetupSer {ds['PlanSetupSer'].iloc[i]}.")
                continue
            print(i)
            TVs=ds2['PTV name'][ds2['PlanSetupSer']==ds['PlanSetupSer'].iloc[i]].tolist()
            try:
                if len(CTfolder)>0:
                    _,rs = analyzeStructuresCT(RS=RS,save=save2,PTVnames=TVs,folder=[f"{base}/{CTf}" for CTf in CTfolder],LoadCT=True,debug=debug,mode=mode)
                else:
                    _,rs = analyzeStructuresCT(RS=RS,save=save2,PTVnames=TVs,debug=debug,mode=mode) #Search for TVs mentioned in excel only instead of general search, because some TVs are not called PTV, and can also get wrong structures when just searching for PTV
            except BaseException:
                logging.exception(f"analyzeStructuresCT failed for PlanSetupSer {ds['PlanSetupSer'].iloc[i]}.")
                continue
            if mode != 'HU':
                if len(rs)!=0: 
                    results2["Name"].extend(rs["Name"])
                    results2["CTOriginSpacingSize"].extend(rs["CTOriginSpacingSize"])
                    results2["XYZ"].extend(rs["XYZ"])
                    results2["TV Type"].extend(rs["TV Type"])
                    for _ in range(len(rs["Name"])):
                        results2['PlanSetupSer'].append(ds['PlanSetupSer'].iloc[i])
    if mode != 'HU':        
        df = pd.DataFrame.from_dict(results2)
        df3 = pd.DataFrame.from_dict(results3)
        if interval:
            df.to_pickle(f"{save}/PTV{interval}.pickle")
            df3.to_pickle(f"{save}/Folders{interval}.pickle")
        else:    
            try:
                shutil.copyfile(f"{save}/PTV.pickle",f"{save}/PTV.pickle.old")
                shutil.copyfile(f"{save}/Folders.pickle",f"{save}/Folders.pickle.old")
                PTV=pd.read_pickle(f"{save}/PTV.pickle")
                Folders=pd.read_pickle(f"{save}/Folders.pickle")
                df=pd.concat([PTV,df])
                df3=pd.concat([Folders,df3])
                df.to_pickle(f"{save}/PTV.pickle")
                df3.to_pickle(f"{save}/Folders.pickle")
            except:
                if not os.path.exists(f"{save}/PTV.pickle"):
                    df.to_pickle(f"{save}/PTV.pickle")
                    df3.to_pickle(f"{save}/Folders.pickle")
                else:
                    logging.exception(f"PTV.pickle exists, but appending failed. Create PTVnew.")
                    df.to_pickle(f"{save}/PTVnew.pickle")
                    df3.to_pickle(f"{save}/Foldersnew.pickle")
        print(f'Unavailable files: {countermoved}')
        logging.warning(f'Unavailable files: {countermoved}')
        return df,df3
    
def batchAnalyzeStructuresCTPlan_folder(folder='//Cifshd/homedir$/Data/Mobius1_4_patient_HD',save=f"//Cifshd/homedir$/Evaluation/Mobius1_4_patient_HD",debug=False,analyzePlan=True):
    """Analyzes Plan and Structure files in one folder
    
    Input:
    folder : folder with DICOM files
    save : folder where results should be stored
    debug : bool ; changes behaviour of code to supply some more information while running
    analyzePlan : bool ; Whether to analyze plans as well or only structures and CT

    Output:
    results2: Pandas dataframe ; contains the TV structures as point clouds
    resultsMU : Pandas dataframe; contains the plan analysis results
    
    Also stores output specified in analyzeStructuresCT function, namely:
    results : Pandas dataframe ; contains the depth analysis results
    results3 : Pandas dataframe; contains HU results
    """
    

    results2 = {'PlanSetupSer':[],"Name" : [],'TV Type':[],"CTOriginSpacingSize" : [],'XYZ' :[]}
    resultsMU = {'PlanSetupSer' : [], 'Beam Name' : [], 'Meterset [MU]' : [], 'Gantry Angle' : [], 'MU per Angle' : [], 'Beam Delivery Duration Limit [s]':[],'Table Top Roll Angle' :[],'Table Top Pitch Angle':[],
                 'Patient Support Angle' : [],'Table Top Eccentric Angle':[],'Beam Limiting Device Position Sequence' : [],'Beam Limiting Device Angle' : [],'SSD [cm]' : [],'Dose Rate Set [MU/min]' : [],
                   'Dose [Gy]':[],'High Dose Technique Type':[],'Radiation Type':[],'Beam Type':[], 'Isocenter [mm]':[]}
    for i,RSf in enumerate(glob.glob(f"{folder}/RS*")):
        RS =  pydicom.dcmread(RSf)
        save2 = f"{save}/{RS.SOPInstanceUID}"
        print(save)
        _,rs = analyzeStructuresCT(RS=RS,save=save2,folder=[folder],LoadCT=True,debug=debug)
        if len(rs)!=0: 
            results2["Name"].extend(rs["Name"])
            results2["CTOriginSpacingSize"].extend(rs["CTOriginSpacingSize"])
            results2["XYZ"].extend(rs["XYZ"])
            results2["TV Type"].extend(rs["TV Type"])
            for _ in range(len(rs["Name"])):
                results2['PlanSetupSer'].append(RS.SOPInstanceUID)
    df = pd.DataFrame.from_dict(results2)
    df.to_pickle(f"{save}/PTV.pickle")
    if analyzePlan:
        for RPfileloc in glob.glob(f"{folder}/RP*"):
            RPfile=pydicom.read_file(RPfileloc)
            angle, mu, beam, Roll, Pitch, Yaw, TableTopEccentricAngle,BeamLimitingDevicePositionSequence,BeamDeliveryDurationLimit, iso, Meterset,Dose,BeamDeliveryDurationLimit,BeamLimitingDeviceAngle,SSD,DoseRateSet,HighDoseTechniqueType,RadiationType,BeamType=analyzePlan(RPfile)
            if len(angle)!=0: 
                resultsMU["Gantry Angle"].append(angle)
                resultsMU["MU per Angle"].append(mu)
                resultsMU["Beam Name"].append(beam)
                resultsMU["Table Top Roll Angle"].append(Roll)
                resultsMU["Table Top Pitch Angle"].append(Pitch)
                resultsMU["Patient Support Angle"].append(Yaw)
                resultsMU["Table Top Eccentric Angle"].append(TableTopEccentricAngle)
                resultsMU["Beam Limiting Device Position Sequence"].append(BeamLimitingDevicePositionSequence)
                resultsMU["Beam Delivery Duration Limit [s]"].append(BeamDeliveryDurationLimit)
                resultsMU["Isocenter [mm]"].append(iso)
                resultsMU["Meterset [MU]"].append(Meterset)
                resultsMU["Dose [Gy]"].append(Dose)
                resultsMU["Beam Limiting Device Angle"].append(BeamLimitingDeviceAngle)
                resultsMU["SSD [cm]"].append(SSD)
                resultsMU["Dose Rate Set [MU/min]"].append(DoseRateSet)
                resultsMU["High Dose Technique Type"].append(HighDoseTechniqueType)
                resultsMU["Radiation Type"].append(RadiationType)
                resultsMU["Beam Type"].append(BeamType)
                resultsMU['PlanSetupSer'].append(RPfile.ReferencedStructureSetSequence[0].ReferencedSOPInstanceUID)
        dfMU = pd.DataFrame.from_dict(resultsMU)
        dfMU.to_pickle(f"{save}/MU.pickle")
    return df

#### Analyze HU

In [6]:
def compileHU(folder,interval=None):
    """
    Go through all HU results in folders
    
    
    Input:
    - folder : folder where analyzeStructuresCT stored the HU dataframes
    - interval : list of int ; Only load locations in this interval from folder

    Output:
    - HU.pickle : Pandas dataframe; contains HU results for all plans. Columns: "PlanSetupSer","Name", 'TV Type', 'HU Counts', 'HU Values', 'HU Mean', 'HU First Quartile', 'HU Median', 'HU Third Quartile', 'In Lung', 'In Left Lung', 'In Right Lung'
    """

    results = {"PlanSetupSer" : [],"Name" : [], 'TV Type' : [], 'HU Counts' : [], 'HU Values' : [], 'HU Mean' : [], 'HU First Quartile' : [], 'HU Median' : [], 'HU Third Quartile' : [], 'In Lung' : [], 'In Left Lung' : [], 'In Right Lung' : []}
    try:
        done=pd.read_pickle(f"{folder}/HU.pickle")
    except:
        pass
    PSSs=glob.glob(f"{folder}/[0-9]*")
    if interval:
        PSSs=PSSs[interval[0]:interval[1]]
    for j,f1 in enumerate(PSSs):
        if 'done' in locals():
            if np.any(int(f1.split('\\')[-1]) == done['PlanSetupSer']):
                continue
        print(j, f1)
        try:
            df = pd.read_pickle(f"{f1}/HU.pickle")
        except:
            continue
        for i in range(len(df['Name'])):
            try:
                results['PlanSetupSer'].append(int(f1.split('\\')[-1]))
            except:
                results['PlanSetupSer'].append(f1.split('\\')[-1])
            results["Name"].append(df['Name'][i])
            results["TV Type"].append(df['Type'][i])
            results["HU Counts"].append(df['HU Counts'][i])
            results["HU Values"].append(df['HU Values'][i])
            results["HU Mean"].append(df['HU Mean'][i])
            results["HU First Quartile"].append(df['HU First Quartile'][i])
            results["HU Median"].append(df['HU Median'][i])
            results["HU Third Quartile"].append(df['HU Third Quartile'][i])
            results["In Lung"].append(df['In Lung'][i])
            results["In Left Lung"].append(df['In Left Lung'][i])
            results["In Right Lung"].append(df['In Right Lung'][i])
    df=pd.DataFrame.from_dict(results)
    if 'done' in locals():
        df=pd.concat((done,df))
    df.to_pickle(f"{folder}/HU{interval if np.any(interval) else ''}.pickle")
    return df

#### Analyze Depth

In [7]:
def determineMinMaxDepths(ds, dist):
    """
    Determine Max and Min Depths of contours
    
    Input:
    ds : Pandas dataframe; Contains the depths results generated for one structure by analyzeStructuresCT
    dist : str ; Name of the column of the depth of interest

    Output:
    viableSlicesPercentage : float ; percentage of slices where max min depth is smaller than 8
    maxmindepth : float ; distance needed at minimum to treat whole structure
    minmindepth : float ; distance needed at minimum to reach structure
    dang : float ; Mean of depth across points and slices per angle. ATTENTION: Angle for one point is not the same as for another, given that each point is iso. But maybe okay if source is far? Otherwise just use first point, which is iso in the center
    """
    depth=np.array([np.max(np.min(z,axis=0)) for z in ds[dist]]) #min of angles, max of points gives one value per slice
    viableslices=depth<8
    try:
        depthmin=np.array([np.min(z[np.all([(z-np.roll(z,1,0))<0.5,(z-np.roll(z,1,0))>0.1],0)]) for z in ds[dist]]) #Filter out min outliers. Black magic happening. Basically, check that distance jump to next angle is not too big, but also not very small as for multiple outliers next to each other
        minmindepth=np.min(depthmin[~np.isinf(depthmin)])
    except:
        minmindepth=np.nan    
    maxmindepth=np.max(depth[~np.isinf(depth)]) #take max dist as depth. More accurate than mean, as that is what really has to be treated at most
    dang = np.mean([np.mean(z,axis=1) for z in ds[dist]],axis=0) #Mean of depth across points and slices per angle. ATTENTION: Angle for one point is not the same as for another, given that each point is iso. But maybe okay if source is far? Otherwise just use first point, which is iso in the center
    viableSlicesPercentage=np.sum(viableslices)/len(viableslices)
    return viableSlicesPercentage, maxmindepth, minmindepth, dang

def planDetermineMinMaxDepths(df,cols = ["q0","q0_att","q0_attLung"]):
    """Wrapper for depths function to use it in batch mode."""
    Names = df['Name'].unique()
    treatableq2CT=[]
    treatableq2L=[]
    treatableq2=[]
    maxMinDepth_q0_att=[]
    maxMinDepth_q0=[]
    maxMinDepth_q0_attLung=[]
    minMinDepth_q0_att=[]
    minMinDepth_q0=[]
    minMinDepth_q0_attLung=[]
    dang2CT=[]
    dang2=[]
    dang2L=[]
    for Name in Names:   
        dfPTV = df[df['Name']==Name]
        for d in cols:
            if d == "q0_att": 
                if np.any(np.isnan(dfPTV[d].iloc[0])):
                    treatableq2CT.append(np.nan)
                    maxMinDepth_q0_att.append(np.nan)
                    minMinDepth_q0_att.append(np.nan)
                    dang2CT.append(np.nan)
                else:
                    tr,mmd, minmindepth,dang=determineMinMaxDepths(dfPTV,d)
                    treatableq2CT.append(tr)    
                    maxMinDepth_q0_att.append(mmd)
                    minMinDepth_q0_att.append(minmindepth)
                    dang2CT.append(dang)
            elif d == "q0_attLung": 
                    tr,mmd, minmindepth,dang=determineMinMaxDepths(dfPTV,d)
                    treatableq2L.append(tr)    
                    maxMinDepth_q0_attLung.append(mmd)
                    minMinDepth_q0_attLung.append(minmindepth)
                    dang2L.append(dang)
            elif d == "q0": 
                    tr,mmd, minmindepth,dang=determineMinMaxDepths(dfPTV,d)
                    treatableq2.append(tr)    
                    maxMinDepth_q0.append(mmd)
                    minMinDepth_q0.append(minmindepth)
                    dang2.append(dang)
    return Names, treatableq2CT, treatableq2L, treatableq2,maxMinDepth_q0_att,maxMinDepth_q0_attLung,maxMinDepth_q0,minMinDepth_q0_att,minMinDepth_q0_attLung,minMinDepth_q0, dang2CT, dang2L, dang2

def batchDetermineMinMaxDepths(folder,plot=False,interval=None):
    """
    Wrapper for planDetermineMinMaxDepths to go through all results in folders
    
    Input:
    - folder : folder where analyzeStructuresCT stored the HU dataframes
    - interval : list of int ; Only load locations in this interval from folder

    Output:
    - Treatable.pickle : Pandas dataframe; contains Distance results for all plans. Columns: "PlanSetupSer", "Name","Treatable q0","Treatable q0_att", "Treatable q0_attLung", "Max Depth q0", "Max Depth q0_att", "Max Depth q0_attLung", "Min Depth q0", "Min Depth q0_att", "Min Depth q0_attLung", "Mean Depth q0 per Angle", "Mean Depth q0_att per Angle","Mean Depth q0_attLung per Angle"
    """
    results = {"PlanSetupSer" : [], "Name" : [],"Treatable q0" : [],"Treatable q0_att" : [], "Treatable q0_attLung" : [], "Max Depth q0" : [], "Max Depth q0_att" : [], "Max Depth q0_attLung" : [], "Min Depth q0" : [], "Min Depth q0_att" : [], "Min Depth q0_attLung" : [], "Mean Depth q0 per Angle" : [], "Mean Depth q0_att per Angle" : [],"Mean Depth q0_attLung per Angle" : []}
    try:
        done=pd.read_pickle(f"{folder}/Treatable.pickle")
    except:
        pass
    PSSs=glob.glob(f"{folder}/[0-9]*")
    if interval:
        PSSs=PSSs[interval[0]:interval[1]]
    for j,f1 in enumerate(PSSs):
        if 'done' in locals():
            if np.any(int(f1.split('\\')[-1]) == done['PlanSetupSer']):
                continue
        print(j, f1)
        df = pd.read_pickle(f"{f1}/Dist.pickle")
        if plot:
            TVs, mdptvbc, meptvbc, treatableq2CT,  treatableq2, maxMinDepth_q0_att, maxMinDepth_q0, dang2CT, dang2= plothmbatch(df,save=f"{f1}/")
        else:
            TVs, treatableq2CT,  treatableq2L,  treatableq2, maxMinDepth_q0_att, maxMinDepth_q0_attLung, maxMinDepth_q0,minMinDepth_q0_att,minMinDepth_q0_attLung,minMinDepth_q0, dang2CT, dang2L, dang2= planDetermineMinMaxDepths(df)
        for i in range(len(TVs)):
            try:
                results['PlanSetupSer'].append(int(f1.split('\\')[-1]))
            except:
                results['PlanSetupSer'].append(f1.split('\\')[-1])
            results["Name"].append(TVs[i])
            results["Treatable q0_att"].append(treatableq2CT[i])  
            results["Treatable q0_attLung"].append(treatableq2L[i]) 
            results["Treatable q0"].append(treatableq2[i])
            results["Max Depth q0_att"].append(maxMinDepth_q0_att[i])  
            results["Max Depth q0_attLung"].append(maxMinDepth_q0_attLung[i]) 
            results["Max Depth q0"].append(maxMinDepth_q0[i])
            results["Min Depth q0_att"].append(minMinDepth_q0_att[i])  
            results["Min Depth q0_attLung"].append(minMinDepth_q0_attLung[i]) 
            results["Min Depth q0"].append(minMinDepth_q0[i])
            results["Mean Depth q0_att per Angle"].append(dang2CT[i])
            results["Mean Depth q0_attLung per Angle"].append(dang2L[i])
            results["Mean Depth q0 per Angle"].append(dang2[i])
    df=pd.DataFrame.from_dict(results)
    if 'done' in locals():
        df=pd.concat((done,df))
    df.to_pickle(f"{folder}/Treatable{interval if np.any(interval) else ''}.pickle")
    return df

### TV Mesh/Shape Analysis

In [8]:
import trimesh as tm
from scipy.stats import skew

from sklearn.neighbors import NearestNeighbors


def chamfer_distance(x, y, metric='l2', direction='bi'):
    """Chamfer distance between two point clouds
    Parameters
    ----------
    x: numpy array [n_points_x, n_dims]
        first point cloud
    y: numpy array [n_points_y, n_dims]
        second point cloud
    metric: string or callable, default 'l2'
        metric to use for distance computation. Any metric from scikit-learn or scipy.spatial.distance can be used.
    direction: str
        direction of Chamfer distance.
            'y_to_x':  computes average minimal distance from every point in y to x
            'x_to_y':  computes average minimal distance from every point in x to y
            'bi': compute both
    Returns
    -------
    chamfer_dist: float
        computed bidirectional Chamfer distance:
            sum_{x_i \\in x}{\\min_{y_j \\in y}{||x_i-y_j||**2}} + sum_{y_j \\in y}{\\min_{x_i \\in x}{||x_i-y_j||**2}}

    https://gist.github.com/sergeyprokudin/c4bf4059230da8db8256e36524993367
    """
    
    if direction=='y_to_x':
        x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(x)
        min_y_to_x = x_nn.kneighbors(y)[0]
        chamfer_dist = np.mean(min_y_to_x)
    elif direction=='x_to_y':
        y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(y)
        min_x_to_y = y_nn.kneighbors(x)[0]
        chamfer_dist = np.mean(min_x_to_y)
    elif direction=='bi':
        x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(x)
        min_y_to_x = x_nn.kneighbors(y)[0]
        y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(y)
        min_x_to_y = y_nn.kneighbors(x)[0]
        chamfer_dist = np.mean(min_y_to_x) + np.mean(min_x_to_y)
    else:
        raise ValueError("Invalid direction type. Supported types: \'y_x\', \'x_y\', \'bi\'")
        
    return chamfer_dist

def genMeshfromVox(vox,spacing,slicespacing,debug):
    mesh = tm.voxel.ops.matrix_to_marching_cubes(vox.transpose((1,0,2)))

    #rescale binary to real dimensions of PTV
    mesh.vertices=mesh.vertices*[spacing[0],spacing[1],slicespacing]#+[minxyz[0],minxyz[1],unz[0]]
    mc=mesh.center_mass
    mesh.vertices-=mc
    
    pca = PCA(3)
    pca.fit(mesh.vertices)
    if not debug: mesh.vertices = pca.transform(mesh.vertices)
    
    # compactness (Ratio of the volume of the object to the volume of a sphere with the same surface area. More compact shapes have values closer to 1.)
    r2=np.sqrt(mesh.area/(4*np.pi))
    cs=np.abs(mesh.volume)/(mesh.area*r2/3)  

    #solidity
    s=np.abs(mesh.volume/mesh.convex_hull.volume )

    return  mesh, mc, pca, cs, s
def genMesh(df,XYZ, PSS, debug=0,spacing=None):

    maxxyz=np.max(XYZ,axis=0)
    unz = np.unique(XYZ[:,2])

    if not np.any(spacing):
        oss = df[df["PlanSetupSer"]==PSS].iloc[0]['CTOriginSpacingSize']
        slicespacing=oss[1][2]
        try:
            minxyz,spacing,size=getoss(XYZ)
            spacing/=2 #?? depends on the interpolation in to_grid
            size*=2 #??
        except:
            minxyz=np.min(XYZ,axis=0)
            spacing=np.abs(oss[1][:2])/2
            size=np.roll(np.ceil(np.abs((np.max(XYZ[:,:2],0)-np.min(XYZ[:,:2],0)))/spacing[0]).astype(int),1)+np.array([4,4])
    
    else:
        minxyz=np.min(XYZ,axis=0)
        size=np.roll(np.ceil(np.abs((np.max(XYZ[:,:2],0)-np.min(XYZ[:,:2],0))/spacing[0])).astype(int),1)+np.array([4,4])
        slicespacing=spacing[2]
        spacing=spacing[:2]
    nslices=np.ceil(np.abs((maxxyz[2]-minxyz[2])/slicespacing)+1).astype(int) 
    vox = np.full((size[0],size[1],nslices),False)
    for _,z in enumerate(unz): 
        vox[:,:,((z-minxyz[2])/slicespacing).astype(int)]=binary_erosion(binary_fill_holes(binary_dilation(to_grid(XYZ,z,minxyz,spacing,size)))) #problem with double walls
    
    if debug: 
        print(f"{maxxyz=},{minxyz=},{vox.shape}")
        fig = px.imshow(vox[:,:,j], color_continuous_scale='gray',title=z)
        fig.show()
        fig = px.imshow(to_grid(XYZ,z,minxyz,spacing,size), color_continuous_scale='gray',title=z)
        fig.show()

  
    mesh, mc, pca, cs, s = genMeshfromVox(vox,spacing,slicespacing,debug)
    return mesh, mc, vox, minxyz, maxxyz, spacing, slicespacing, size, pca, cs, s

def compileTVShapes(dfmain,save=None,debug=False,interval=None):
    """
    This is a wrapper for the genMesh function.

    In:
    - dfmain : Pandas Dataframe, PTV.pickle generated by batchAnalyzeStructuresCT_sheet
    - save : Bool, save pcikle or only return dataframe
    - debug : Bool, show some additional info
    - interval : int array [Start, End], only use specified rows of dfmain
    
    Out:
    - PCAMesh.pickle : Pandas dataframe, containing the columns:
    'PlanSetupSer',"Name",'TV Type', 'CTOriginSpacingSize','PTVOriginSpacingSize', 'Center', 'max' ,'min', 'PC', 'PC_Var','PC_Skew', 'PC_min', 'PC_max','Size', 'Solidity', 'Compactness', "Volume"}
    
    """
    PSSs=dfmain['PlanSetupSer'].unique()
    if not np.any(interval==None):
        PSSs=PSSs[interval[0]:interval[1]]
    if save: logging.basicConfig(filename=f'{save}/LogsPCA.log', encoding='utf-8', level=logging.WARNING)
    results2 = {'PlanSetupSer':[],"Name" : [],'TV Type':[], 'CTOriginSpacingSize':[],'PTVOriginSpacingSize':[], 'Center' : [], 'max' : [],'min' : [], 'PC' : [], 'PC_Var' : [],'PC_Skew' : [], 'PC_min' : [], 'PC_max' : [],'Size' : [], 'Solidity' : [], 'Compactness' : [], "Volume" : []}
    for j,PSS in enumerate(PSSs):
        dfpss=dfmain[dfmain['PlanSetupSer']==PSS]
        print(j, PSS)
        for TVType in ['PTV','CTV','GTV']:
            df=dfpss[dfpss["TV Type"]==TVType]
            if len(df['Name']) > 1:
                cumvox=None
                cumvox_minxyz=None
                cumvox_maxxyz=None
                cumvox_spacing=None
            for i in range(len(df)):
                try:
                    XYZ=df['XYZ'].iloc[i]
                    if debug: print(df['Name'].iloc[i])
                    if i > 0:
                        mesh, mc, vox, minxyz, maxxyz, spacing, slicespacing, size,pca,cs,s = genMesh(df,XYZ,df['PlanSetupSer'].iloc[i],debug,spacing=cumvox_spacing)
                    else: 
                        mesh, mc, vox, minxyz, maxxyz, spacing, slicespacing, size,pca,cs,s = genMesh(df,XYZ,df['PlanSetupSer'].iloc[i],debug)
                    if len(df['Name']) > 1:
                        if not np.any(cumvox):
                            cumvox=vox
                            cumvox_minxyz=minxyz
                            cumvox_maxxyz=maxxyz
                            cumvox_spacing=np.array([spacing[0],spacing[1],slicespacing])
                        else:
                            cumvox_minxyz_temp=np.min([cumvox_minxyz,minxyz],axis=0)
                            cumvox_maxxyz=np.max([cumvox_maxxyz,maxxyz],axis=0)
                            cumvox_size = np.ceil(np.abs((cumvox_maxxyz-cumvox_minxyz_temp))/(cumvox_spacing*np.array([2,2,1]))).astype(int)*np.array([2,2,1])+np.array([4,4,2])
                            cumvox_temp = np.full((cumvox_size[1],cumvox_size[0],cumvox_size[2]),False)
                            start=coordinate_space(cumvox_minxyz,cumvox_minxyz_temp,cumvox_spacing,True)[[1,0,2]]
                            cumvox_temp[start[0]:start[0]+cumvox.shape[0],start[1]:start[1]+cumvox.shape[1],start[2]:start[2]+cumvox.shape[2]]=cumvox
                            start=coordinate_space(minxyz,cumvox_minxyz_temp,cumvox_spacing,True)[[1,0,2]]
                            cumvox_temp[start[0]:start[0]+vox.shape[0],start[1]:start[1]+vox.shape[1],start[2]:start[2]+vox.shape[2]] = cumvox_temp[start[0]:start[0]+vox.shape[0],start[1]:start[1]+vox.shape[1],start[2]:start[2]+vox.shape[2]] | vox
                            cumvox=cumvox_temp
                            cumvox_minxyz=cumvox_minxyz_temp
                        results2["TV Type"].append(df['TV Type'].iloc[i])
                    else:
                        results2["TV Type"].append(f"{df['TV Type'].iloc[i]} Solo")
                    results2["Name"].append(df['Name'].iloc[i])
                    results2["PlanSetupSer"].append(PSS)
                    results2["CTOriginSpacingSize"].append(df['CTOriginSpacingSize'].iloc[i])
                    results2['PTVOriginSpacingSize'].append([minxyz,spacing,size])
                    results2["Center"].append(np.mean(XYZ,axis=0))
                    results2["min"].append(minxyz)
                    results2["max"].append(maxxyz)
                    results2["PC"].append(pca.components_)
                    results2["PC_Var"].append(pca.explained_variance_)
                    results2["PC_Skew"].append(skew(mesh.vertices))
                    results2["PC_min"].append(np.min(mesh.vertices,axis=0))
                    results2["PC_max"].append(np.max(mesh.vertices,axis=0))
                    results2["Size"].append(4*np.mean(np.abs(mesh.vertices),axis=0))
                    results2["Compactness"].append(cs)
                    results2["Solidity"].append(s)
                    results2["Volume"].append(np.sum(vox)*np.prod(spacing)*slicespacing/1000)
                except BaseException:
                    logging.exception(f"PCA failed for PTV {df['Name'].iloc[i], PSS}.")
                    continue

                if debug:
                    data=[]
                    XYZ-=np.mean([maxxyz,minxyz],0)
                    mesh.vertices-=np.mean([ results2["PC_max"][-1], results2["PC_min"][-1]],0)
                    color=np.zeros(XYZ.shape)
                    color[::2]=1
                    data.append(go.Scatter3d(x=XYZ[:,0],y=XYZ[:,1],z=XYZ[:,2],mode='markers',marker=dict(size=2,color=color,opacity=0.8),name=df['PTV'].iloc[i]))
                    data.append(go.Scatter3d(x=mesh.vertices[:,0],y=mesh.vertices[:,1],z=mesh.vertices[:,2],mode='markers',marker=dict(size=2,opacity=0.8),name=f"Vertices {df['PTV'].iloc[i]}"))
                    fig=go.Figure(data=data)
                    fig.show()
                    input()
            
            if len(df['Name']) > 1 and np.any(cumvox):
                mesh, mc, pca, cs, s = genMeshfromVox(cumvox,cumvox_spacing[:2],cumvox_spacing[2],debug)
                coords=np.transpose(np.nonzero(cumvox))
                cumpts=real_space(coords,cumvox_minxyz,cumvox_spacing)
                results2["Name"].append(f"{df['TV Type'].iloc[i]} All Python")
                results2["TV Type"].append(f"{df['TV Type'].iloc[i]} All")
                results2["PlanSetupSer"].append(PSS)
                results2["CTOriginSpacingSize"].append(df['CTOriginSpacingSize'].iloc[i])
                results2['PTVOriginSpacingSize'].append([cumvox_minxyz,cumvox_spacing,cumvox_size])
                results2["Center"].append(np.mean(cumpts,axis=0))
                results2["min"].append(cumvox_minxyz)
                results2["max"].append(cumvox_maxxyz)
                results2["PC"].append(pca.components_)
                results2["PC_Var"].append(pca.explained_variance_)
                results2["PC_Skew"].append(skew(mesh.vertices))
                results2["PC_min"].append(np.min(mesh.vertices,axis=0))
                results2["PC_max"].append(np.max(mesh.vertices,axis=0))
                results2["Size"].append(4*np.mean(np.abs(mesh.vertices),axis=0))
                results2["Compactness"].append(cs)
                results2["Solidity"].append(s)
                results2["Volume"].append(np.sum(cumvox)*np.prod(cumvox_spacing)/1000)
    if not debug:
        df = pd.DataFrame.from_dict(results2)
        if save:
            df.to_pickle(f"{save}/PCAMesh{interval if interval else ''}.pickle")
        return df

### Plan

In [9]:
def analyzePlan(RP):
    """Function to read metrics from RT Plan DICOM files"""
    try:
        PMRN=int(RP.PatientID)
    except:
        PMRN=np.nan 
    try:
        SUID=RP.ReferencedStructureSetSequence[0].ReferencedSOPInstanceUID
    except:
        SUID=''
    try:
        PlanLabel=RP.RTPlanLabel
    except:
        PlanLabel=''
    angle=[]
    mu=[]
    beam=[]
    iso=[]
    Meterset=[]
    Dose=[]
    Roll=[]
    Pitch=[]
    Yaw=[]
    TableTopEccentricAngle=[]
    BeamLimitingDevicePositionSequence=[]
    BeamDeliveryDurationLimit=[]
    BeamType=[]
    RadiationType=[]
    HighDoseTechniqueType=[]
    DoseRateSet=[]
    BeamLimitingDeviceAngle=[]
    SSD=[]
    BeamDict = {}
    # print(RP.BeamSequence)
    # print(RP.FractionGroupSequence[0])
    for rbs in RP.FractionGroupSequence[0].ReferencedBeamSequence:
        try:
            bms=float(rbs.BeamMeterset) 
        except:
            bms=np.nan
        try:
            BDDL=float(rbs.BeamDeliveryDurationLimit)
        except:
            BDDL=np.nan
        try:
            BD=float(rbs.BeamDose) 
        except:
            BD=np.nan
        BeamDict[rbs.ReferencedBeamNumber]=[bms,BDDL,BD]
    for bs in RP.BeamSequence:
        if bs.TreatmentDeliveryType == 'TREATMENT' and bs.PrimaryDosimeterUnit == 'MU':
            cmw_temp=0
            angles=[]
            BLDPSs=[]
            MUs=[]
            GA_temp=np.nan
            BLDPS_temp=None
            for cp in bs.ControlPointSequence:
                try:
                    mu1=BeamDict[bs.BeamNumber][0]*(-cmw_temp+cp.CumulativeMetersetWeight)#ReferencedDoseReferenceSequence[0].CumulativeDoseReferenceCoefficient
                    cmw_temp=cp.CumulativeMetersetWeight
                    try: #Needed for static treatments
                        GA_temp=cp.GantryAngle
                        BLDPS_temp=cp.BeamLimitingDevicePositionSequence
                    except:
                        pass
                    if mu1>0:
                        MUs.append(mu1)
                        angles.append(GA_temp)
                        BLDPSs.append(BLDPS_temp)
                except:
                    logging.exception(f"MU dose failed for patient {RP.PatientID}")
                    continue
                    # logging.exception(f"MU angle failed for patient {RP.PatientID}")
            try:
                iso.append(np.array(bs.ControlPointSequence[0].IsocenterPosition)) #CONTINUE: Try except nan
            except:
                iso.append(np.nan)
            try:    
                Roll.append(float(bs.ControlPointSequence[0].TableTopRollAngle))
            except:
                Roll.append(np.nan)
            try:
                Pitch.append(float(bs.ControlPointSequence[0].TableTopPitchAngle))
            except:
                Pitch.append(np.nan)
            try:
                Yaw.append(float(bs.ControlPointSequence[0].PatientSupportAngle))
            except:
                Yaw.append(np.nan)
            try:
                BeamType.append(bs.BeamType)
            except:
                BeamType.append(np.nan)
            try:
                RadiationType.append(bs.RadiationType)
            except:
                RadiationType.append(np.nan)
            try:
                HighDoseTechniqueType.append(bs[0x300a, 0x00c7].value)
            except:
                HighDoseTechniqueType.append(np.nan)
            try:
                DoseRateSet.append(float(bs.ControlPointSequence[0].DoseRateSet))
            except:
                DoseRateSet.append(np.nan)
            try:
                SSD.append(float(bs.ControlPointSequence[0][0x300a, 0x0130].value)/10)
            except:
                SSD.append(np.nan)
            try:
                BeamLimitingDeviceAngle.append(float(bs.ControlPointSequence[0].BeamLimitingDeviceAngle))
            except:
                BeamLimitingDeviceAngle.append(np.nan)
            try:
                TableTopEccentricAngle.append(float(bs.ControlPointSequence[0].TableTopEccentricAngle))
            except:
                TableTopEccentricAngle.append(np.nan)
            beam.append(bs.BeamName)
            Meterset.append(BeamDict[bs.BeamNumber][0])
            Dose.append(BeamDict[bs.BeamNumber][2])
            BeamDeliveryDurationLimit.append(BeamDict[bs.BeamNumber][1])
            angle.append(angles)
            mu.append(MUs)
            BeamLimitingDevicePositionSequence.append(BLDPSs)
            
    return PMRN, SUID, PlanLabel, angle, mu, beam, Roll, Pitch, Yaw, TableTopEccentricAngle,BeamLimitingDevicePositionSequence,BeamDeliveryDurationLimit, iso, Meterset,Dose,BeamDeliveryDurationLimit,BeamLimitingDeviceAngle,SSD,DoseRateSet,HighDoseTechniqueType,RadiationType,BeamType


def batchAnalyzePlan(save,locsheet=DICOMLocations,interval=None):
    """
    Wrapper to read plans in batch mode.

    In:
    - save; str, save location
    - locsheet; str, xlsx file with DICOM plan locations
    - interval; int array [start, end], only read specified row from loc sheet

    Out:
    - MU.pickle ; Pandas dataframe, contains the plan information from DICOM plans. Columns: 'PMRN', 'StructureUID', 'Rx Name' ,'Beam Name' , 'Meterset [MU]', 'Gantry Angle', 'MU per Angle', 'Beam Delivery Duration Limit [s]','Table Top Roll Angle','Table Top Pitch Angle',
                 'Patient Support Angle','Table Top Eccentric Angle','Beam Limiting Device Angle','SSD [cm]','Dose Rate Set [MU/min]',
                   'Dose [Gy]','High Dose Technique Type','Radiation Type','Beam Type', 'Isocenter [mm]'
    """
    resultsMU = {'PMRN' : [], 'StructureUID' : [], 'Rx Name' : [] ,'Beam Name' : [], 'Meterset [MU]' : [], 'Gantry Angle' : [], 'MU per Angle' : [], 'Beam Delivery Duration Limit [s]':[],'Table Top Roll Angle' :[],'Table Top Pitch Angle':[],
                 'Patient Support Angle' : [],'Table Top Eccentric Angle':[],'Beam Limiting Device Angle' : [],'SSD [cm]' : [],'Dose Rate Set [MU/min]' : [],
                   'Dose [Gy]':[],'High Dose Technique Type':[],'Radiation Type':[],'Beam Type':[], 'Isocenter [mm]':[]} #'Beam Limiting Device Position Sequence' : [],
    if type(locsheet)==str:
        locs=updateLocs(locsheet)
    else:
        locs=locsheet
    if not np.any(interval==None):
        locs=locs[interval[0]:interval[1]]
    if "PlanSetupSer" in locs.keys():
        resultsMU['PlanSetupSer'] = []
        havePSS=True
    else:
        havePSS=False
    for i,RPfileloc in enumerate(locs["Plan_Path"]):
        if i%100==0: print(i/len(locs))
        if havePSS: PSS=locs["PlanSetupSer"].iloc[i]
        try:
            RPfile=pydicom.read_file(RPfileloc)
        except:
            continue
        if RPfile.SOPInstanceUID == locs["RTPlan_UID"].iloc[i]:
            PMRN, SUID, PlanLabel, angle, mu, beam, Roll, Pitch, Yaw, TableTopEccentricAngle,BeamLimitingDevicePositionSequence,BeamDeliveryDurationLimit, iso, Meterset,Dose,BeamDeliveryDurationLimit,BeamLimitingDeviceAngle,SSD,DoseRateSet,HighDoseTechniqueType,RadiationType,BeamType=analyzePlan(RPfile)
            if len(angle)!=0: 
                if havePSS: resultsMU['PlanSetupSer'].append(PSS)
                resultsMU['PMRN'].append(PMRN)
                resultsMU['StructureUID'].append(SUID)
                resultsMU['Rx Name'].append(PlanLabel)
                resultsMU["Gantry Angle"].append(angle)
                resultsMU["MU per Angle"].append(mu)
                resultsMU["Beam Name"].append(beam)
                resultsMU["Table Top Roll Angle"].append(Roll)
                resultsMU["Table Top Pitch Angle"].append(Pitch)
                resultsMU["Patient Support Angle"].append(Yaw)
                resultsMU["Table Top Eccentric Angle"].append(TableTopEccentricAngle)
                #resultsMU["Beam Limiting Device Position Sequence"].append(BeamLimitingDevicePositionSequence)
                resultsMU["Beam Delivery Duration Limit [s]"].append(BeamDeliveryDurationLimit)
                resultsMU["Isocenter [mm]"].append(iso)
                resultsMU["Meterset [MU]"].append(Meterset)
                resultsMU["Dose [Gy]"].append(Dose)
                resultsMU["Beam Limiting Device Angle"].append(BeamLimitingDeviceAngle)
                resultsMU["SSD [cm]"].append(SSD)
                resultsMU["Dose Rate Set [MU/min]"].append(DoseRateSet)
                resultsMU["High Dose Technique Type"].append(HighDoseTechniqueType)
                resultsMU["Radiation Type"].append(RadiationType)
                resultsMU["Beam Type"].append(BeamType)
    dfMU = pd.DataFrame.from_dict(resultsMU)
    if not os.path.exists(save):
        os.makedirs(save)
    if not np.any(interval==None):
        dfMU.to_pickle(f"{save}/MU{interval}.pickle")
    else:    
        try:
            shutil.copyfile(f"{save}/MU.pickle",f"{save}/MU.pickle.old")
            MU=pd.read_pickle(f"{save}/MU.pickle")
            dfMU=pd.concat([MU,dfMU])
            dfMU.to_pickle(f"{save}/MU.pickle")
        except:
            if not os.path.exists(f"{save}/MU.pickle"):
                dfMU.to_pickle(f"{save}/MU.pickle")
            else:
                logging.exception(f"MU.pickle exists, but appending failed. Create PTVnew.")
                dfMU.to_pickle(f"{save}/MUnew.pickle")
    return dfMU



### Dose

In [10]:
def signed_bwdist(im):
    '''
    Find perim and return masked image (signed/reversed)
    '''    
    im = -bwdist(bwperim(im))*np.logical_not(im) + bwdist(bwperim(im))*im
    return im

def bwdist(im):
    '''
    Find distance map of image
    '''
    dist_im = distance_transform_edt(1-im)
    return dist_im

def bwperim(im):
    return 0 - (im ^ binary_dilation(im))

def interp_shape(top, bottom, precision):
    '''
    Interpolate between two contours

    Input: top 
            [X,Y] - Image of top contour (mask)
           bottom
            [X,Y] - Image of bottom contour (mask)
           precision
             float  - % between the images to interpolate 
                Ex: num=0.5 - Interpolate the middle image between top and bottom image
    Output: out
            [X,Y] - Interpolated image at num (%) between top and bottom

    '''
    if precision>2:
        print("Error: Precision must be between 0 and 1 (float)")

    top = signed_bwdist(top)
    bottom = signed_bwdist(bottom)

    # row,cols definition
    r, c = top.shape

    # Reverse % indexing
    precision = 1+precision

    # rejoin top, bottom into a single array of shape (2, r, c)
    top_and_bottom = np.stack((top, bottom))

    # create ndgrids 
    points = (np.r_[0, 2], np.arange(r), np.arange(c))
    xi = np.rollaxis(np.mgrid[:r, :c], 0, 3).reshape((r*c, 2))
    xi = np.c_[np.full((r*c),precision), xi]

    # Interpolate for new plane
    out = interpn(points, top_and_bottom, xi)
    out = out.reshape((r, c))

    # Threshold distmap to values above 0
    out = out > 0

    return out

def voxdose(struc,origin,spacing,slicepos,dose):
    """Interpolate between slices to fit other resolution"""
    unz=np.unique(struc[:,2])
    imgslices=dict()
    for z in unz:
        imgslices[np.round(z,3)]=binary_erosion(binary_fill_holes(binary_dilation(to_grid(struc,z,origin,spacing,[dose.shape[0],dose.shape[1]]))))
    slicespcaingstruc=np.abs(np.median(unz-np.roll(unz,1))) 
    # print(slicepos,unz)
    vox = np.full(dose.shape,False)
    for i,z in enumerate(slicepos): 
        XYZint=struc
        if not z in unz:
            # find closest slices. Incorrect for disconnected structs, which will become connected. Introduce max distance betwen slices at 5mm?
            pos=unz-z
            if np.any(pos<0) and np.any(pos>0):
                l=np.max(pos[pos<0])
                u=np.min(pos[pos>0])
                lu=-l+u
                if lu<=slicespcaingstruc:
                    # Run interpolation
                    XYZint = interp_shape(imgslices[np.round(u+z,3)],imgslices[np.round(l+z,3)], np.abs(l)/lu)
                    vox[:,:,i]=XYZint
        else:
            vox[:,:,i]=imgslices[np.round(z,3)]
    return vox

def tryexceptMinDist(pc,XYZOARi):
    """Reduce size of point clouds until they fit memory in a quick n dirty way"""
    try:
        result=np.min(np.linalg.norm(pc[:,np.newaxis]-XYZOARi,axis=2))
    except np.core._exceptions._ArrayMemoryError:
        result=tryexceptMinDist(pc[::2],XYZOARi[::2])
    except:
        result=np.nan
    return result

def analyzeDose(RDfile,XYZ,ri,RS, Names, TimmermanFrac,debug=True,calc=True,TimmermanDict=TimmermanDict):
    """Reads RT DICOM dose files in combination with structure files to calculate dose metrics related to Timmerman tables. Could probably be merged with depth code to save some run time.
    In:
    RDfile : DICOM Dose file
    XYZ : list of PTV contours. Set to 'None' to load PTV names.
    ri : list of ints ; Prescribed Target Doses
    RS : RT DICOM Structure file
    Names : list of str ; Names of PTVs. Set to 'None' to find PTVs
    TimmermanFrac : int ; Number of Fractions in Timmerman List to compare to
    debug : bool ; Prints some extra info
    calc : bool ; Uses stored dose per voxel to recalculate metrics, if off only DVH are loaded
    TimmermanDict : dict ; contains Timmermann dose constraints
    """
    # Initialize result dicts
    ctVri={}
    cV={}
    cmind={}
    cmaxd={}
    cmeand={}
    ctotald={}
    coverlap={}
    coverlapborder={}
    Vri=0
    tVri={}
    V={}
    mind={}
    maxd={}
    meand={}
    timvoldose={}
    dvhdict={}
    ROIdict={}
    ROIdictOAR={}
    ROIdictOARsearch={}
    ROIdictorigname={}
    NumberPTVs=len(XYZ)   
    NamesN=[]     
    Searchterms=['None' for i in Names] 
    SearchtermsN=[]
    OrigNames=Names.copy()
    OrigNamesN=[]
    Names=['PTV' for i in Names]
    #Store Structure names and match them to categories
    f = open("StructureNames.txt", "a")
    for s in RS.StructureSetROISequence:
        # print(s.ROIName)
        cfname=s.ROIName.casefold()
        found=False
        if cfname[0] not in 'xz^' and 'ptv' not in cfname and 'ctv' not in cfname and 'gtv' not in cfname:  
            f.write(cfname)   
            f.write("\n")
            for oar in TimmermanDict.keys():
                if oar in cfname:
                    found=oar
                    break      
        if cfname == 'body':
            ROIdict[s.ROINumber] = 'body'
            ROIdictOARsearch[s.ROINumber]= "body"
            ROIdictorigname[s.ROINumber]='body'
        elif found:
            ROIdictOARsearch[s.ROINumber]=found
            ROIdictOAR[s.ROINumber]=TimmermanDict[found]
            ROIdictorigname[s.ROINumber]=cfname
            if debug: print(cfname, found, TimmermanDict[found])
        else:
            if np.any(OrigNames==None):
                if "ptv" in cfname[:4]: #in general want PTV and then blank,_,^,number
                    if not "avoid" in cfname and not "overlap" in cfname:
                        ROIdictorigname[s.ROINumber]=s.ROIName
                        ROIdict[s.ROINumber] = 'PTV'
            else:
                if s.ROIName in OrigNames:
                    ROIdictorigname[s.ROINumber]=s.ROIName
                    ROIdict[s.ROINumber] = 'PTV'
    f.close()
    #Match structures to their contours
    ctrs = RS.ROIContourSequence
    OARs = []
    TVs = []
    for c in ctrs:
        try:
            _=c.ContourSequence
        except:
                continue
        try:
            Names.append(ROIdictOAR[c.ReferencedROINumber])
            Searchterms.append(ROIdictOARsearch[c.ReferencedROINumber])
            OrigNames.append(ROIdictorigname[c.ReferencedROINumber])
            OARs.append(c)
        except: 
            TV=ROIdict.get(c.ReferencedROINumber, False)
            if TV:
                TVs.append(TV)
    if np.any(XYZ==None):
        XYZ=[get_points([PTV],True) for PTV in TVs]
    #Calculate min distance between PTV and OAR contours
    if len(OARs)>0 and len(Names):
        XYZOAR=[get_points([OAR],False)[0] for OAR in OARs]
        MinDists=np.array([[tryexceptMinDist(pc,XYZOARi) for XYZOARi in XYZOAR] for pc in XYZ])
        NamesPTVDists=np.repeat(OrigNames[:len(XYZ)],len(XYZOAR))
        NamesOARDists=np.concatenate(np.repeat([OrigNames[len(XYZ):]],len(XYZ),axis=0))
        XYZ.extend(XYZOAR)
    else:
        MinDists=np.array([])
        NamesPTVDists=[]
        NamesOARDists=[]
    Centers=dict(zip(OrigNames,[np.mean(pc,axis=0) for pc in XYZ]))
    Centers['body']=np.array([np.nan,np.nan,np.nan])
    ROIdict = ROIdict | ROIdictOAR
    ridict = dict(zip(OrigNames,ri)) | dict(zip(TimmermanFrac['OAR'],TimmermanFrac['Volume max [Gy]']*100))#dict(zip(OARNames,OARLimits))
    ridict['body']=np.min(ri)
    rivoldict = dict(zip(TimmermanFrac['OAR'],np.array(TimmermanFrac['Volume [cc]'],dtype=float)))
    #Get Metrics from DVHs
    for DVH in RDfile.DVHSequence:
        try:
            name=ROIdict[DVH.DVHReferencedROISequence[0].ReferencedROINumber]
            searchterm=ROIdictOARsearch.get(DVH.DVHReferencedROISequence[0].ReferencedROINumber, "None")
            origname=ROIdictorigname.get(DVH.DVHReferencedROISequence[0].ReferencedROINumber, "None")
        except:
            continue
        NamesN.append(name)
        SearchtermsN.append(searchterm)
        OrigNamesN.append(origname)
        DVHdose=np.cumsum(DVH.DVHData[::2])*100
        DVHvols=np.array(DVH.DVHData[1::2])
        try:
            if name=='PTV':
                rin=ridict[origname]
            else:
                rin=ridict[name]
        except:
            print(ri,name,"Not found in Ridict",Names)
            rin=np.nan #ri[0]
        if np.isnan(rin):
            tVri[origname]=np.nan
        else:
            tVri[origname]=DVHvols[np.argmin(np.abs(DVHdose-rin))]
        if name == 'body':
            Vri=tVri["body"]
        V[origname]=float(DVHvols[0])
        dif=(np.concatenate(([0],DVHvols))-np.roll(np.concatenate(([0],DVHvols)),len(DVHvols)))[1:]
        mind[origname]=DVHdose[DVHvols==DVHvols[0]][-1]#float(DVH.DVHMinimumDose)*100
        maxd[origname]=np.max(DVHdose)#float(DVH.DVHMaximumDose)*100 Sometimes completely off, whyever
        meand[origname]=np.sum(DVHdose*dif)/np.sum(dif)#float(DVH.DVHMeanDose)*100
        dvhdict[origname]=np.array(DVH.DVHData)
        rivol=rivoldict.get(name,np.nan)
        if np.isnan(rivol):
            timvoldose[origname]=np.nan
        elif rivol<0:
            timvoldose[origname]=DVHdose[np.argmin(np.abs(DVHvols-DVHvols[np.argmin(np.abs(DVHdose-meand[origname]))]))]
        else:
            timvoldose[origname]=DVHdose[np.argmin(np.abs(DVHvols-rivol))]
    for namei,origname in enumerate(OrigNamesN):
        # print(name, name in NamesN)
        if not origname in OrigNamesN:
            NamesN.append(NamesN[namei])
            SearchtermsN.append(Searchterms[namei])
            OrigNamesN.append(origname)
            V[origname]=np.nan
            tVri[origname]=np.nan
            maxd[origname]=np.nan
            mind[origname]=np.nan
            meand[origname]=np.nan
            dvhdict[origname]=np.nan
    try:
        V['body']
    except:
        V['body']=np.nan
        tVri['body']=np.nan
        Vri=np.nan
        maxd['body']=np.nan
        mind['body']=np.nan
        meand['body']=np.nan
        dvhdict['body']=np.nan

    if calc:
        dose1=(RDfile.pixel_array*RDfile.DoseGridScaling).transpose(1,2,0)*100
        zoomfactor=2
        dose=zoom(dose1,zoomfactor,grid_mode=True,mode='grid-constant')
        if debug:
            print(dose1.shape)
            fig1 = px.imshow(dose1[:,:,54])#, animation_frame=2, labels=dict(animation_frame="Slice"))
            fig1.show()
            fig = px.imshow(dose[:,:,108])#[::3,::3,::3], animation_frame=2, labels=dict(animation_frame="Slice"))
            fig.show()
            print(ri,np.max(dose))
            print(dose.shape)
            input()
        origin=np.array(RDfile.ImagePositionPatient)
        spacing=np.array(RDfile.PixelSpacing)/zoomfactor
        slicepos=np.array(RDfile.GridFrameOffsetVector)
        # print(slicepos)
        # slicepos=zoom(slicepos,zoomfactor,grid_mode=True) #continue
        slicespacing0=slicepos-np.roll(slicepos,1)
        if np.any(slicespacing0[2:-2]!=slicespacing0[3]):
            logging.exception(f"Slicepos is not always evenly spaced. Change code{RDfile.ReferencedRTPlanSequence[0].ReferencedSOPInstanceUID}")
        slicespacing=np.abs(np.median(slicespacing0)) / zoomfactor
        slicepos=np.cumsum(np.ones(dose.shape[2])*slicespacing)-slicespacing
        # print(slicepos)
        voxtovol=np.prod(spacing)*slicespacing/1000
        cVri=np.sum(dose>np.min(ri))*voxtovol
        if slicepos[0]==0:
            slicepos+=origin[2]
        vox_acc_dict={}
        vals,counts=np.unique(Names,return_counts=True)
        for val in vals[counts>1]:
            vox_acc_dict[val]=np.full(dose.shape,False)
        if not 'PTV' in vox_acc_dict.keys():
            vox_acc_dict['PTV']=np.full(dose.shape,False)
        minz=np.inf
        maxz=-np.inf
        #Recalculate metrics using voxel dose
        for k,struc in enumerate(XYZ):
            minz=np.min([minz,np.min(struc[:,2])])
            maxz=np.max([maxz,np.max(struc[:,2])])
            vox=voxdose(struc,origin,spacing,slicepos,dose)
            cV[OrigNames[k]]=np.sum(vox)*voxtovol
            tdose=dose[vox]
            cmaxd[OrigNames[k]]=np.max(tdose)
            cmind[OrigNames[k]]=np.min(tdose)
            cmeand[OrigNames[k]]=np.mean(tdose)
            ctotald[OrigNames[k]]=np.sum(tdose)
            if Names[k]=='PTV':
                coverlap[OrigNames[k]]=cV[OrigNames[k]]
                coverlapborder[OrigNames[k]]=1
                ctVri[OrigNames[k]]=np.sum((dose>ridict.get(OrigNames[k],np.nan)) & vox)*voxtovol
            else:
                coverlap[OrigNames[k]]=np.sum((vox & vox_acc_dict['PTV']))*voxtovol
                inner=binary_dilation(vox_acc_dict['PTV'],iterations=1)
                circ=binary_dilation(inner,iterations=1)^inner
                intsec=circ & vox
                coverlapborder[OrigNames[k]]= np.sum(intsec)/np.sum(circ)
                ctVri[OrigNames[k]]=np.sum((dose>ridict.get(Names[k],np.nan)) & vox)*voxtovol
            if Names[k] in vox_acc_dict.keys():
                vox_acc_dict[Names[k]]=(vox | vox_acc_dict[Names[k]])
        vox=np.full(dose.shape,False)
        vox[:,:,np.max([0,int((minz-origin[2])/slicespacing)-10]):int((maxz-origin[2])/slicespacing)+10]=dose[:,:,np.max([0,int((minz-origin[2])/slicespacing)-10]):int((maxz-origin[2])/slicespacing)+10]>1e-5
        vox=(vox ^ vox_acc_dict['PTV'])
        ctVri['body']=np.sum(((dose>np.min(ri)) & vox))*voxtovol
        cV['body']=np.sum(vox)*voxtovol
        tdose=dose[vox]
        cmaxd['body']=np.max(tdose)
        cmind['body']=np.min(tdose)
        cmeand['body']=np.mean(tdose)
        ctotald['body']=np.sum(tdose)
        coverlap['body']=0
        coverlapborder['body']=1

        #Merge structures of one type and calculate metrics
        for key in vox_acc_dict.keys():
            if key=='PTV' and NumberPTVs<2:
                continue
            NamesN.append(f'{key} All Python')
            OrigNamesN.append(NamesN[-1])
            SearchtermsN.append('None')
            vox_acc=vox_acc_dict[key]
            if key =="PTV":
                MinDists=np.concatenate((MinDists.flatten(),np.min(MinDists,axis=0)))
                NamesPTVDists=np.concatenate((NamesPTVDists,np.repeat(NamesN[-1],len(Names)-NumberPTVs)))
                NamesOARDists=np.concatenate((NamesOARDists,OrigNames[NumberPTVs:]))
                rin=np.min(ri)
            else:
                rin=ridict.get(key,np.nan)
            keyorignames=np.array(OrigNames)[np.array(Names)==key]
            Centers[NamesN[-1]]=np.mean(np.array([Centers[n] for n in keyorignames]),axis=0)
            ctVri[NamesN[-1]]=np.sum((dose*vox_acc)>rin)*voxtovol
            cV[NamesN[-1]]=np.sum(vox_acc)*voxtovol
            tdose=dose[vox_acc]
            cmaxd[NamesN[-1]]=np.max(tdose)
            cmind[NamesN[-1]]=np.min(tdose)
            cmeand[NamesN[-1]]=np.mean(tdose)
            ctotald[NamesN[-1]]=np.sum(tdose)
            coverlap[NamesN[-1]]=np.sum((vox_acc & vox_acc_dict['PTV']))*voxtovol
            inner=binary_dilation(vox_acc_dict['PTV'],iterations=1)
            circ=binary_dilation(inner,iterations=1)^inner
            intsec=circ & vox_acc
            coverlapborder[NamesN[-1]]= np.sum(intsec)/np.sum(circ)
            vtemp=np.array([V[n] for n in keyorignames])
            V[NamesN[-1]]=np.sum(vtemp)
            tVri[NamesN[-1]]=np.sum(np.array([tVri[n] for n in keyorignames]))
            maxd[NamesN[-1]]=np.max(np.array([maxd[n] for n in keyorignames]))
            mind[NamesN[-1]]=np.min(np.array([mind[n] for n in keyorignames]))
            meand[NamesN[-1]]=np.sum(vtemp*np.array([meand[n] for n in keyorignames]))/V[NamesN[-1]]
            dvhstemp=[dvhdict[n] for n in keyorignames]
            dvhlengths=[len(dvh) for dvh in dvhstemp]
            maxdvh=np.argmax(dvhlengths)
            dvhtemp=np.copy(dvhstemp[maxdvh])
            tosum=np.arange(len(dvhlengths))
            tosum=tosum[tosum!=maxdvh]
            for i in tosum:
                dvhtemp[1:dvhlengths[i]:2]+=dvhstemp[i][1::2]
            dvhdict[NamesN[-1]]=dvhtemp   
            rivol=rivoldict.get(key,np.nan)
            if np.isnan(rivol):
                timvoldose[NamesN[-1]]=np.nan
            else:
                DVHdose=np.cumsum(dvhtemp[::2])*100
                DVHvols=np.array(dvhtemp[1::2])
                timvoldose[NamesN[-1]]=DVHdose[np.argmin(np.abs(DVHvols-rivol))]
    else:
        for name in OrigNamesN:
            ctVri[name]=np.nan
            cV[name]=np.nan
            cmaxd[name]=np.nan
            cmind[name]=np.nan
            cmeand[name]=np.nan
            ctotald[name]=np.nan
            coverlap[name]=np.nan
            coverlapborder[name]=np.nan
        ctVri['body']=np.nan
        cV['body']=np.nan
        cmaxd['body']=np.nan
        cmind['body']=np.nan
        cmeand['body']=np.nan
        ctotald['body']=np.nan
        coverlap['body']=np.nan
        coverlapborder['body']=np.nan
        cVri=np.nan
    
    return NamesN,Vri,tVri,V,mind,maxd,meand,timvoldose,coverlapborder,cVri,ctVri,cV,cmind,cmaxd,cmeand,ctotald,coverlap,Centers,MinDists.flatten(),NamesPTVDists,NamesOARDists,SearchtermsN,OrigNamesN,dvhdict

def batchAnalyzeDose(save,timmermansheet=TimmermanSheet,verifiersheet=PTVInfoSheet,locsheet=DICOMLocations,interval=None,debug=False,calc=True):
    """
    Wrapper to read dose files in batch mode.

    In:
    - save; str, save location
    - timmermansheet; str to Timmerman.csv is a CSV sheet containing simplified constraints from the Timmerman tables
    - verifiersheet; str to PTVInfoSheet, xlsx file with plan infos
    - locsheet; str, xlsx file with DICOM plan locations
    - interval; int array [start, end], only read specified row from loc sheet
    - debug; bool, show some extra info
    - calc; bool, recalculate DVH statistics from dose image instead of only reading them from stored DICOM DVHs

    Out:
    - Dose.pickle ; Pandas dataframe, contains dose statistics from DICOM dose file. Columns: 'PlanSetupSer','Name', 'Search Term', 'Original Name', 'Timmerman Max Point Dose [cGy]', 'Timmerman Volume Max Dose [cGy]', 'Timmerman Volume Max [cc]', 'Number Fractions','Timmerman Matched Number Fractions', 'Center [mm]', 'Dose at Timmerman Volume Max [cGy]','Volume Reference Isodose [cc]', 'Target Volume Reference Isodose [cc]', 'Target Volume [cc]', 'Min Dose [cGy]', 'Max Dose [cGy]', 'Mean Dose [cGy]', 'Calc Volume Reference Isodose [cc]', 'Calc Target Volume Reference Isodose [cc]', 'Calc Target Volume [cc]', 'Calc Min Dose [cGy]', 'Calc Max Dose [cGy]', 'Calc Mean Dose [cGy]', 'Calc Total Dose [cGy]', 'Calc Overlap PTV [cc]', 'Calc Overlap with Dilated Shell of PTV All', 'DICOM DVH'
    - DistsPTVOAR.pickle ; Pandas dataframe, contains distances between PTV and OARs. Columns: 'PlanSetupSer', 'PTV Name', 'OAR Name', 'Min Distance Between Contours [mm]'
    """
    
    results = {'PlanSetupSer' : [],'Name' : [], 'Search Term':[], 'Original Name' : [], 'Timmerman Max Point Dose [cGy]':[], 'Timmerman Volume Max Dose [cGy]':[], 'Timmerman Volume Max [cc]':[], 'Number Fractions' :[],'Timmerman Matched Number Fractions':[], 'Center [mm]': [], 'Dose at Timmerman Volume Max [cGy]':[],'Volume Reference Isodose [cc]' : [], 'Target Volume Reference Isodose [cc]' : [], 'Target Volume [cc]' : [], 'Min Dose [cGy]' : [], 'Max Dose [cGy]' : [], 'Mean Dose [cGy]' : [], 'Calc Volume Reference Isodose [cc]' : [], 'Calc Target Volume Reference Isodose [cc]' : [], 'Calc Target Volume [cc]' : [], 'Calc Min Dose [cGy]' : [], 'Calc Max Dose [cGy]' : [], 'Calc Mean Dose [cGy]' : [], 'Calc Total Dose [cGy]' : [], 'Calc Overlap PTV [cc]' : [], 'Calc Overlap with Dilated Shell of PTV All' : [], 'DICOM DVH' : []}
    distresults = {'PlanSetupSer' : [], 'PTV Name' : [], 'OAR Name' : [], 'Min Distance Between Contours [mm]' : []}
    if interval:
        locs=updateLocs(locsheet,debug)[interval[0]:interval[1]]
    else:    
        locs=updateLocs(locsheet,debug)
    try:
        structs=pd.read_pickle(f"{save}/PTV{interval if interval else ''}.pickle")
    except:
        structs=pd.read_pickle(f"{save}/PTV.pickle")
    structs=structs[['PTV' in r for r in structs['TV Type']]]
    ver=pd.read_excel(verifiersheet).rename(columns={'PTV name' : 'Name'})[["PlanSetupSer","Name", "Prescribed Dose [cGy]", "Num Fractions"]]
    structs=pd.merge(ver,structs,on=['PlanSetupSer','Name'])
    logging.basicConfig(filename=f'{save}/LogsDose.log', encoding='utf-8', level=logging.WARNING)
    TimFracs=np.array([1,2,3,4,5,8,10,15,20,30])
    Timmerman=pd.read_csv(timmermansheet)
    for i,RDfileloc in enumerate(locs["Dose_Path"]):
        try:
            PSS=locs["PlanSetupSer"].iloc[i]
            try:
                with gzip.open(RDfileloc, 'rb') as f:
                    RDfile = pydicom.dcmread(f)
            except:
                continue
            if RDfile.ReferencedRTPlanSequence[0].ReferencedSOPInstanceUID == locs["RTPlan_UID"].iloc[i] and RDfile.ReferencedStructureSetSequence[0].ReferencedSOPInstanceUID == locs["StructureSet_UID"].iloc[i]:
                Strucfile = pydicom.dcmread(locs['StructureSet_Path'].iloc[i])
                targets=structs[structs['PlanSetupSer']==PSS]
                if len(targets)==0:
                    continue
                targetfractions=int(targets['Num Fractions'].iloc[0])
                matchedtimfractions=TimFracs[np.argmin(np.abs(TimFracs-targetfractions))]
                TimmermanFrac=Timmerman[Timmerman['Number Fractions']==matchedtimfractions]
                NamesN,Vri,tVri,V,mind,maxd,meand,timvoldose,coverlapborder,cVri,ctVri,cV,cmind,cmaxd,cmeand,ctotald,coverlap,Centers,MinDists,NamesPTVDists,NamesOARDists,SearchtermsN,OrigNamesN,dvhdict=analyzeDose(RDfile,targets['XYZ'].to_list(),targets['Prescribed Dose [cGy]'].to_list(),Strucfile,targets['Name'].tolist(),TimmermanFrac,debug,calc) 
                print(i)
                for ji,j in enumerate(OrigNamesN):
                    mask=[tf in NamesN[ji] for tf in TimmermanFrac['OAR']]
                    if np.any(mask):
                        tmpd=TimmermanFrac['Max point dose [Gy]'][mask].iloc[0]*100
                        tvmd=TimmermanFrac['Volume max [Gy]'][mask].iloc[0]*100
                        tvm=TimmermanFrac['Volume [cc]'][mask].iloc[0]
                    else:
                        tmpd=np.nan
                        tvmd=np.nan
                        tvm=np.nan
                    results['PlanSetupSer'].append(PSS)
                    results['Name'].append(NamesN[ji])
                    results['Search Term'].append(SearchtermsN[ji])
                    results['Original Name'].append(j)
                    results['Timmerman Max Point Dose [cGy]'].append(tmpd)
                    results['Timmerman Volume Max Dose [cGy]'].append(tvmd)
                    results['Timmerman Volume Max [cc]'].append(tvm)
                    results['Number Fractions'].append(targetfractions)
                    results['Timmerman Matched Number Fractions'].append(matchedtimfractions)
                    results['Center [mm]'].append(Centers[OrigNamesN[ji]])
                    results['Dose at Timmerman Volume Max [cGy]'].append(timvoldose[j])
                    results['Volume Reference Isodose [cc]'].append(Vri)
                    results['Target Volume Reference Isodose [cc]'].append(tVri[j])
                    results['Target Volume [cc]'].append(V[j])
                    results['Min Dose [cGy]'].append(mind[j])
                    results['Max Dose [cGy]'].append(maxd[j])
                    results['Mean Dose [cGy]'].append(meand[j])
                    results['Calc Volume Reference Isodose [cc]'].append(cVri)
                    results['Calc Target Volume Reference Isodose [cc]'].append(ctVri[j])
                    results['Calc Target Volume [cc]'].append(cV[j])
                    results['Calc Min Dose [cGy]'].append(cmind[j])
                    results['Calc Max Dose [cGy]'].append(cmaxd[j])
                    results['Calc Mean Dose [cGy]'].append(cmeand[j])
                    results['Calc Total Dose [cGy]'].append(ctotald[j])
                    results['Calc Overlap PTV [cc]'].append(coverlap[j])
                    results['Calc Overlap with Dilated Shell of PTV All'].append(coverlapborder[j])
                    results['DICOM DVH'].append(dvhdict[j])
                distresults["PlanSetupSer"].extend(np.repeat(PSS,len(MinDists)))
                distresults['PTV Name'].extend(NamesPTVDists)
                distresults['OAR Name'].extend(NamesOARDists)
                distresults['Min Distance Between Contours [mm]'].extend(MinDists)
                if debug:
                    for key in distresults.keys():
                        print(key,len(distresults[key]))
                    for key in results.keys():
                        print(key,len(results[key]))
        except KeyboardInterrupt:
            print ('KeyboardInterrupt exception is caught')
            raise
        except:
            print(f"Failed {RDfileloc}")
            logging.exception(f"Failed {RDfileloc}")
    df = pd.DataFrame.from_dict(results)
    df=calcDoseMetrics(df)
    dfd = pd.DataFrame.from_dict(distresults)
    if interval:
        df.to_pickle(f"{save}/Dose{interval}.pickle")
        dfd.to_pickle(f"{save}/DistsPTVOAR{interval}.pickle")
    else:    
        try:
            shutil.copyfile(f"{save}/Dose.pickle",f"{save}/Dosepickle.old")
            Dose=pd.read_pickle(f"{save}/Dose.pickle")
            df=pd.concat([Dose,df])
            df.to_pickle(f"{save}/Dose.pickle")
            shutil.copyfile(f"{save}/DistsPTVOAR.pickle",f"{save}/DistsPTVOARpickle.old")
            Dists=pd.read_pickle(f"{save}/DistsPTVOAR.pickle")
            dfd=pd.concat([Dists,dfd])
            dfd.to_pickle(f"{save}/DistsPTVOAR.pickle")
        except:
            if not os.path.exists(f"{save}/Dose.pickle"):
                df.to_pickle(f"{save}/Dose.pickle")
                dfd.to_pickle(f"{save}/DistsPTVOAR.pickle")
            else:
                logging.exception(f"Dose.pickle exists, but appending failed. Create Dosenew.")
                df.to_pickle(f"{save}/Dosenew.pickle")
                dfd.to_pickle(f"{save}/DistsPTVOARnew.pickle")

    return df,dfd

#Calc Dose Metrics
def calcDoseMetrics(dose):
    verifier= pd.read_excel(PTVInfoSheet).rename(columns={'PTV name' : 'Name', 'PTV volume (cc)' :'Volume Planning System [cm]'})
    temp=dose.loc[dose['Name']=='body'][['PlanSetupSer','Calc Target Volume Reference Isodose [cc]','Calc Total Dose [cGy]']]
    temp.rename(columns={'Calc Target Volume Reference Isodose [cc]':'Calc body Volume Reference Isodose [cc]', 'Calc Total Dose [cGy]' : 'Calc body Total Dose [cGy]'},inplace=True)
    # temp=dose.loc[dose['Name']=='body'][['PlanSetupSer','Calc Target Volume Reference Isodose [cc]']]#,'Calc Total Dose [cGy]']]
    # temp.rename(columns={'Calc Target Volume Reference Isodose [cc]':'Calc body Volume Reference Isodose [cc]'},inplace=True)#, 'Calc Total Dose [cGy]' : 'Calc body Total Dose [cGy]'
    dose=pd.merge(dose,temp,on=['PlanSetupSer'],how='left')
    dose=pd.merge(dose,verifier[['PlanSetupSer','Prescribed Dose [cGy]','Name']],on=['PlanSetupSer','Name'],how='left')
    dose['Conformity Index 1']=dose['Volume Reference Isodose [cc]']/dose['Target Volume [cc]']
    dose['Calc Conformity Index 1']=(dose['Calc body Volume Reference Isodose [cc]']+dose['Calc Target Volume Reference Isodose [cc]'])/dose['Calc Target Volume [cc]']
    dose['Conformity Index 2']=dose['Target Volume Reference Isodose [cc]']/dose['Volume Reference Isodose [cc]']
    dose['Calc Conformity Index 2']=dose['Calc Target Volume Reference Isodose [cc]']/(dose['Calc body Volume Reference Isodose [cc]']+dose['Calc Target Volume Reference Isodose [cc]'])
    dose['Conformity Index 3']=dose['Target Volume Reference Isodose [cc]']**2/dose['Target Volume [cc]']/dose['Volume Reference Isodose [cc]']
    dose['Calc Conformity Index 3']=dose['Calc Target Volume Reference Isodose [cc]']**2/dose['Calc Target Volume [cc]']/(dose['Calc body Volume Reference Isodose [cc]']+dose['Calc Target Volume Reference Isodose [cc]'])
    dose['Calc Conformity Index 4']=dose['Calc Total Dose [cGy]']/(dose['Calc Total Dose [cGy]']+dose['Calc body Total Dose [cGy]'])
    dose['Homogeneity Index 1']=dose['Max Dose [cGy]']/dose['Prescribed Dose [cGy]']
    dose['Homogeneity Index 2']=(dose['Target Volume [cc]']-dose['Target Volume Reference Isodose [cc]'])/dose['Target Volume [cc]']
    dose['Calc Homogeneity Index 2']=(dose['Calc Target Volume [cc]']-dose['Calc Target Volume Reference Isodose [cc]'])/dose['Calc Target Volume [cc]']
    dose['Max Dose/Timmerman Max Point Dose']=dose['Max Dose [cGy]']/dose['Timmerman Max Point Dose [cGy]']
    dose['Calc Max Dose/Timmerman Max Point Dose']=dose['Calc Max Dose [cGy]']/dose['Timmerman Max Point Dose [cGy]']
    dose['Target Volume Reference Isodose/Timmerman Volume Max']=dose['Target Volume Reference Isodose [cc]']/dose['Timmerman Volume Max [cc]']
    dose['Calc Target Volume Reference Isodose/Timmerman Volume Max']=dose['Calc Target Volume Reference Isodose [cc]']/dose['Timmerman Volume Max [cc]']
    dose['Dose at Timmerman Volume Max/Timmerman Volume Max Dose']=dose['Dose at Timmerman Volume Max [cGy]']/dose['Timmerman Volume Max Dose [cGy]']
    return dose

In [ ]:
#batch run, have to change batchAnalyzeDose by commenting out loading of this and passing them as parameters instead
# structs=pd.read_pickle(f"{SaveFolder}/PTV.pickle") #{interval if interval else ''}
# structs=structs[['PTV' in r for r in structs['TV Type']]]
# ver=pd.read_excel(PTVInfoSheet).rename(columns={'PTV name' : 'Name'})[["PlanSetupSer","Name", "Prescribed Dose [cGy]", "Num Fractions"]]
# structs=pd.merge(ver,structs,on=['PlanSetupSer','Name'])
# bigint=[0,2000]
# locs=updateLocs(locsheet=DICOMLocations,debug=False)[bigint[0]:bigint[1]]
# done=np.loadtxt('//cifshd/homedir$/Evaluation/EllaOAR/DonePSS.txt')
# with open(f"{SaveFolder}/LogsDose.log", 'r') as file:
#     logs = file.read()
# maskstr=[type(s)==str for s in np.array(locs['Dose_Path'])]
# maskdone=[s in done for s in np.array(locs['PlanSetupSer'])]
# maskerr=[s in logs for s in np.array(locs['Dose_Path'])[maskstr]]
# locs['Dose_Path'].iloc[np.arange(len(locs))[maskstr][maskerr]]=np.nan
# locs['Dose_Path'][maskdone]=np.nan
# print(np.sum([type(s)==str for s in np.array(locs['Dose_Path'])]), len(locs), np.sum(maskstr), np.sum(maskerr), np.sum(maskdone))
# structs=structs[[s in np.array(locs['PlanSetupSer']) for s in np.array(structs['PlanSetupSer'])]]
# interval=np.array([bigint[0],bigint[0]+50],dtype=int)
# while interval[1]<=bigint[1]:
#     dfDose3,dfd=batchAnalyzeDose(SaveFolder,structs,locs[interval[0]-bigint[0]:interval[1]-bigint[0]],locsheet=DICOMLocations,debug=False,calc=True,interval=[interval[0],interval[1]])
#     interval+=50

### Merge and Anonymize

In [31]:
def mergeAllResults(PTV_sheet=PTVInfoSheet,CGTV_sheet=CGTVInfoSheet,save=SaveFolder):
    """
    Merges all different pickle dataframes generated by the different functions into one big dataframe.

    In:
    - PTV_sheet; str to PTVInfoSheet Excel file
    - CGTV_sheet; str to CGTVInfoSheet Excel File
    - save; str to folder where all the pickles are stored
    
    Out:
    - MergedResults.pickle; Pandas dataframe, contains all the final results of the analysis
    """
    dfPCA_sheet = pd.read_pickle(f"{save}/PCAMesh.pickle")
    if PTV_sheet:
        dfPTV_sheet= pd.read_excel(PTV_sheet).rename(columns={'PTV name' : 'Name', 'PTV volume (cc)' :'Volume Planning System [cm]'})
        dfPTV_sheet2=dfPTV_sheet[['PlanSetupSer','Rx site','Rx technique','Prescription']].groupby('PlanSetupSer').first()
        dfmerged = pd.merge(dfPCA_sheet, dfPTV_sheet2, on=['PlanSetupSer'])
        dfPTV_sheet3=dfPTV_sheet[['PlanSetupSer', 'Name', 'Mean dose (cGy)', 'Max dose (cGy)', 'Min dose (cGy)','Volume Planning System [cm]', 'Prescribed Dose [cGy]']]
        dfCGTV_sheet= pd.read_excel(CGTV_sheet).rename(columns={'Target name' : 'Name', 'Target volume (cc)' : 'Volume Planning System [cm]'})
        dfCGTV_sheet3=dfCGTV_sheet[['PlanSetupSer', 'Name', 'Mean dose (cGy)', 'Max dose (cGy)', 'Min dose (cGy)','Volume Planning System [cm]']]
        dfdose=pd.concat([dfPTV_sheet3,dfCGTV_sheet3])
        dfmerged= pd.merge(dfmerged, dfdose, on=['PlanSetupSer', 'Name'],how='left')
    else:
        dfmerged=dfPCA_sheet
    
    # sites,counts = np.unique(dfmerged['Rx site'],return_counts=True)
    # for site in sites[counts==1]:
    #     dfmerged=dfmerged.drop(dfmerged.loc[dfmerged['Rx site']==site])
    dfmergedAll = dfmerged[['All' in tv for tv in dfmerged['TV Type']]]
    dftreat_sheet = pd.read_pickle(f"{save}/Treatable.pickle")
    dftreat_sheet.drop(['Treatable q0_attLung', 'Treatable q0','Treatable q0_att'],axis=1,inplace=True)
    dfmerged4 = pd.merge(dftreat_sheet,dfmerged, on=['PlanSetupSer','Name'])
    dfmergedSolo = dfmerged4.copy()
    dfmerged4['TV Type']=[f"{tv} All" for tv in dfmerged4['TV Type']]
    grouped=dfmerged4.groupby(['PlanSetupSer','TV Type'])
    dang1=grouped['Mean Depth q0_att per Angle'].mean()
    dang1.name='Mean Depth q0_att per Angle'
    dfmergedAll = pd.merge(dang1,dfmergedAll, on=['PlanSetupSer','TV Type'])
    dang3=grouped['Mean Depth q0 per Angle'].mean()
    dang3.name='Mean Depth q0 per Angle'
    dfmergedAll = pd.merge(dang3,dfmergedAll, on=['PlanSetupSer','TV Type'])
    dang2=grouped['Mean Depth q0_attLung per Angle'].mean()
    dang2.name='Mean Depth q0_attLung per Angle'
    dfmergedAll = pd.merge(dang2,dfmergedAll, on=['PlanSetupSer','TV Type'])
    d1=grouped['Max Depth q0_att'].max()
    d1.name='Max Depth q0_att'
    dfmergedAll = pd.merge(d1,dfmergedAll, on=['PlanSetupSer','TV Type'])
    d3=grouped['Max Depth q0'].max()
    d3.name='Max Depth q0'
    dfmergedAll = pd.merge(d3,dfmergedAll, on=['PlanSetupSer','TV Type'])
    d2=grouped['Max Depth q0_attLung'].max()
    d2.name='Max Depth q0_attLung'
    dfmergedAll = pd.merge(d2,dfmergedAll, on=['PlanSetupSer','TV Type'])
    d1=grouped['Min Depth q0_att'].min()
    d1.name='Min Depth q0_att'
    dfmergedAll = pd.merge(d1,dfmergedAll, on=['PlanSetupSer','TV Type'])
    d3=grouped['Min Depth q0'].min()
    d3.name='Min Depth q0'
    dfmergedAll = pd.merge(d3,dfmergedAll, on=['PlanSetupSer','TV Type'])
    d2=grouped['Min Depth q0_attLung'].min()
    d2.name='Min Depth q0_attLung'
    dfmergedAll = pd.merge(d2,dfmergedAll, on=['PlanSetupSer','TV Type'])
    # treatmeanlung=grouped['Treatable q0_attLung'].mean()
    # treatmeanlung.name='Treatable q0_attLung'
    # dfmergedAll = pd.merge(treatmeanlung,dfmergedAll, on=['PlanSetupSer','TV Type'])
    # treatmean0=grouped['Treatable q0'].mean()
    # treatmean0.name='Treatable q0'
    # dfmergedAll = pd.merge(treatmean0,dfmergedAll, on=['PlanSetupSer','TV Type'])
    # treatmean=grouped['Treatable q0_att'].mean()
    # treatmean.name='Treatable q0_att'
    # dfmergedAll = pd.merge(treatmean,dfmergedAll, on=['PlanSetupSer','TV Type'])
    dfmerged9 = pd.concat([dfmergedAll,dfmergedSolo])
    print(len(dfPCA_sheet),len(dfmerged),len(dftreat_sheet),len(dfmerged9))

    dfMU_sheet = pd.read_pickle(f"{save}/MU.pickle")
    # dfMU_sheet = dfMU_sheet.drop('Beam Limiting Device Position Sequence',axis=1)
    dfMU_sheet=pd.merge(dfmerged9[[(('All' in tv or 'Solo' in tv) and 'PTV' in tv) for tv in dfmerged9['TV Type']]][['PlanSetupSer','Name']],dfMU_sheet,on=['PlanSetupSer'],how='left')
    if PTV_sheet:
        dfPTV_sheet4=dfPTV_sheet[['PlanSetupSer','Num Fractions', 'Dose/Frc (cGy)']].groupby('PlanSetupSer').first()
        dfMU_sheet = pd.merge(dfMU_sheet,dfPTV_sheet4,on=['PlanSetupSer'])
    dfmerged9 = pd.merge(dfmerged9,dfMU_sheet,on=['PlanSetupSer','Name'],how='left')
    print(len(dfmerged9))

    
    dose = pd.read_pickle(f"{save}/Dose.pickle")
    dose.rename(columns={'Name':'New Name', 'Original Name' : 'Name'},inplace=True)
    dose.drop('Prescribed Dose [cGy]',axis=1,inplace=True)
    dfmerged9=pd.merge(dfmerged9,dose,on=['PlanSetupSer','Name'],how='outer')
   
    
    
    dfHU = pd.read_pickle(f"{save}/HU.pickle") #Fix: Lung TV Type column clashes with other TV Type column. Lung HU different entry from Lung Timmerman Dose
    dfHU.drop('TV Type',axis=1,inplace=True)
    dfmerged9=pd.merge(dfHU,dfmerged9,on=['PlanSetupSer','Name'],how='outer')
    print(dfmerged9.columns)
    a=np.array(['Mets' if 'met' in str(p).casefold() else '' for p in dfmerged9['Prescription']],dtype=str)
    b=np.array(['WBRT' if 'wbrt' in str(p).casefold() else '' for p in dfmerged9['Prescription']],dtype=str)
    c=np.array(['Eye' if 'eye' in str(p).casefold() else '' for p in dfmerged9['Prescription']],dtype=str)
    d=np.array(['Prostate' if 'prostate' in str(p).casefold() and dfmerged9['Volume'].iloc[i]<400 and 'PTV Solo' in dfmerged9['TV Type'].iloc[i] else '' for i,p in enumerate(dfmerged9['Prescription'])],dtype=str)
    dfmerged9['Specific Site Info']=[a[i]+b[i]+c[i]+d[i] for i in range(len(a))]
    
    # dfmerged9.to_csv(f"{save}/Merged.csv", index=False)
    dfmerged9.to_pickle(f"{save}/MergedResults.pickle")
    return dfmerged9#, dfanon

def anonymizeResults(dfmerged9,save=SaveFolder,savename='DataAnonymized',loaddict=None):
    """
    Remove and rename columns to fit wanted data. Code plan ID and structure name using a dictionary to make it anonymous.
    If you already have a dictionary of anonymized indices, use loaddict to load it and add to it.

    In:
    - dfmerged9; Pandas dataframe generated by mergeAllResults, stored as MergedResults.pickle
    - save; str to folder where all the pickles are stored
    - savename; str, Name of the anonymized pickle file
    - loaddict; str, location of anonymization dictionary. If none, a new dictionary will be created.

    Out:
    - savename.pickle; Pandas dataframe, anonymized results
    - loaddict; Json dictionary with the code to the anonymized indices
    """
    dfanon=dfmerged9.copy()
    if loaddict:
        with open(loaddict, 'r') as fp:
            scramblePSSf = json.load(fp)
        scramblePSS = {v: k for k, v in scramblePSSf.items()}
        scramblePSSonly = {v.split(',')[0]: k.split(',')[0] for k, v in scramblePSSf.items()}
        newpsss=[]
        newname=[]
        if "OAR Name" in dfanon.columns:
            newOARname=[]
            dfanon.rename(columns={'PTV Name' : 'Name'},inplace=True)
        for i,psss in enumerate(dfanon['PlanSetupSer']):
            name=dfanon['Name'].iloc[i]
            temp=scramblePSS.get(f"{int(psss)},{name}",False)
            if temp:
                p,s=temp.split(',')
                newpsss.append(p)
                newname.append(s)
            else:
                temp=scramblePSSonly.get(str(int(psss)),False)
                if temp:
                    newpsss.append(temp)
                else:
                    npss=np.random.randint(1e9)
                    while str(int(npss)) in scramblePSSonly.keys():
                        npss=np.random.randint(1e9)
                    newpsss.append(npss)
                nn=np.random.randint(1e5)
                # print(f"{newpsss[-1]},{int(nn)}")
                while f"{newpsss[-1]},{int(nn)}" in scramblePSS.values():
                    print( f"{newpsss[-1]},{int(nn)}")
                    nn=np.random.randint(1e5)
                newname.append(nn)
                scramblePSSonly[str(int(psss))]=str(newpsss[-1])
                scramblePSS[f"{int(psss)},{name}"]=f"{newpsss[-1]},{newname[-1]}"
            if "OAR Name" in dfanon.columns:
                name=dfanon['OAR Name'].iloc[i]
                temp=scramblePSS.get(f"{int(psss)},{name}",False)
                if temp:
                    p,s=temp.split(',')
                    newOARname.append(s)
                else:
                    nn=np.random.randint(1e3)
                    while f"{int(nn)},{name}" in scramblePSS.keys():
                        nn=np.random.randint(1e3)
                    newOARname.append(nn)
                    scramblePSS[f"{int(psss)},{name}"]=f"{newpsss[-1]},{newname[-1]}"
        dfanon['PlanSetupSer']=newpsss
        dfanon['Name']=newname
        if "OAR Name" in dfanon.columns:
            dfanon['OAR Name']=newOARname
            dfanon.rename(columns={'Name' : 'PTV Name'},inplace=True)
        dfanon=dfanon[dfanon['PlanSetupSer']!=-1]
    else:
        if save:
            loaddict=f"{save}/AnonymizationDictPlan,Struc.json"
        PSSunique=dfmerged9['PlanSetupSer'].unique()
        newPSS=np.arange(len(PSSunique))
        np.random.shuffle(newPSS)
        scramblePSS=dict(zip(PSSunique,newPSS))
        newName=np.arange(len(dfanon['Name']))
        np.random.shuffle(newName)
        newPSSS=np.array([scramblePSS[int(psss)] for psss in dfanon['PlanSetupSer']])
        scramblePSS=dict(zip((newPSSS.astype(str)+np.repeat(',',len(newPSSS))+newName.astype(str)),np.sum(np.array([dfanon['PlanSetupSer'].astype(int).astype(str), np.repeat(',',len(dfanon)),dfanon['Name']]).T,axis=1)))  
        dfanon['PlanSetupSer']=newPSSS
        dfanon['Name']=newName 
    if 'Prescription' in dfanon.columns:
        dfanon.drop(['Original Name','PMRN','Rx Name','StructureUID','Prescription', "In Lung", 'In Left Lung', 'In Right Lung','Mean Depth q0_attLung per Angle', 'Mean Depth q0 per Angle', 'Mean Depth q0_att per Angle', 'CTOriginSpacingSize', 'PTVOriginSpacingSize'],axis=1,inplace=True,errors='ignore')
        dfanon.rename(columns={'PC_min':'PC_Min [mm]','PC_max':'PC_Max [mm]','PC_Center':'PC_Center [mm]','min':'Min [mm]','max':'Max [mm]','Center':'Center [mm]','Size':'Size [mm]', 'Volume' : 'Volume [cc]', 'Min Depth q0': 'Min Depth q0 [cm]', 'Min Depth q0_attLung': 'Min Depth q0_attLung [cm]', 'Min Depth q0_att': 'Min Depth q0_att [cm]', 'Max Depth q0': 'Max Depth q0 [cm]', 'Max Depth q0_attLung': 'Max Depth q0_attLung [cm]', 'Max Depth q0_att': 'Max Depth q0_att [cm]'}, inplace=True)
    dfanon.rename(columns={'PlanSetupSer': 'Plan Index (Anonymized)'}, inplace=True)
    dfanon['Plan Index (Anonymized)'] =pd.to_numeric(dfanon['Plan Index (Anonymized)'])
    if "OAR Name" not in dfanon.columns:   
        dfanon.rename(columns={'Name': 'Structure Index (Anonymized)'}, inplace=True)
        dfanon.set_index('Structure Index (Anonymized)',drop=False,inplace=True)
        temp=dfanon.pop('Structure Index (Anonymized)')
        dfanon.insert(1,'Structure Index (Anonymized)',temp)
        dfanon['Structure Index (Anonymized)'] =pd.to_numeric(dfanon['Structure Index (Anonymized)'])
    else:
        dfanon.rename(columns={'PTV Name': 'PTV Structure Index (Anonymized)','OAR Name': 'OAR Structure Index (Anonymized)'}, inplace=True)
        dfanon['PTV Structure Index (Anonymized)'] =pd.to_numeric(dfanon['PTV Structure Index (Anonymized)'])
        dfanon['OAR Structure Index (Anonymized)'] =pd.to_numeric(dfanon['OAR Structure Index (Anonymized)'])

        
    if save:
        if not os.path.exists(save):
            os.makedirs(save)
        dfanon.to_csv(f"{save}/{savename}.csv", index=False)
        dfanon.to_pickle(f"{save}/{savename}.pickle")
        with open(loaddict, 'w') as fp:
            json.dump({v: k for k, v in scramblePSS.items()}, fp, sort_keys=True, indent=4)
    return dfanon


def verInfo(identifiers,dictLocation,PTV_sheet=PTVInfoSheet):
    """Get original indices for anonymized plan identifiers"""
    verifier= pd.read_excel(PTV_sheet)
    with open(dictLocation, 'r') as fp:
        scramblePSSf = json.load(fp)
    scramblePSSonly = {v.split(',')[0]: k.split(',')[0] for v, k in scramblePSSf.items()}
    idPSS=np.array([scramblePSSonly.get(str(id)) for id in identifiers],dtype=int)
    sortorder=np.argsort(np.argsort(idPSS))
    found=verifier[[vID in idPSS for vID in verifier['PlanSetupSer']]]
    first=np.unique(found['PlanSetupSer'], return_index=True)[1]
    found=found.iloc[first]
    found=found.iloc[sortorder]
    found.to_csv(f"{PTVInfoSheet}Searched.csv")

### Analysis/Plotting

In [59]:
colors = ['rgb(204,81,81)',
 'rgb(127,51,51)',
 'rgb(81,204,204)',
 'rgb(51,127,127)',
 'rgb(142,204,81)',
 'rgb(89,127,51)',
 'rgb(142,81,204)',
 'rgb(89,51,127)',
 'rgb(204,173,81)',
 'rgb(127,108,51)',
 'rgb(81,204,112)',
 'rgb(51,127,70)',
 'rgb(81,112,204)',
 'rgb(51,70,127)',
 'rgb(204,81,173)',
 'rgb(127,51,108)',
 'rgb(204,127,81)',
 'rgb(127,79,51)',
 'rgb(188,204,81)',
 'rgb(117,127,51)',
 'rgb(96,204,81)',
 'rgb(60,127,51)',
 'rgb(81,204,158)',
 'rgb(51,127,98)',
 'rgb(81,158,204)',
 'rgb(51,98,127)',
 'rgb(96,81,204)',
 'rgb(60,51,127)',
 'rgb(188,81,204)',
 'rgb(117,51,127)',
 'rgb(204,81,127)',
 'rgb(127,51,79)',
 'rgb(204,104,81)',
 'rgb(127,65,51)',
 'rgb(204,150,81)',
 'rgb(127,94,51)',
 'rgb(204,196,81)',
 'rgb(127,122,51)',
 'rgb(165,204,81)',
 'rgb(103,127,51)',
 'rgb(119,204,81)',
 'rgb(74,127,51)',
 'rgb(81,204,89)',
 'rgb(51,127,55)',
 'rgb(81,204,135)',
 'rgb(51,127,84)',
 'rgb(81,204,181)',
 'rgb(51,127,113)',
 'rgb(81,181,204)',
 'rgb(51,113,127)',
 'rgb(81,135,204)',
 'rgb(51,84,127)',
 'rgb(81,89,204)',
 'rgb(51,55,127)',
 'rgb(119,81,204)',
 'rgb(74,51,127)',
 'rgb(165,81,204)',
 'rgb(103,51,127)',
 'rgb(204,81,196)',
 'rgb(127,51,122)',
 'rgb(204,81,150)',
 'rgb(127,51,94)',
 'rgb(204,81,104)',
 'rgb(127,51,65)',
 'rgb(204,93,81)',
 'rgb(127,58,51)',
 'rgb(204,116,81)',
 'rgb(127,72,51)',
 'rgb(204,138,81)',
 'rgb(127,86,51)',
 'rgb(204,161,81)',
 'rgb(127,101,51)',
 'rgb(204,184,81)',
 'rgb(127,115,51)',
 'rgb(200,204,81)',
 'rgb(125,127,51)',
 'rgb(177,204,81)',
 'rgb(110,127,51)',
 'rgb(154,204,81)',
 'rgb(96,127,51)',
 'rgb(131,204,81)',
 'rgb(82,127,51)',
 'rgb(108,204,81)',
 'rgb(67,127,51)',
 'rgb(85,204,81)',
 'rgb(53,127,51)',
 'rgb(81,204,100)',
 'rgb(51,127,62)',
 'rgb(81,204,123)',
 'rgb(51,127,77)',
 'rgb(81,204,146)',
 'rgb(51,127,91)',
 'rgb(81,204,169)',
 'rgb(51,127,105)',
 'rgb(81,204,192)',
 'rgb(51,127,120)',
 'rgb(81,192,204)',
 'rgb(51,120,127)',
 'rgb(81,169,204)',
 'rgb(51,105,127)']

def cartesian_to_spherical(x,y,z):
    theta = np.arctan2(np.sqrt(x ** 2 + y ** 2), z)
    phi = np.arctan2(y, x) #if x >= 0 else np.arctan2(y, x) + np.pi
    return theta, phi

def plotPC(dfpca,save=None,show=True,pP=True,TVType='PTV',site=''):
    PC=np.concatenate(dfpca['PC'].to_numpy()).reshape(-1,3,3)
    if len(PC)<1000:
        size=4
    else:
        size=2
    if save:
        folder = f"{save}"
        if not os.path.exists(folder):
            os.makedirs(folder)
    f11=go.Scatter3d(x=PC[:,0,0],y=PC[:,0,1],z=PC[:,0,2],mode='markers',name='Primary',marker=dict(size=size,color='blue'))
    f12=go.Scatter3d(x=PC[:,1,0],y=PC[:,1,1],z=PC[:,1,2],mode='markers',name='Seconary',marker=dict(size=size,color='red'))
    f13=go.Scatter3d(x=PC[:,2,0],y=PC[:,2,1],z=PC[:,2,2],mode='markers',name='Tertiary',marker=dict(size=size,color='green'))
    traces1=[f11,f12,f13]
    for i,t in enumerate(traces1):
        fig1=go.Figure(t)
        title1i=f"Directions Principal Component {i+1} in 3D <br>for {TVType}{' Plan' if pP else ''}s {site}"
        fig1.update_layout(      title=title1i,
                                    margin=dict(r=0, b=0, l=0, t=60),
                                    height=600,
                                    width=600,
                                    scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
                                    )
        if show: fig1.show()
        if save: 
            fig1.write_image(f"{folder}/{title1i.replace('<br>','')}.pdf")
            fig1.write_image(f"{folder}/{title1i.replace('<br>','')}.png")
    fig1=go.Figure(traces1)
    title1=f"Directions of Principal Components in 3D <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig1.update_layout(      title=title1,
                                margin=dict(r=0, b=0, l=0, t=60),
                                height=600,
                                width=600,
                                scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
                                )
    if show: fig1.show()
    t1,p1 = cartesian_to_spherical(PC[:,0,0],y=PC[:,0,1],z=PC[:,0,2])
    t2,p2 = cartesian_to_spherical(PC[:,1,0],y=PC[:,1,1],z=PC[:,1,2])
    t3,p3 = cartesian_to_spherical(PC[:,2,0],y=PC[:,2,1],z=PC[:,2,2])
    f1=go.Scatter(y=t1,x=p1,mode='markers',name='Primary',marker=dict(size=size,color='blue'))
    f2=go.Scatter(y=t2,x=p2,mode='markers',name='Seconary',marker=dict(size=size,color='red'))
    f3=go.Scatter(y=t3,x=p3,mode='markers',name='Tertiary',marker=dict(size=size,color='green'))
    traces=[f1,f2,f3]
    for i,t in enumerate(traces):
        fig=go.Figure(t)
        titlei=f"Directions of Principal Component {i+1} in Spherical Coordinates <br>for {TVType}{' Plan' if pP else ''}s {site}"
        fig.update_layout(      title=titlei,
                                    yaxis_title='Polar Angle θ [rad]',
                                    xaxis_title='Azimuth φ [rad]',
                                    xaxis = dict(
                                        tickmode = 'array',
                                        tickvals = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                                        ticktext = ['-π', '-π/2', '0', 'π/2', 'π']
                                    ),
                                    yaxis = dict(
                                        tickmode = 'array',
                                        tickvals = [0, np.pi/2, np.pi],
                                        ticktext = ['0', 'π/2', 'π']
                                    ),
                                    margin=dict(r=0, b=0, l=0, t=60),
                                    yaxis_range=[0,np.pi], xaxis_range=[-np.pi,np.pi], height=400, width=800
                                    )
        if show: fig.show()
        if save: 
            fig.write_image(f"{folder}/{titlei.replace('<br>','')}.pdf")
            fig.write_image(f"{folder}/{titlei.replace('<br>','')}.png")
    fig=go.Figure(traces)
    title=f"Directions of Principal Components in Spherical Coordinates <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig.update_layout(      title=title,
                                yaxis_title='Polar Angle θ [rad]',
                                xaxis_title='Azimuth φ [rad]',
                                xaxis = dict(
                                    tickmode = 'array',
                                    tickvals = [-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                                    ticktext = ['-π', '-π/2', '0', 'π/2', 'π']
                                ),
                                yaxis = dict(
                                    tickmode = 'array',
                                    tickvals = [0, np.pi/2, np.pi],
                                    ticktext = ['0', 'π/2', 'π']
                                ),
                                margin=dict(r=0, b=0, l=0, t=60),
                                yaxis_range=[0,np.pi], xaxis_range=[-np.pi,np.pi], height=400, width=800
                                )
    if show: fig.show()
    if save:
        fig.write_image(f"{folder}/{title.replace('<br>','')}.pdf")
        fig1.write_image(f"{folder}/{title1.replace('<br>','')}.pdf")
        fig.write_image(f"{folder}/{title.replace('<br>','')}.png")
        fig1.write_image(f"{folder}/{title1.replace('<br>','')}.png")

def plotPCVar(dfpca,save=None,show=True,pP=True,TVType='PTV',site=''):
    PC_Var=np.concatenate(dfpca['PC_Var'].to_numpy()).reshape(-1,3)
    if save:
        save=f"{save}/Var"
    plotTertHist(PC_Var,save, f"Ternary Plot of Variance Ratio of Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}",show,pP,TVType,site)

def plotPCStd(dfpca,save=None,show=True,pP=True,TVType='PTV',site=''):
    PC_Std=np.sqrt(np.concatenate(dfpca['PC_Var'].to_numpy()).reshape(-1,3))
    if save:
        save=f"{save}/Std"
    plotTertHist(PC_Std,save,f"Ternary Plot of Standard Deviation Ratio of Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}",show,pP,TVType,site)

def plotPCEfSize(dfpca,save=None,show=True,pP=True,TVType='PTV',site=''):
    length=np.concatenate(dfpca['Size'].to_numpy()).reshape(-1,3)/10
    # print("Eff Size",np.mean(length), np.std(length), np.median(length))
    if len(length)<1000:
        size=4
    else:
        size=2
    rmin = np.min(length[:,:2])
    rmax = np.max(length[:,:2])
    fig=go.Figure(go.Scatter(x=length[:,0],y=length[:,1],marker=dict(color=length[:,2],size=size,showscale=True,colorbar=dict(title="Tertiary [cm]")),mode='markers'), layout={"height":600, "width":600})
    title=f"Effective Size along Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig.update_layout(title=title,xaxis_title='Primary [cm]', yaxis_title='Secondary [cm]',xaxis_range=[rmin,rmax], yaxis_range=[rmin,rmax])
    dtick=rmax/10
    if dtick > 1: dtick=np.round(dtick)
    else: dtick = np.round(dtick,1)
    fig.update_yaxes(dtick=dtick)
    fig.update_xaxes(dtick=dtick)
    if show: fig.show()
    if save:
        folder = f"{save}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        fig.write_image(f"{folder}/{title.replace('<br>','')}.pdf")
        fig.write_image(f"{folder}/{title.replace('<br>','')}.png")
    plotTertHist(length,save,f"Ternary Plot of Effective Size Ratio of Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}",show,pP,TVType,site)

def plotMinMax(dfpca,save=None,show=True,pP=True,TVType='PTV',site=''):
    try:
        PC_min=np.concatenate(dfpca['min'].to_numpy()).reshape(-1,3)
        PC_max=np.concatenate(dfpca['max'].to_numpy()).reshape(-1,3)
    except:
        PC_min=dfpca['min'].to_numpy().reshape(-1,3)
        PC_max=dfpca['max'].to_numpy().reshape(-1,3)
    length=(PC_max-PC_min)/10
    # print("Max Size Couch",np.mean(length), np.std(length), np.median(length))
    if len(length)<1000:
        size=4
    else:
        size=2
    fig=go.Figure(go.Scatter(x=length[:,0],y=length[:,1],marker=dict(color=length[:,2],size=size,showscale=True,colorbar=dict(title="z [cm]")),mode='markers',), layout={"height":600, "width":600})
    rmin = np.min(length[:,:2])
    rmax = np.max(length[:,:2])
    title=f"Max Size along X, Y, Z (Couch System) <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig.update_layout(title=title,xaxis_title='X [cm]', yaxis_title='Y [cm]',xaxis_range=[rmin,rmax], yaxis_range=[rmin,rmax])
    dtick=rmax/10
    if dtick > 1: dtick=np.round(dtick)
    else: dtick = np.round(dtick,1)
    fig.update_yaxes(dtick=dtick)
    fig.update_xaxes(dtick=dtick)
    if show: fig.show()
    if save:
        folder = f"{save}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        fig.write_image(f"{folder}/{title.replace('<br>','')}.pdf")
        fig.write_image(f"{folder}/{title.replace('<br>','')}.png")

def plotPCL(dfpca,save=None,show=True,pP=True,TVType='PTV',site=''):
    try:
        PC_min=np.concatenate(dfpca['PC_min'].to_numpy()).reshape(-1,3)
        PC_max=np.concatenate(dfpca['PC_max'].to_numpy()).reshape(-1,3)
    except:
        PC_min=dfpca['PC_min'].to_numpy().reshape(-1,3)
        PC_max=dfpca['PC_max'].to_numpy().reshape(-1,3)

    length=(PC_max-PC_min)/10
    # print("Max Size",np.mean(length), np.std(length), np.median(length))
    # plot 3d instead of 2d
    # fig=go.Figure(go.Scatter3d(x=length[:,0],y=length[:,1],z=length[:,2],mode='markers'))
    # fig.update_layout(      title="Principal Components Measures",
    #                             scene = dict(xaxis_title='Primary',
    #                             yaxis_title='Secondary',
    #                             zaxis_title='Tertiary'),
    #                             # legend=dict(yanchor="top",
    #                             #             y=0,
    #                             #             xanchor="left",
    #                             #             x=1),
    #                             # #width=700,
    #                             margin=dict(r=0, b=0, l=0, t=60),
    #                             # scene_camera=dict(eye=dict(x=2.0, y=2.0, z=0.75))
    #                             )
    
    if len(length)<1000:
        size=4
    else:
        size=2
    rmin = np.min(length[:,:2])
    rmax = np.max(length[:,:2])
    fig=go.Figure(go.Scatter(x=length[:,0],y=length[:,1],marker=dict(color=length[:,2],size=size,showscale=True,colorbar=dict(title="Tertiary [cm]")),mode='markers'), layout={"height":600, "width":600})
    title=f"Max Size along Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig.update_layout(title=title,xaxis_title='Primary [cm]', yaxis_title='Secondary [cm]',xaxis_range=[rmin,rmax], yaxis_range=[rmin,rmax])#margin=dict(r=0, b=0, l=0, t=60))
    dtick=rmax/10
    if dtick > 1: dtick=np.round(dtick)
    else: dtick = np.round(dtick,1)
    fig.update_yaxes(dtick=dtick)
    fig.update_xaxes(dtick=dtick)
    if show: fig.show()
    if save:
        folder = f"{save}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        fig.write_image(f"{folder}/{title.replace('<br>','')}.pdf")
        fig.write_image(f"{folder}/{title.replace('<br>','')}.png")
        save=f"{save}/Max Size"
    # plotTertHist(length,save,f"Ternary Plot of Max Size Ratio of Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}",show,pP,TVType,site)

def plothistdf(df,x,save=None,show=True,pP=True,TVType='PTV',site=''):
    fig1 = px.histogram(df,x=x)
    title=f"{x} <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig1.update_layout(      title=title,
                                xaxis_title=x,
                                yaxis_title=f"Number of {TVType}{' Plan' if pP else ''}s",
                                height=300,
                                width=600,
                                margin=dict(r=0, b=0, l=0, t=60)
                                )
    if x=='Volume':
        fig1.update_xaxes(range=[0,2000])

    if show: fig1.show()
    if save:
        if not os.path.exists(save):
            os.makedirs(save)
        fig1.write_image(f"{save}/{title.replace('<br>','').replace('/','_')}.pdf")
        fig1.write_image(f"{save}/{title.replace('<br>','').replace('/','_')}.png")

def plothistdfs(dfs,cats,x,save=None,show=True,pP=True,TVType='PTV'):
    try:
        fig1=go.Figure()
        hist_data=[]
        names=[]
        for i,df in enumerate(dfs):
            h=df[x][~np.isnan(df[x])]
            if len(h)>1:
                if cats[i]!='Benign':
                    hist_data.append(h)
                    names.append(cats[i])

        # for i,df in enumerate(dfs):
        #     fig1.add_trace(go.Histogram(x=df[x],name=cats[i]))
        fig1=ff.create_distplot(hist_data, names, show_hist=False, show_rug=False,colors=colors[:3*len(cats):3])
        title=f"Histogram of {x} <br>for {TVType}{' Plan' if pP else ''}s"
        fig1.update_layout(      title=title,
                                    xaxis_title=x,
                                    yaxis_title=f"Kernel Density Estimation of Count",
                                    height=400,
                                    width=600,
                                    margin=dict(r=0, b=0, l=0, t=60)
                                    )
        # # Overlay both histograms
        # fig1.update_layout(barmode='overlay')
        # # Reduce opacity to see both histograms
        # fig1.update_traces(opacity=0.75)
        if x=='Volume':
            fig1.update_xaxes(range=[0,2000])
        elif x=='HU Median':
            fig1.update_xaxes(range=[-800,200])

        if show: fig1.show()
        if save:
            if not os.path.exists(save):
                os.makedirs(save)
            fig1.write_image(f"{save}/{title.replace('<br>','').replace('/','_')}.pdf")
            fig1.write_image(f"{save}/{title.replace('<br>','').replace('/','_')}.png")
    except:
        print(f"Failed Histogram of {x} <br>for {TVType}{' Plan' if pP else ''}s, too little data?")

def plotattvsattlung(df,save=None,show=True,pP=True,TVType='PTV',site=''):
    fig1 = px.histogram(x=df["Max Depth q0_attLung"]-df["Max Depth q0_att"])
    title=f"Difference Depth q0_attLung - q0_att <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig1.update_layout(      title=title,
                                xaxis_title="Max Depth q0_attLung - q0_att [cm]",
                                yaxis_title=f"Number of {TVType}{' Plan' if pP else ''}s",
                                height=300,
                                width=600,
                                margin=dict(r=0, b=0, l=0, t=60)
                                )
    if show: fig1.show()
    if save:
        if not os.path.exists(save):
            os.makedirs(save)
        fig1.write_image(f"{save}/{title.replace('<br>','')}.pdf")
        fig1.write_image(f"{save}/{title.replace('<br>','')}.png")

def plotTechnique(df,save=None,show=True,pP=True,TVType='PTV',site=''):
    fig1 = px.histogram(x=df["Rx technique"], text_auto=True)
    title=f"Technique <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig1.update_layout(      title=title,
                                xaxis_title="Technique",
                                yaxis_title=f"Number of {TVType}{' Plan' if pP else ''}s",
                                height=300,
                                width=600,
                                margin=dict(r=0, b=0, l=0, t=60)
                                )
    if show: fig1.show()
    if save:
        if not os.path.exists(save):
            os.makedirs(save)
        fig1.write_image(f"{save}/{title.replace('<br>','')}.pdf")
        fig1.write_image(f"{save}/{title.replace('<br>','')}.png")

def plotMUAnglehist(df,save=None,show=True,pP=True,TVType='PTV',site=''):
    y=[]
    for entry in df["MU per Angle"]:
        try:
            for e in entry:
                for ee in e:
                    y.append(ee)
        except:
            pass
    x=[]
    for entry in df["Gantry Angle"]:
        try:
            for e in entry:
                for ee in e:
                    x.append(ee)
        except:
            pass
    fig1 = px.histogram(x=x,y=y)
    title=f"MU per Gantry Angle Histogram <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig1.update_layout(      title=title,
                                xaxis_title="Gantry Angle",
                                yaxis_title='MU',
                                height=600,
                                width=1200,
                                margin=dict(r=0, b=0, l=0, t=60)
                                )
    if show: fig1.show()
    if save:
        if not os.path.exists(save):
            os.makedirs(save)
        fig1.write_image(f"{save}/{title.replace('<br>','')}.pdf")
        fig1.write_image(f"{save}/{title.replace('<br>','')}.png")

def plotskew(df,save=None,show=True,pP=True,TVType='PTV',site=''):
    try:
        skew=np.concatenate(df['PC_Skew'].to_numpy()).reshape(-1,3)
    except:
        skew=df['PC_Skew'].to_numpy().reshape(-1,3)
    # plot 3d instead of 2d
    norm=np.linalg.norm(skew,axis=1)
    fig1 = px.histogram(x=norm)
    title1=f"L2 norm of Skewness <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig1.update_layout(      title=title1, #does that actually make sense? Why not L1?
                                xaxis_title='L2 norm of Skewness',
                                yaxis_title=f"Number of {TVType}{' Plan' if pP else ''}s",
                                height=300,
                                width=600,
                                margin=dict(r=0, b=0, l=0, t=60),
                                )
    if show: fig1.show()
    fig2=go.Figure(go.Scatter3d(x=skew[:,0],y=skew[:,1],z=skew[:,2],mode='markers',marker=dict(color=norm,size=3)))
    title2=f"Skewness along Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig2.update_layout(      title=title2,
                                scene = dict(xaxis_title='Primary',
                                yaxis_title='Secondary',
                                zaxis_title='Tertiary'),
                                height=300,
                                width=600,
                                margin=dict(r=0, b=0, l=0, t=60),
                                scene_camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
                                )
    if show: fig2.show()
    rmin = np.min(skew[:,:2])
    rmax = np.max(skew[:,:2])
    fig3=go.Figure(go.Scatter(x=skew[:,0],y=skew[:,1],marker=dict(color=skew[:,2],showscale=True,colorbar=dict(title="Tertiary")),mode='markers'), layout={"height":600, "width":600})
    title3=f"Skewness along Principal Components <br>for {TVType}{' Plan' if pP else ''}s {site}"
    fig3.update_layout(title=title3,xaxis_title='Primary', yaxis_title='Secondary',xaxis_range=[rmin,rmax], yaxis_range=[rmin,rmax])#,margin=dict(r=0, b=0, l=0, t=60))
    dtick=rmax/10
    if dtick > 1: dtick=np.round(dtick)
    else: dtick = np.round(dtick,1)
    fig3.update_yaxes(dtick=dtick)
    fig3.update_xaxes(dtick=dtick)
    if show: fig3.show()
    
    if save:
        folder = f"{save}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        fig1.write_image(f"{folder}/{title1.replace('<br>','')}.pdf")#, height=600, width=1800)
        fig2.write_image(f"{folder}/{title2.replace('<br>','')}.pdf")#, height=600, width=1800)
        fig3.write_image(f"{folder}/{title3.replace('<br>','')}.pdf")#, height=600, width=1800)
        fig1.write_image(f"{folder}/{title1.replace('<br>','')}.png")#, height=600, width=1800)
        fig2.write_image(f"{folder}/{title2.replace('<br>','')}.png")#, height=600, width=1800)
        fig3.write_image(f"{folder}/{title3.replace('<br>','')}.png")#, height=600, width=1800)

from sklearn.preprocessing import normalize

def plotTertHist(PC_Var,save=None,title="Ternary Plot of Ratio of Principal Components",show=True,pP=True,TVType='PTV',site=''):
    
    spherical=(PC_Var[:,2]/PC_Var[:,0])>0.66
    discoid=(PC_Var[:,1][~spherical]/PC_Var[:,0][~spherical])>0.8
    ellipsoid=(PC_Var[:,2][~spherical][~discoid]/PC_Var[:,1][~spherical][~discoid])<0.8
    cylindrical=~ellipsoid
    fig1=px.histogram(x=['Cylindrical','Ellipsoid', 'Discoid', 'Spherical'],y=[np.sum(cylindrical),np.sum(ellipsoid),np.sum(discoid),np.sum(spherical)])
    title1=f"Histogram of number of {TVType}{' Plan' if pP else ''}s <br>classified per Shape {site}"
    fig1.update_layout(      title=title1,
                                xaxis_title='Classified Shape',
                                yaxis_title=f"Number of {TVType}{' Plan' if pP else ''}s",
                                height=300,
                                width=600,
                                margin=dict(r=0, b=0, l=0, t=60)
                                )
    if show: fig1.show()
    norm=normalize(PC_Var,norm='l1')
    sphere=norm[spherical]
    
    if len(norm)<1000:
        size=4
    else:
        size=2
    f1=go.Scatterternary(a=sphere[:,0],b=sphere[:,1],c=sphere[:,2],mode='markers',name="Spherical",marker=dict(size=size))
    disc=norm[~spherical][discoid]
    f2=go.Scatterternary(a=disc[:,0],b=disc[:,1],c=disc[:,2],mode='markers',name='Discoid',marker=dict(size=size))
    ellipse=norm[~spherical][~discoid][ellipsoid]
    f3=go.Scatterternary(a=ellipse[:,0],b=ellipse[:,1],c=ellipse[:,2],mode='markers',name='Ellipsoid',marker=dict(size=size))
    cylinder=norm[~spherical][~discoid][~ellipsoid]
    f4=go.Scatterternary(a=cylinder[:,0],b=cylinder[:,1],c=cylinder[:,2],mode='markers',name='Cylindrical',marker=dict(size=size))
    fig=go.Figure([f1,f2,f3,f4], layout={"height":600, "width":600})
    fig.update_layout(title=title)
    if show: fig.show()
    
    if save:
        folder = f"{save}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        fig1.write_image(f"{folder}/{title1.replace('<br>','')}.pdf")#, height=600, width=1800)
        fig.write_image(f"{folder}/{title.replace('<br>','')}.pdf")#, height=600, width=1800)
        fig1.write_image(f"{folder}/{title1.replace('<br>','')}.png")#, height=600, width=1800)
        fig.write_image(f"{folder}/{title.replace('<br>','')}.png")#, height=600, width=1800)

    # plot all pointclouds per category
    
    # sphere = np.concatenate(dfpca['XYZ'].to_numpy()[spherical]).reshape(-1,3)
    # fig2= go.Figure(go.Scatter3d(x=sphere[:,0],y=sphere[:,1],z=sphere[:,2],mode='markers',marker=dict(size=2)))
    # fig2.show()
    # disc = np.concatenate(dfpca['XYZ'].to_numpy()[~spherical][discoid]).reshape(-1,3)
    # fig3= go.Figure(go.Scatter3d(x=disc[:,0],y=disc[:,1],z=disc[:,2],mode='markers',marker=dict(size=2)))
    # fig3.show()
    # ellipse = np.concatenate(dfpca['XYZ'].to_numpy()[~spherical][~discoid][ellipsoid]).reshape(-1,3)
    # fig4= go.Figure(go.Scatter3d(x=ellipse[:,0],y=ellipse[:,1],z=ellipse[:,2],mode='markers',marker=dict(size=2)))
    # fig4.show()
    # cylinder = np.concatenate(dfpca['XYZ'].to_numpy()[~spherical][~discoid][~ellipsoid]).reshape(-1,3)
    # fig5= go.Figure(go.Scatter3d(x=cylinder[:,0],y=cylinder[:,1],z=cylinder[:,2],mode='markers',marker=dict(size=2)))
    # fig5.show()

def plotTreatableDepthVol(df,save,show,depth,vol,dcolumns,vcolumn,pP,TVType,site):
    for dcolumn in dcolumns:
        dft=df.loc[~np.isnan(df[dcolumn])]
        total=lenu(dft['PlanSetupSer'],pP)
        data=[]
        for v in vol:
            # total=np.array([lenu(df['PlanSetupSer'],pP)-lenu(df['PlanSetupSer'][np.any([df[vcolumn]>v,df[dcolumn]>d],0)],pP) for d in depth])
            totaltr=np.array([lenu(dft['PlanSetupSer'][np.all([dft[vcolumn]<=v,dft[dcolumn]<=d],0)],pP) for d in depth])
            y=totaltr/total*100
            std=np.sqrt(totaltr)/total*100
            data.append(go.Scatter(x=depth,y=y,error_y=dict(array=std),name=f"{v}"),)
        title=f"Treatable {TVType}{' Plan' if pP else ''}s <br>for {dcolumn}, {vcolumn}, {site}"
        fig=go.Figure(data)
        fig.update_layout(      title=title,
                                    xaxis_title=f"Max. Treatable {dcolumn} [cm]",
                                    yaxis_title='Percent Treatable of Total',
                                    height=600,
                                    width=600,
                                    margin=dict(r=0, b=0, l=0, t=60),
                                    legend={"title":f"Max. Treatable <br> {vcolumn} [cc]"}
                                    )
        if show: fig.show()
        if save:
            if not os.path.exists(save):
                os.makedirs(save)
            fig.write_image(f"{save}/{title.replace('<br>','')}.pdf")
            fig.write_image(f"{save}/{title.replace('<br>','')}.png")




def plotbardfcat(dfs,cats,x,plot,pP,TVType,width,show,func=None,log=False): 
    if x=="Length" or x=="LengthXYZ" or x=="Std" or x=="Skew" or x=='Effective Size':     
        if x=="Length":
            length=[(np.concatenate(d['PC_max'].to_numpy()).reshape(-1,3)-np.concatenate(d['PC_min'].to_numpy()).reshape(-1,3))/10 for d in dfs]
            yaxis_title=f"Max Size along Principal Axes [cm]"# with 95% Confidence Interval [cm]"   
            title=f"Max Size along Principal Axes per Site"
        elif x=="LengthXYZ":
            length=[(np.concatenate(d['max'].to_numpy()).reshape(-1,3)-np.concatenate(d['min'].to_numpy()).reshape(-1,3))/10 for d in dfs]
            yaxis_title=f"Max Size along XYZ Axes [cm]"# with 95% Confidence Interval [cm]"
            title=f"Max Size along XYZ Axes (Couch System) per Site"
        elif x=="Std":
            length=[(np.sqrt(np.concatenate(d['PC_Var'].to_numpy()).reshape(-1,3)))/10 for d in dfs]
            yaxis_title=f"Standard Deviation along Principal Axes [cm]"# with 95% Confidence Interval [cm]"
            title=f"Standard Deviation along Principal Axes per Site"
        elif x=="Effective Size":
            length=[np.concatenate(d['Size'].to_numpy()).reshape(-1,3)/10 for d in dfs]
            yaxis_title=f"Effective Size along Principal Axes [cm]"# with 95% Confidence Interval [cm]"
            title=f"Effective Size along Principal Axes per Site"
        elif x=="Skew":
            length=[np.concatenate(d['PC_Skew'].to_numpy()).reshape(-1,3) for d in dfs]
            yaxis_title=f"Skewness along Principal Axes"# with 95% Confidence Interval"
            title=f"Skewness along Principal Axes per Site"
        else:
            warnings.warn(f"{x} not recognized")
            return
        data=[]
        for i in range(3):
            y=np.concatenate([l[:,i] for l in length])
            catss=[]
            for c,cat in enumerate(cats):
                for _ in range(len(length[c])):
                    catss.append(cat)
            data.append(go.Box(name=f"Axis {i+1}",y=y,x=catss,boxmean=True,notched=True,showlegend=True))
        fig = go.Figure(data)#
        fig.update_layout(boxmode='group')
        if not x=='Skew':
            fig.update_yaxes(range=[0,30])
        else:
            fig.update_yaxes(range=[-1,1])
            # fig.update_yaxes(range=[0,None],autorange='max')#rangemode="tozero")
        # fig.update_layout(barmode='group')
        xaxis_title='Site'
    elif x=="Dif. of Max Depth q0_attLung - q0_att":
        xaxis_title='Site'
        title=f"{x} per Site"
        dif=[d["Max Depth q0_attLung"]-d["Max Depth q0_att"] for d in dfs]
        catss=[]
        for c,cat in enumerate(cats):
            for _ in range(len(dif[c])):
                catss.append(cat)
        yaxis_title=f"{x} [cm] with SD"
        boxmean='sd'
        fig=go.Figure(go.Box(y=np.concatenate(dif),x=catss,boxmean=boxmean,notched=True,showlegend=False))
        fig.update_yaxes(range=[-5,5])
    elif x=="Dif. of Max Depth q0 - q0_att":
        xaxis_title='Site'
        title=f"{x} per Site"
        dif=[d["Max Depth q0"]-d["Max Depth q0_att"] for d in dfs]
        catss=[]
        for c,cat in enumerate(cats):
            for _ in range(len(dif[c])):
                catss.append(cat)
        yaxis_title=f"{x} [cm] with SD"
        boxmean='sd'
        fig=go.Figure(go.Box(y=np.concatenate(dif),x=catss,boxmean=boxmean,notched=True,showlegend=False))
        fig.update_yaxes(range=[-5,5])
    elif x=="Dif. of Max Depth q0 - q0_attLung":
        xaxis_title='Site'
        title=f"{x} per Site"
        dif=[d["Max Depth q0"]-d["Max Depth q0_attLung"] for d in dfs]
        catss=[]
        for c,cat in enumerate(cats):
            for _ in range(len(dif[c])):
                catss.append(cat)
        yaxis_title=f"{x} [cm] with SD"
        boxmean='sd'
        fig=go.Figure(go.Box(y=np.concatenate(dif),x=catss,boxmean=boxmean,notched=True,showlegend=False))
        fig.update_yaxes(range=[0,5])
    elif x=="Count TV":
        xaxis_title='Site'
        title=f"Number of {TVType}s per Plan per Site"
        dif=[np.unique(d['PlanSetupSer'],return_counts=True)[1] for d in dfs]
        catss=[]
        for c,cat in enumerate(cats):
            for _ in range(len(dif[c])):
                catss.append(cat)
        yaxis_title=f"Number of {TVType}s per Plan with SD"
        boxmean='sd'
        fig=go.Figure(go.Box(y=np.concatenate(dif),x=catss,boxmean=boxmean,notched=True,showlegend=False))
    elif x=='Prescribed Dose (cGy)':
        xaxis_title='Site'
        title=f"{x} per Site"
        y=[d['Num Fractions']*d['Dose/Frc (cGy)'] for d in dfs]
        catss=[]
        for c,cat in enumerate(cats):
            for _ in range(len(y[c])):
                catss.append(cat)
        fig=go.Figure()
        boxmean=True
        yaxis_title=f"{x}"
        fig.add_box(y=np.concatenate(y),x=catss,boxmean=boxmean,notched=True,showlegend=False)
        fig.update_yaxes(range=[0,None],autorange='max')
    else:
        xaxis_title='Site'
        title=f"{x} per Site"
        if func:
            y=[[func(a) for a in d[x]] for d in dfs]
        else:    
            y=[d[x] for d in dfs]
        catss=[]
        for c,cat in enumerate(cats):
            for _ in range(len(y[c])):
                catss.append(cat)
        fig=go.Figure()
        if x=="Max Depth q0_attLung" or x=="Max Depth q0_att" or x=="Max Depth q0" or x=="Min Depth q0_attLung" or x=="Min Depth q0_att" or x=="Min Depth q0":
            boxmean='sd'
            yaxis_title=f"{x} [cm]"
            fig.add_box(y=np.concatenate(y),x=catss,boxmean=boxmean,notched=True,showlegend=False)
            # fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)
            # fig.add_scatter(name='8 cm',x = [0, 2], y = [8, 8], mode='lines', xaxis='x2',line=dict(dash='dash', color = "firebrick", width = 2))
            # fig.update_layout(yaxis_range=[0,15])
            # fig.update_yaxes(range=[0,None],autorange='max')
        else:
            boxmean=True
            yaxis_title=f"{x}"
            fig.add_box(y=np.concatenate(y),x=catss,boxmean=boxmean,notched=True,showlegend=False)
    
    if 'olume' in x:
        fig.update_layout(yaxis_range=[0,3000])
        yaxis_title=f"{yaxis_title} [cc]"
        ## if 'Plan' in x:
        # fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)
        # fig.add_scatter(name='125 cc',x = [0, 2], y = [125, 125], mode='lines', xaxis='x2',line=dict(dash='dash', color = "firebrick", width = 2))
    elif "SSD" in x:
        fig.update_layout(yaxis_range=[80,100])
    elif "Beam Delivery Duration Limit [s]"==x:
        fig.update_layout(yaxis_range=[0,1000])
    elif "HU Median"==x:
        fig.update_layout(yaxis_range=[-500,500])
    # elif "Index" in x:
    #     fig.update_layout(yaxis_range=[0,100])
    if log:
        fig.update_yaxes(dict(type='log'))
    if width>1000:
        height=600
    else:
        height=300
    fig.update_layout(      title=f"{title}<br> {TVType if not TVType in title else ''}{' Plans' if pP else ''}",
                                    yaxis_title=yaxis_title,
                                    xaxis_title=xaxis_title
                                    )
    
    
    if not os.path.exists(plot):
        os.makedirs(plot)
    fig.write_html(f"{plot}/{title.replace('/','_')} {TVType}{' Plans' if pP else ''}.html")
    fig.update_layout(  height=height,
                        width=width,
                         margin=dict(r=0, b=0, l=0, t=60))
    if show: fig.show()
    fig.write_image(f"{plot}/{title.replace('/','_')} {TVType}{' Plans' if pP else ''}.pdf")
    fig.write_image(f"{plot}/{title.replace('/','_')} {TVType}{' Plans' if pP else ''}.png")

def lenu(x,do):
    if do:
        return len(x.unique())
    else:
        return len(x)

def plottreatable(df,dfs,cats,x,plot,pP,TVType,width,vol=125,depth=8,show=True):
    if x=='Treatable_q0_att_VolumeVerifier' or x=='Treatable_q0_attLung_VolumeVerifier' or x=='Treatable_q0_att_VolumeCalculated' or x=='Treatable_q0_attLung_VolumeCalculated':
        se1=[np.sqrt((lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]==False],pP)))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs]
        fig1=go.Bar(name="per Site",y=[(lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]==False],pP))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs],x=cats,error_y=dict(array=se1))
        y=[(lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]==False],pP))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        se2=[np.sqrt((lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]==False],pP)))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        total=lenu(df['PlanSetupSer'],pP)-lenu(df['PlanSetupSer'][df[x]==False],pP)
        fig2=go.Bar(name=f"Total {int(np.round(total/lenu(df['PlanSetupSer'],pP)*100))} ± {int(np.round(np.sqrt(total)/lenu(df['PlanSetupSer'],pP)*100))}%",y=y,x=cats,error_y=dict(array=se2))
        title=f"Treatable {TVType}{' Plans' if pP else ''} (Depth {'_'.join(x.split('_')[1:3])} <8cm, Volume<125cc)"
    elif x=='Treatable q0_att' or x=='Treatable q0_attLung':
        se1=[np.sqrt((lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]<1],pP)))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs]
        fig1=go.Bar(name="per Site",y=[(lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]<1],pP))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs],x=cats,error_y=dict(array=se1))
        y=[(lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]<1],pP))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        se2=[np.sqrt((lenu(d["PlanSetupSer"],pP)-lenu(d["PlanSetupSer"][d[x]<1],pP)))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        total=lenu(df['PlanSetupSer'],pP)-lenu(df['PlanSetupSer'][df[x]<1],pP)
        fig2=go.Bar(name=f"Total {int(np.round(total/lenu(df['PlanSetupSer'],pP)*100))} ± {int(np.round(np.sqrt(total)/lenu(df['PlanSetupSer'],pP)*100))}%",y=y,x=cats,error_y=dict(array=se2))
        title=f"Treatable {TVType}{' Plans' if pP else ''} <br>(Depth {'_'.join(x.split('_')[0:2])} <8cm)"
    elif x=='Plan PTV Volume Verifier' or x=='Volume' or x=='Max Depth q0_att' or x=='Max Depth q0_attLung' or x=='Max Depth q0':
        if 'Volume' in x:
            lim=vol
        else:
            lim=depth
        se1=[np.sqrt((lenu(d["PlanSetupSer"][d[x]<=lim],pP)))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs]
        fig1=go.Bar(name="per Site",y=[(lenu(d["PlanSetupSer"][d[x]<=lim],pP))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs],x=cats,error_y=dict(array=se1))
        y=[(lenu(d["PlanSetupSer"][d[x]<=lim],pP))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        se2=[np.sqrt((lenu(d["PlanSetupSer"][d[x]<=lim],pP)))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        total=lenu(df['PlanSetupSer'][df[x]<=lim],pP)
        fig2=go.Bar(name=f"Total {int(np.round(total/lenu(df['PlanSetupSer'],pP)*100))} ± {int(np.round(np.sqrt(total)/lenu(df['PlanSetupSer'],pP)*100))}%",y=y,x=cats,error_y=dict(array=se2))
        title=f"Treatable {TVType}{' Plans' if pP else ''} <br>({x} <{lim}{'cc' if 'Volume' in x else 'cm'})"
    elif x=='Volume Max Depth q0_att' or x=='Volume Max Depth q0_attLung' or x=='Volume Max Depth q0':
        x1='Volume'
        x2=" ".join(x.split(" ")[1:])
        se1=[np.sqrt((lenu(d["PlanSetupSer"][np.all([d[x1]<=vol,d[x2]<=depth],0)],pP)))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs]
        fig1=go.Bar(name="per Site",y=[(lenu(d["PlanSetupSer"][np.all([d[x1]<=vol,d[x2]<=depth],0)],pP))/lenu(d["PlanSetupSer"],pP)*100 for d in dfs],x=cats,error_y=dict(array=se1))
        y=[(lenu(d["PlanSetupSer"][np.all([d[x1]<=vol,d[x2]<=depth],0)],pP))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        se2=[np.sqrt((lenu(d["PlanSetupSer"][np.all([d[x1]<=vol,d[x2]<=depth],0)],pP)))/lenu(df["PlanSetupSer"],pP)*100 for d in dfs]
        total=lenu(df['PlanSetupSer'][np.all([df[x1]<=vol,df[x2]<=depth],0)],pP)
        fig2=go.Bar(name=f"Total {int(np.round(total/lenu(df['PlanSetupSer'],pP)*100))} ± {int(np.round(np.sqrt(total)/lenu(df['PlanSetupSer'],pP)*100))}%",y=y,x=cats,error_y=dict(array=se2))
        title=f"Treatable {TVType}{' Plans' if pP else ''} <br>({x2} <{depth}cm, {x1} <{vol}cc)"
    else:
        warnings.warn(f"{x} not recognized")
        return
    fig=go.Figure([fig1,fig2])
    
    if width>1000:
        height=600
    else:
        height=300
    fig.update_layout(      title=title,
                                xaxis_title='Site',
                                yaxis_title=f"Percentage of Treatable {TVType}{' Plans' if pP else 's'} with Estimated Standard Error (\u221An)",
                                height=height,
                                width=width,
                                margin=dict(r=0, b=0, l=0, t=60),
                                barmode='group'
                                )
    fig.update_yaxes(range=[0, 100])
    if show: fig.show()
    if plot:
        plot=f"{plot}/Treatable {x}"
        if not os.path.exists(plot):
            os.makedirs(plot)
        fig.write_image(f"{plot}/{title.replace('<br>','').replace('<', '')}.pdf")
        fig.write_image(f"{plot}/{title.replace('<br>','').replace('<', '')}.png")

def plotBeamType(dfs,cats,plot,width,show):
    static=[]
    dynamic=[]
    for d in dfs:
        staticd=[]
        dynamicd=[]
        for i,entry in enumerate(d["Beam Type"]):
            try:
                if entry[0]=='STATIC':
                    staticd.append(i)
                elif entry[0]=='DYNAMIC':
                    dynamicd.append(i)
            except:
                pass
        static.append(staticd)
        dynamic.append(dynamicd)
    ys=[len(d) for d in static]
    yd=[len(d) for d in dynamic]
    fig1=go.Bar(name=f"Static <br>(Total {np.sum(ys)})",y=ys,x=cats)
    fig2=go.Bar(name=f"Dynamic <br>(Total {np.sum(yd)})",y=yd,x=cats)
    title=f"Beam Type per Site Histogram"
    fig=go.Figure([fig1,fig2])
    
    if width>1000:
        height=600
    else:
        height=300
    fig.update_layout(      title=title,
                                xaxis_title='Site',
                                yaxis_title=f"Number of Plans with Beam Type",
                                height=height,
                                width=width,
                                margin=dict(r=0, b=0, l=0, t=60),
                                barmode='group'
                                )
    if show: fig.show()
    if plot:
        if not os.path.exists(plot):
            os.makedirs(plot)
        fig.write_image(f"{plot}/{title.replace('<br>','').replace('<', '')}.pdf")
        fig.write_image(f"{plot}/{title.replace('<br>','').replace('<', '')}.png")


def plotSRS(dfs,cats,plot,width,show):
    static=[]
    for d in dfs:
        staticd=[]
        for i,entry in enumerate(d["High Dose Technique Type"]):
            try:
                if entry[0]=='SRS':
                    staticd.append(i)
            except:
                pass
        static.append(staticd)
    ys=[len(d) for d in static]
    fig1=go.Bar(y=ys,x=cats)
    title=f"SRS per Site Histogram (Total {np.sum(ys)})"
    fig=go.Figure([fig1])
    
    if width>1000:
        height=600
    else:
        height=300
    fig.update_layout(      title=title,
                                xaxis_title='Site',
                                yaxis_title=f"Number of SRS Plans",
                                height=height,
                                width=width,
                                margin=dict(r=0, b=0, l=0, t=60),
                                showlegend=False
                                )
    if show: fig.show()
    if plot:
        if not os.path.exists(plot):
            os.makedirs(plot)
        fig.write_image(f"{plot}/{title.replace('<br>','').replace('<', '')}.pdf")
        fig.write_image(f"{plot}/{title.replace('<br>','').replace('<', '')}.png")

def plotavailabledata(df,dfs,cats,plot,title,width,show):
    if 'Plan' in title:
        y=np.array([len(d["PlanSetupSer"].unique()) for d in dfs])
        total=len(df["PlanSetupSer"].unique())
    else:
        y=np.array([len(d["PlanSetupSer"]) for d in dfs])
        total=len(df["PlanSetupSer"])

    fig1=go.Bar(name="Percent",y=y/total*100,x=cats,opacity=0)
    fig2=go.Bar(name=f"Total",y=y,x=cats,text=y,textposition='auto')
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(fig2,secondary_y=False)
    fig.add_trace(fig1,secondary_y=True)
    
    if width>1000:
        height=600
    else:
        height=300
    fig.update_layout(      title=f"{title} (Total: {total})",
                                xaxis_title='Site',
                                #yaxis_title=["Percentage of Plans",'# of Plans'],
                                height=height,
                                width=width,
                                margin=dict(r=0, b=0, l=0, t=60),
                                showlegend=False
                                # barmode='group'
                                )
    
    fig.update_yaxes(title_text="Percentage of Total", secondary_y=True)
    fig.update_yaxes(title_text="Count", secondary_y=False)
    # fig.layout.xaxis2 = go.layout.XAxis(overlaying='x', range=[0, 2], showticklabels=False)
    # fig.add_scatter(name='100',x = [0, 2], y = [100, 100], mode='lines', xaxis='x2',line=dict(dash='dash', color = "firebrick", width = 2))
    if show: fig.show()
    if plot:
        if not os.path.exists(plot):
            os.makedirs(plot)
    fig.write_image(f"{plot}/{title}.pdf")
    fig.write_image(f"{plot}/{title}.png")

def plotScatter(df,x,y,save,show,pP,TVType,site,func=None,log=False, line=False):
    try:
        xaxis_title=x
        yaxis_title=y
        title=f"{xaxis_title} vs {yaxis_title}<br> for {TVType}{' Plan' if pP else ''}s {site}"
        if save:
            save=f"{save}/{x.replace('<br>','').replace('/','_')}"
        if func:
            x=[func(a) for a in df[x]]
            if line=='id':  
                y=[func(a) for a in df[y]]
            else:
                y=df[y]
        else:
            x=df[x]
            y=df[y]
        
        if 'Mean Dose/Prescribed Dose' in yaxis_title:
            mask=y<1.3
            x=x[mask]
            y=y[mask]
        # mask=y<(np.mean(y)+5*np.std(y))
        # fig1=px.scatter(x=x[mask],y=y[mask])
            
        # if 'Conformity Index 1' in y or 'Conformity Index 4' in y:
        #     log=True
        # else:
        #     log=False
            
        # fig1=px.scatter(df,x,y,log_y=log)
        colorscale = ['#7A4579', '#D56073', 'rgb(236,158,105)', (1, 1, 0.2), (0.98,0.98,0.98)]
        if len(x)>1000:
            point_size=1
        else:
            point_size=3
        # fig1=px.density_heatmap(x=x,y=y)
        
        fig1=ff.create_2d_density(
        x, y, colorscale=colorscale, point_size=point_size)
        if func==np.cbrt:
            xaxis_title=f"{xaxis_title}^(1/3)"
            if line:   
                yaxis_title=f"{yaxis_title}^(1/3)"
        
        if 'Volume' in x:
            fig1.update_xaxes(range=[0,2000])
        if line=='id':
            p1=np.min([np.max(np.array(x)[~np.isnan(x)] ),np.max(np.array(y)[~np.isnan(y)])])
            fig1.add_scatter(x=[0,p1],y=[0,p1],name='1:1',mode='lines', 
                        opacity=0.5,line=dict(
                        # color='#0061ff',
                        width=1
                    ))
        elif line=='one':
            xnan=np.array(x)[~np.isnan(x)]
            fig1.add_scatter(x=[np.min(xnan),np.max(xnan)],y=[1,1],name='1',mode='lines', 
                        opacity=0.5,line=dict(
                        # color='#0061ff',
                        width=1))
        if 'Total' in xaxis_title:
            try:
                result = sm.OLS(y, x,'drop').fit()
                # printing the summary table
                print(result.summary())
                slope=result.params[0]
                # print(slope)
                # x=np.array(x)
                # y=np.array(y)
                # b1=x>100
                # x=x[b1]
                # y=y[b1]
                # b2=y>100
                # x=x[b2]
                # y=y[b2]
                # fig2=px.scatter(x=x, y=y, trendline="ols")#,trendline_options=dict(add_constant=False,log_x=True))
                # fig2.data = [t for t in fig2.data if t.mode == "lines"]
                # fig2.show()
                # p1=np.max(np.array(x)[~np.isnan(x)])
                p2=np.max(np.array(y)[~np.isnan(y)])
                fig1.add_scatter(x=[0,p2/slope],y=[0,p2],name=f'Fit Slope={np.round(slope,2)}',mode='lines', 
                            opacity=0.5,line=dict(
                            # color='#0061ff',
                            width=1
                        ))
                title=f"{title} (Fit Slope={np.round(slope,2)})"
            except:
                print('Fitting Failed.')
    # if "Index" in y:
    #     fig1.update_layout(yaxis_range=[0,100])
    
        fig1.update_layout(      title=title,
                                    xaxis_title=xaxis_title,
                                    yaxis_title=yaxis_title,
                                    legend=dict(visible=True,
                                                yanchor="top",
                                                y=0.99,
                                                xanchor="left",
                                                x=0.01)
                                    )
        if show: fig1.show()
        if save:
            if not os.path.exists(save):
                os.makedirs(save)
                fig1.write_html(f"{save}/{title.replace('<br>','').replace('/','_')}.html")
                fig1.update_layout(height=500,
                                    width=600,
                                    margin=dict(r=0, b=0, l=0, t=60),)
                fig1.write_image(f"{save}/{title.replace('<br>','').replace('/','_')}.pdf")
                fig1.write_image(f"{save}/{title.replace('<br>','').replace('/','_')}.png")
    except:
        print(f'{save} failed. Not enough data points?')

def plotIndices(df,x2,save,show,pP,TVType,site,func=None,log=True):
    plotScatter(df,x2,'Conformity Index 1',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Conformity Index 2',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Conformal Number',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Calc Conformity Index 1',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Calc Conformity Index 2',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Calc Conformal Number',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Dose Balance Index',save,show,pP,TVType,site,func,log)
    plotScatter(df,x2,'Homogeneity Index 1',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Lesion Underdosage Factor',save,show,pP,TVType,site,func,False)
    plotScatter(df,x2,'Calc Lesion Underdosage Factor',save,show,pP,TVType,site,func,False)


def plotall(df,depth,vol,dcolumn,vcolumn,pP,TVType,site,save=None,show=True):
    
    
    plotScatter(df,'Compactness','Solidity',save,show,pP,TVType,site)
    plotScatter(df,'Volume','Compactness',save,show,pP,TVType,site,np.cbrt)
    plotScatter(df,'Volume','Solidity',save,show,pP,TVType,site,np.cbrt)
    plotScatter(df,'Volume','Max Depth q0_att',save,show,pP,TVType,site,np.cbrt)
    plotTreatableDepthVol(df,save,show,depth,vol,dcolumn,vcolumn,pP,TVType,site)
    try:
        # plotPC(df,save,show,pP,TVType,site)
        plotMinMax(df,save,show,pP,TVType,site)
        # plotPCStd(df,save,show,pP,TVType,site)
        plotPCL(df,save,show,pP,TVType,site)
        # plotPCVar(df,save,show,pP,TVType,site)
        plotskew(df,save,show,pP,TVType,site)
        # try: 
        #     plothistdf(df,'PTV volume (cc)',save,show)
        # except:
        #     print('PTV volume (cc) not found')
        plotPCEfSize(df,save,show,pP,TVType,site)
    except:
        print(f'size stuff failed pP{pP}TV{TVType}{site}')
    plothistdf(df,'Volume',save,show,pP,TVType,site)
    plothistdf(df,'Compactness',save,show,pP,TVType,site)
    plothistdf(df,'Solidity',save,show,pP,TVType,site)
    plothistdf(df,'Max Depth q0_attLung',save,show,pP,TVType,site)
    plothistdf(df,'Max Depth q0_att',save,show,pP,TVType,site)
    plothistdf(df,'Max Depth q0',save,show,pP,TVType,site)
    if TVType=='PTV':
        plotIndices(df,'Volume',save,show,pP,TVType,site,np.cbrt)
        plotIndices(df,'Max Depth q0_att',save,show,pP,TVType,site)
        plotIndices(df,'Compactness',save,show,pP,TVType,site)
        plotIndices(df,'Solidity',save,show,pP,TVType,site)
        plotScatter(df,'Volume Reference Isodose [cc]', 'Target Volume Reference Isodose [cc]', save, show, pP, TVType,site, line='id', func=np.cbrt)
        plotScatter(df,'Calc Target Volume [cc]', 'Calc Target Volume Reference Isodose [cc]', save, show, pP, TVType,site, line='id', func=np.cbrt)
        plotScatter(df,'Target Volume [cc]', 'Target Volume Reference Isodose [cc]', save, show, pP, TVType,site, line='id', func=np.cbrt)
        if pP:
            plotTechnique(df,save,show,pP,TVType,site)
            plotScatter(df,'Calc Volume Reference Isodose [cc]', 'Calc Target Volume Reference Isodose [cc]', save, show, pP, TVType,site, line='id', func=np.cbrt)
            plotScatter(df,'Total Dose TV [cGy]', 'Total Dose body sin. TV [cGy]', save, show, pP, TVType,site,line='id', func=np.cbrt)
            # plothistdf(df,'Total Meterset/Prescribed Dose [MU/cGy]',save,show,pP,TVType,site)
            plothistdf(df,'Total Meterset/Dose per Fraction [MU/cGy]',save,show,pP,TVType,site)
            # plothistdf(df,'Dose/Frc/Total Meterset [cGy/MU]',save,show,pP,TVType,site)
            plotIndices(df,'Total Meterset [MU]',save,show,pP,TVType,site)
            plotScatter(df,'Total Meterset/Dose per Fraction [MU/cGy]','Max Depth q0_att',save,show,pP,TVType,site)
            plotScatter(df,'Total Meterset/Dose per Fraction [MU/cGy]','Compactness',save,show,pP,TVType,site)
            plotScatter(df,'Total Meterset/Dose per Fraction [MU/cGy]','Solidity',save,show,pP,TVType,site)
            plotScatter(df,'Total Meterset/Dose per Fraction [MU/cGy]','Volume',save,show,pP,TVType,site,log=True)
            plotScatter(df,'Total Meterset [MU]','Max Depth q0_att',save,show,pP,TVType,site)
            plotScatter(df,'Total Meterset [MU]','Volume',save,show,pP,TVType,site,log=True)
            plotScatter(df,'Total Meterset [MU]','Dose/Frc (cGy)',save,show,pP,TVType,site)
            # plotMUAnglehist(df,save,show,pP,TVType,site)
            plothistdf(df,"Num Fractions",save,show,pP,TVType,site)
            plothistdf(df,"Dose/Frc (cGy)",save,show,pP,TVType,site)

        else:
            plothistdf(df,'Mean Dose/Prescribed Dose',save,show,pP,TVType,site)
            plotScatter(df,'Prescribed Dose [cGy]','Mean dose (cGy)',save,show,pP,TVType,site,line='id')
            plotScatter(df,'Prescribed Dose [cGy]','Mean Dose/Prescribed Dose',save,show,pP,TVType,site,line='one')
            plothistdf(df,"Mean dose (cGy)",save,show,pP,TVType,site)
    # plotattvsattlung(df,save,show,pP,TVType,site)

def analyzebycategory(dfmain,save,perPlan=True,fine=False,plotSiteDetails=False,TVTypes=['PrePTV','PTV','CTV','GTV'],show=True):
    """
    Takes the final results dataframe and performs the statistical analysis in form of plots.

    In:
    - dfmain; Pandas dataframe, MergedResults.pickle generated by mergeAllResults
    - save; str, folder in which plots will be stored
    - perPlan; bool, determines whether the analysis should be perdormed per plan (structs merged), or per structure
    - fine; bool, If true, will plot subsites, if false, will plot main sites
    - plotSiteDetails; bool, If true, generates plots for each site, if false, only generates site comparison plots
    - TVType; str array ['PrePTV','PTV','CTV','GTV'], determine which kinds of TV to perform the analysis on
    - show; bool, If true, plots will be shown in notebook, if False, only stored. Set to false for many plots

    Out:
    - dfs; Pandas dataframe, filtered according to the conditions given as input
    - Many, many plots in save folder.

    """
    dfmain=dfmain.dropna(subset=['TV Type'])
    dfmain.rename(columns={'Calc Conformity Index 4':'Dose Balance Index','Calc Homogeneity Index 2':'Calc Lesion Underdosage Factor', 'Homogeneity Index 2' : 'Lesion Underdosage Factor','Calc Conformity Index 3':'Calc Conformal Number','Conformity Index 3':'Conformal Number','Calc Total Dose [cGy]' : 'Total Dose TV [cGy]','Calc body Total Dose [cGy]':'Total Dose body sin. TV [cGy]'},inplace=True)
    if fine: dfmain["Rx site"]=[c.replace('/','_') for c in dfmain["Rx site"]]
    # dfmain['Prescribed Dose [cGy]']=dfmain['Num Fractions']*dfmain['Dose/Frc (cGy)']
    dfmain['Mean Dose/Prescribed Dose']=dfmain['Mean dose (cGy)']/dfmain['Prescribed Dose [cGy]']
    dfmain['Total Meterset [MU]']=[np.sum(a) for a in dfmain['Meterset [MU]']]
    # dfmain['Dose/Frc/Total Meterset [cGy/MU]']=dfmain['Dose/Frc (cGy)']/dfmain['Total Meterset [MU]']
    # dfmain['Total Meterset/Prescribed Dose [MU/cGy]']=dfmain['Total Meterset [MU]']/dfmain['Prescribed Dose [cGy]']
    dfmain['Total Meterset/Dose per Fraction [MU/cGy]']=dfmain['Total Meterset [MU]']/dfmain['Dose/Frc (cGy)']

    if perPlan:
        dfmain=dfmain[[('All' in tv or 'Solo' in tv) for tv in dfmain['TV Type']]]
    else:
        dfmain=dfmain[['All' not in tv for tv in dfmain['TV Type']]]
    for TVType in TVTypes:
        if TVType=='PrePTV':
            try:
                dfmaing=dfmain.groupby('PlanSetupSer').filter(lambda x: not np.any(["CTV" in y for y in x['TV Type']]))
            except:
                dfmaing=dfmain.groupby('Plan Index (Anonymized)').filter(lambda x: not np.any(["CTV" in y for y in x['TV Type']]))
            df=pd.concat([dfmain[['CTV' in tv for tv in dfmain['TV Type']]],dfmaing[['GTV' in tv for tv in dfmaing['TV Type']]]]).copy()
        else:
            df=dfmain[[TVType in tv for tv in dfmain['TV Type']]].copy()
        try:
            dfCT=df.loc[~np.isnan(df['Max Depth q0_att'])]
        except:
            dfCT=df.loc[~np.isnan(df['Max Depth q0_att [cm]'])]
        if fine:
            cats=df["Rx site"].unique()
            cats.sort()
            sites=[[c] for c in cats]
            catsCT=dfCT["Rx site"].unique()
            catsCT.sort()
            sitesCT=[[c] for c in catsCT]
            width=1800
            plot=f"{save}/Sub Sites/{'Per Plan' if perPlan else 'Per Structure'}/{TVType}"
        else:
            cats=["Pediatric", "Breast", "H&N", "Skin", "GI", "GU", "Gyn", "CNS", "Heme", "Sarcoma", "Thoracic", "Secondary", "Benign"]
            sites=[["Peds"],
                    ["Breast"],
                    ["H&N"],
                    ["Skin"],
                    ["GI"],
                    ["GU"],
                    ["Gyn"],
                    ["CNS"],
                    ["Heme"],
                    ["Sarcoma"],
                    ["Thoracic"],
                    ["Secondary"],
                    ["Benign"]]
            catsCT=cats
            sitesCT=sites
            width=600
            plot=f"{save}/Major Sites/{'Per Plan' if perPlan else 'Per Structure'}/{TVType}"
        # df=df[~np.isnan(df['Max Depth q0'])]
        dfs=[df[[np.any([s in p for s in sites[c]]) for p in df["Rx site"]]] for c,_ in enumerate(cats)] 
        # dmax=df['Max Depth q0_att'][~np.isnan(df['Max Depth q0_att'])]
        # dmin=df['Min Depth q0_att'][~np.isnan(df['Min Depth q0_att'])]
        # print('dmax',np.median(dmax),np.mean(dmax),np.std(dmax))
        # print('dmin',np.median(dmin),np.mean(dmin),np.std(dmin))
        # print('vol',np.median(df['Volume']),np.mean(df['Volume']),np.std(df['Volume']))
        # print('HU',np.median(df['HU Median'][~np.isnan(df['HU Median'])]),np.mean(df['HU Median']),np.std(df['HU Median']))
        if plot:  
            if 'Lung' not in TVType: 
                if plotSiteDetails:
                    plotall(df,np.arange(4,15.1,0.5),np.round(np.arange(1,7,1)**3).astype(int),['Max Depth q0_attLung','Max Depth q0_att','Max Depth q0'],'Volume',perPlan,TVType,'All Sites',f'{plot}/AllSites',show=show)
                    for i,dfci in enumerate(dfs):
                        plotall(dfci,np.arange(4,15.1,0.5),np.round(np.arange(1,7,1)**3).astype(int),['Max Depth q0_attLung','Max Depth q0_att','Max Depth q0'],'Volume',perPlan,TVType,cats[i],f'{plot}/{cats[i]}',show=False)
                plotavailabledata(df,dfs,cats,plot,f"{TVType}{' Plans' if perPlan else ''} per Site",width,show=show)
                # for depth in [4,8,10,12,15]:
                #     plottreatable(df,dfs,cats,'Volume Max Depth q0_attLung',plot,perPlan,TVType,width,depth=depth,show=show)
                #     plottreatable(df,dfs,cats,'Max Depth q0_attLung',plot,perPlan,TVType,width,depth=depth,show=show)
                #     plottreatable(df,dfs,cats,'Volume Max Depth q0',plot,perPlan,TVType,width,depth=depth,show=show)
                #     plottreatable(df,dfs,cats,'Max Depth q0',plot,perPlan,TVType,width,depth=depth,show=show)
                
                for vol in np.arange(1,7,1)**3:
                    plottreatable(df,dfs,cats,'Volume',plot,perPlan,TVType,width,vol=vol,show=show)
                dfsCT=[dfCT[[np.any([s in p for s in sitesCT[c]]) for p in dfCT["Rx site"]]] for c,_ in enumerate(catsCT)]
                plotavailabledata(dfCT,dfsCT,catsCT,plot,f"{TVType}{' Plans' if perPlan else ''} with CT per Site",width,show=show)
                # for depth in [4,8,10,12,15]:
                #     plottreatable(dfCT,dfsCT,catsCT,'Volume Max Depth q0_att',plot,perPlan,TVType,width,depth=depth,show=show)
                #     plottreatable(dfCT,dfsCT,catsCT,'Max Depth q0_att',plot,perPlan,TVType,width,depth=depth,show=show)
                
                # try:
                #     plottreatable(df,dfs,cats,'PTV Volume Verifier',plot,perPlan,TVType,width)
                #     plotbardfcat(dfs,cats,'PTV Volume Verifier',plot,'Mean',width)
                # except:
                #     print("Volume Verifier not found.")

                if not "Pre" in TVType and not perPlan:
                    plotbardfcat(dfs,cats,'Mean Dose/Prescribed Dose',plot,perPlan,TVType,width,show)
                    plothistdfs(dfs,cats,'Mean Dose/Prescribed Dose',plot,show=show,pP=perPlan,TVType=TVType)
                    plotbardfcat(dfs,cats,"Mean dose (cGy)",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Min dose (cGy)",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Max dose (cGy)",plot,perPlan,TVType,width,show)
                
                if TVType=='PTV' and perPlan:
                    plotbardfcat(dfs,cats,'Total Meterset/Dose per Fraction [MU/cGy]',plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,'Total Meterset [MU]',plot,perPlan,TVType,width,show)
                    # plotbardfcat(dfs,cats,'Dose/Frc/Total Meterset [cGy/MU]',plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Num Fractions",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Dose/Frc (cGy)",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Prescribed Dose (cGy)",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"SSD [cm]",plot,perPlan,TVType,width,show,np.mean)      
                    plotbardfcat(dfs,cats,"Dose Rate Set [MU/min]",plot,perPlan,TVType,width,show,np.mean)
                    plotbardfcat(dfs,cats,"Dose [Gy]",plot,perPlan,TVType,width,show,np.sum)
                    plotbardfcat(dfs,cats,"Beam Delivery Duration Limit [s]",plot,perPlan,TVType,width,show,np.sum)
                    plotBeamType(dfs,cats,plot,width,show)
                    plotSRS(dfs,cats,plot,width,show)
                    plotbardfcat(dfs,cats,"Conformity Index 1",plot,perPlan,TVType,width,show,log=True)
                    plotbardfcat(dfs,cats,"Calc Conformity Index 1",plot,perPlan,TVType,width,show,log=True)
                    plotbardfcat(dfs,cats,"Conformity Index 2",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Calc Conformity Index 2",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Conformal Number",plot,perPlan,TVType,width,show)
                    plotbardfcat(dfs,cats,"Calc Conformal Number",plot,perPlan,TVType,width,show)
                    plothistdfs(dfs,cats,"Calc Conformal Number",plot,show=show,pP=perPlan,TVType=TVType)
                    plotbardfcat(dfs,cats,"Dose Balance Index",plot,perPlan,TVType,width,show)
                    plothistdfs(dfs,cats,"Dose Balance Index",plot,show=show,pP=perPlan,TVType=TVType)
                plotbardfcat(dfs,cats,"Max Depth q0",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Max Depth q0_attLung",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Max Depth q0_att",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Min Depth q0",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Min Depth q0_attLung",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Min Depth q0_att",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Dif. of Max Depth q0_attLung - q0_att",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Dif. of Max Depth q0 - q0_att",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Dif. of Max Depth q0 - q0_attLung",plot,perPlan,TVType,width,show)
                # plotbardfcat(dfs,cats,"PTV volume (cc)",plot,'Median')
                plotbardfcat(dfs,cats,"Volume",plot,perPlan,TVType,width,show)
                plothistdfs(dfs,cats,"Volume",plot,show=show,pP=perPlan,TVType=TVType)
                plothistdfs(dfs,cats,"Max Depth q0_att",plot,show=show,pP=perPlan,TVType=TVType)
                plotbardfcat(dfs,cats,"Compactness",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Solidity",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Skew",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Length",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"LengthXYZ",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,"Effective Size",plot,perPlan,TVType,width,show)
                # plotbardfcat(dfs,cats,"Std",plot,perPlan,TVType,width,show)
                if not perPlan: plotbardfcat(dfs,cats,"Count TV",plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,'Homogeneity Index 1',plot,perPlan,TVType,width,show)
                plotbardfcat(dfs,cats,'Lesion Underdosage Factor',plot,perPlan,TVType,width,show)
                plothistdfs(dfs,cats,'Lesion Underdosage Factor',plot,show=show,pP=perPlan,TVType=TVType)
                plotbardfcat(dfs,cats,'Calc Lesion Underdosage Factor',plot,perPlan,TVType,width,show)
            plotbardfcat(dfs,cats,'HU Mean',plot,perPlan,TVType,width,show)
            plotbardfcat(dfs,cats,'HU Median',plot,perPlan,TVType,width,show)   
            plothistdfs(dfs,cats,"HU Median",plot,show=show,pP=perPlan,TVType=TVType)
            plotbardfcat(dfs,cats,'HU First Quartile',plot,perPlan,TVType,width,show)   
            plotbardfcat(dfs,cats,'HU Third Quartile',plot,perPlan,TVType,width,show)    
    return dfs

## Function Calls

The different function calls which built partly on top of each other to run the analysis. Hover over the functions to see their respective in and outputs.

In [ ]:
#Instead of doing the complete sheet, only do a specified interval. Can be used to split sheet up to work in parallel.
interval=[0,3]
#1: Extract TV point clouds, depth and HU data from Dicom structure and image files. Generates PTV.pickle, Files.pickle
dfPointCloud,dfFolder=batchAnalyzeStructuresCT_sheet(loc_sheet=DICOMLocations,debug=0,interval=interval)
#1.1: Take TV point clouds and calculate shape statistics. Generates PCAMesh.pickle
dfShape = compileTVShapes(dfPointCloud, save=SaveFolder,interval=interval)
#1.2: Take HU data and compile into one dataframe. Generates HU.pickle
dfHU=compileHU(SaveFolder,interval=interval)
#1.3: Take per slice depth data and calculate global max and min for each case, combine all plans into one dataframe. Generates Treatable.pickle
dfDepth=batchDetermineMinMaxDepths(SaveFolder,plot=False,interval=interval)
#2: Extract MU and plan info from DICOM plan files, directly compile into one dataframe. Generates MU.pickle
dfMU=batchAnalyzePlan(SaveFolder,interval=interval)
#3: Extract dose data from DICOM dose and structure files. Returns one dataframe with dose results and one with distances between OAR and TV. Generates Dose.pickle, DistsPTVOAR.pickle
dfDose,dfOARDistance=batchAnalyzeDose(SaveFolder,locsheet=DICOMLocations,debug=False,calc=True,interval=interval)
#4: If you chose intervals, merge them all into one dataframe. Generates all pickles without intervals
mergeIntervalResults()
#5: Merge all different kinds of analysis into one big overarching dataframe. Generates MergedAll.pickle
dfAll=mergeAllResults()
#6: If needed, anonymize dataframe. Generates DataAnonymized.pickle
dfAnon=anonymizeResults(dfAll)
#7: Generate statistics plots according to your wishes. Generates plots
dfPlot=analyzebycategory(dfAll,f"{SaveFolder}/Plots",perPlan=True,fine=True,plotSiteDetails=True,show=False)

If you already have the MergedResults.pickle and only want to analyze it, you can load it using 

df=pd.read_pickle('MergedResults.pickle')